# Imports

In [471]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [472]:
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'

# Main

In [473]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")

In [474]:
start_music_recommender()

# Run

In [ ]:
launch_admin_ui()

# Clearing history

In [ ]:
import pandas as pd

def clear_user_and_history_data(users_csv_path, history_csv_path):
    # Clear users.csv
    users_df = pd.read_csv(users_csv_path)
    empty_users = pd.DataFrame(columns=users_df.columns)
    empty_users.to_csv(users_csv_path, index=False)
    print(f"✅ Cleared all data in {users_csv_path} (kept columns).")

    # Clear listening_history.csv
    history_df = pd.read_csv(history_csv_path)
    empty_history = pd.DataFrame(columns=history_df.columns)
    empty_history.to_csv(history_csv_path, index=False)
    print(f"✅ Cleared all data in {history_csv_path} (kept columns).")


In [ ]:
'''import pandas as pd

# Load the full history
history_csv = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv'
df = pd.read_csv(history_csv)

# Keep only the original columns
original_columns = ['user_id', 'song_id', 'mood_before', 'mood_after','start_date','end_date','is_recommended']
df_cleaned = df[original_columns]

# Overwrite the original file
df_cleaned.to_csv(history_csv, index=False)

print("✅ History CSV cleaned back to original structure.")'''


'import pandas as pd\n\n# Load the full history\nhistory_csv = \'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv\'\ndf = pd.read_csv(history_csv)\n\n# Keep only the original columns\noriginal_columns = [\'user_id\', \'song_id\', \'mood_before\', \'mood_after\',\'start_date\',\'end_date\',\'is_recommended\']\ndf_cleaned = df[original_columns]\n\n# Overwrite the original file\ndf_cleaned.to_csv(history_csv, index=False)\n\nprint("✅ History CSV cleaned back to original structure.")'

In [ ]:
clear_user_and_history_data(f"{data_dir}users.csv",f"{data_dir}listening_history.csv")

✅ Cleared all data in /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/users.csv (kept columns).
✅ Cleared all data in /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv (kept columns).


# Admin

In [ ]:
def launch_admin_ui():
    from IPython.display import display, clear_output, HTML
    import pandas as pd
    import os
    import ipywidgets as widgets

    output = widgets.Output()
    users_table = widgets.Output()
    history_table = widgets.Output()

    def load_users(_=None):
        users_table.clear_output()
        with users_table:
            if os.path.exists(users_csv):
                df = pd.read_csv(users_csv)
                display(df)
            else:
                display(HTML("<i>No users.csv file found</i>"))

    def load_history(_=None):
        history_table.clear_output()
        with history_table:
            if os.path.exists(history_csv):
                df = pd.read_csv(history_csv)
                display(df)
            else:
                display(HTML("<i>No listening_history.csv file found</i>"))

    refresh_users = widgets.Button(description="🔄 Refresh Users")
    refresh_users.on_click(load_users)

    refresh_history = widgets.Button(description="🔄 Refresh History")
    refresh_history.on_click(load_history)

    title = widgets.HTML("<h3>🛠 Admin Panel: View Databases</h3>")

    load_users()
    load_history()

    display(widgets.VBox([
        title,
        widgets.HBox([refresh_users, refresh_history]),
        widgets.HTML("<h4>👥 Users Table:</h4>"),
        users_table,
        widgets.HTML("<h4>🎧 Listening History:</h4>"),
        history_table,
        output
    ]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from datetime import datetime, timedelta
import os

def simulated_log_ui(songs_csv, history_csv):
    songs_df = pd.read_csv(songs_csv)

    user_id = widgets.IntText(description="User ID")
    artist = widgets.Text(description="Artist")
    song = widgets.Text(description="Song")
    mood_before = widgets.Dropdown(options=["happy", "sad", "relaxed", "anger"], description="Mood Before")
    mood_after = widgets.Dropdown(options=["happy", "sad", "relaxed", "anger"], description="Mood After")
    is_recommended = widgets.Checkbox(value=False, description="Recommended?")
    duration = widgets.IntText(description="Duration (s)", value=120)
    submit_btn = widgets.Button(description="➕ Add Entry")
    output = widgets.Output()

    def on_submit_clicked(b):
        with output:
            clear_output()
            match = songs_df[
                (songs_df['artist'].str.lower() == artist.value.strip().lower()) &
                (songs_df['song'].str.lower() == song.value.strip().lower())
            ]
            if match.empty:
                print("❌ No matching song found.")
                return

            song_id = match.iloc[0]['song_id']
            start_time = datetime.now()
            end_time = start_time + timedelta(seconds=duration.value)

            entry = {
                'user_id': user_id.value,
                'song_id': song_id,
                'mood_before': mood_before.value,
                'mood_after': mood_after.value,
                'start_date': start_time,
                'end_date': end_time,
                'is_recommended': int(is_recommended.value)
            }

            if os.path.exists(history_csv):
                hist_df = pd.read_csv(history_csv)
            else:
                hist_df = pd.DataFrame(columns=entry.keys())

            hist_df = pd.concat([hist_df, pd.DataFrame([entry])], ignore_index=True)
            hist_df.to_csv(history_csv, index=False)

            print(f"✅ Logged entry for song ID {song_id} with {duration.value} sec duration.")

    submit_btn.on_click(on_submit_clicked)

    display(widgets.VBox([
        user_id, artist, song,
        mood_before, mood_after,
        is_recommended, duration,
        submit_btn, output
    ]))


In [ ]:
simulated_log_ui(songs_csv, history_csv)

In [ ]:
'''import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Load your songs.csv
df = pd.read_csv(data_dir+'songs.csv')

# Define numeric audio feature columns
ignore_cols = ['song_id', 'artist', 'song', 'title', 'genre', 'emotion']
feature_cols = [c for c in df.columns if c not in ignore_cols and df[c].dtype in [np.float64, np.int64]]

# Fit a scaler
scaler = StandardScaler()
scaler.fit(df[feature_cols].fillna(0))

# Save it
joblib.dump(scaler, 'scaler.joblib')
print("✅ Saved scaler.joblib")'''

'import pandas as pd\nimport numpy as np\nimport joblib\nfrom sklearn.preprocessing import StandardScaler\n\n# Load your songs.csv\ndf = pd.read_csv(data_dir+\'songs.csv\')\n\n# Define numeric audio feature columns\nignore_cols = [\'song_id\', \'artist\', \'song\', \'title\', \'genre\', \'emotion\']\nfeature_cols = [c for c in df.columns if c not in ignore_cols and df[c].dtype in [np.float64, np.int64]]\n\n# Fit a scaler\nscaler = StandardScaler()\nscaler.fit(df[feature_cols].fillna(0))\n\n# Save it\njoblib.dump(scaler, \'scaler.joblib\')\nprint("✅ Saved scaler.joblib")'

In [ ]:
songs_df = pd.read_csv(data_dir+'songs.csv')

In [ ]:
# prompt: lets see how many songs from each emotion

emotion_counts = songs_df['emotion'].value_counts()
print("Number of songs for each emotion:")
emotion_counts

Number of songs for each emotion:


,count
emotion,
joy,91528
sadness,71156
anger,47191
love,12802
fear,11960
surprise,2332


# Drafts

In [ ]:
#13.05.25, 12:39 - probably a stable version, using real recomendation module, dont log favorite artists v.1.6:
#probably nothinh changed from last version but just in case
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")'''

In [ ]:
#11.05.25, 17:38 - probably a stable version, but not using real recomendation module, dont log favorite artists v.1.5:
#recomended artists now fixed to similar to favs+favs
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager  # ✅ import the new class\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.cs

In [ ]:
#11.05.25, 16:55 - probably a stable version, but not using real recomendation module, dont log favorite artists v.1.4:
#fixed favorite artists into a list, conftorble ui, recomended artists updating but only to favorite artists.
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################'''
'''def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")'''

'def start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\n\n    user_manager = UserManager(users_csv)\n    emotion_service = EmotionService()\n    recommender = RecommendationService(songs_csv, history_csv, users_csv)\n    logger = LoggingService(history_csv)\n\n    artist_list = sorted(pd.read_csv(songs_csv)[\'artist\'].dropna().unique().tolist())\n\n    session = {\n        \'user_id\': None,\n        \'username\': \'\',\n        \'mode\': \'none\',\n        \'current_page\': \'start\'\n    }\n\n    output = music_recommender_output\n\n    def show_status(message, status=\'info\'):\n        colors = {\'info\': \'orange\', \'success\': \'green\', \'error\': \'red\'}\n        with output:\n            clear_output()\n            if message:\n                display

In [ ]:
#11.05.25, 15:31 - probably a stable version, but not using real recomendation module, dont log favorite artists v.1.3:
#added favorit artists insertion, editing, and updating recomended artists acordingly
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################


def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_favorites = widgets.Text(description='Fav Artists')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': su_favorites.value,
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value

                # update recommendations
                favorite_list = [a.strip() for a in su_favorites.value.split(',') if a.strip()]
                expanded = recommender.artist_similarity.recommend_from_favorites(favorite_list)
                recommended = [a for a, _ in expanded]
                user_manager.update_recommended_artists(new_user_id, recommended)

                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender, su_favorites,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎧 Edit Favorite Artists</h3>")
        current = ', '.join(user_manager.get_user(session['user_id']).get('favorite_artists', '').split(','))
        fav_input = widgets.Text(value=current, description='New List:')
        save_button = widgets.Button(description='Save')
        cancel_button = widgets.Button(description='Cancel')

        def on_save(_):
            new_list = fav_input.value
            users_df = user_manager.load_users()
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = new_list
            users_df.to_csv(users_csv, index=False)

            new_list_parsed = [a.strip() for a in new_list.split(',') if a.strip()]
            expanded = recommender.artist_similarity.recommend_from_favorites(new_list_parsed)
            recommended = [a for a, _ in expanded]
            user_manager.update_recommended_artists(session['user_id'], recommended)
            show_status("✅ Favorite artists updated.", 'success')
            switch_to_page('session')

        save_button.on_click(on_save)
        cancel_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            fav_input,
            widgets.HBox([save_button, cancel_button]),
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])
            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)
            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''


'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager  # ✅ import the new class\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.cs

In [ ]:
#11.05.25, 15:31 - probably a stable version, but not using real recomendation module, dont log favorite artists v.1.2:
#im not sure if anything changed in that draft so saved just in case
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)
    logger = LoggingService(history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager  # ✅ import the new class\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.cs

In [ ]:
#11.05.25, 13:30 - probably a stable version, but not using real recomendation module, dont log favorite artists v.1.1:
#added in recomendation the option of the user to chose one song out of the recomended songs the recomendation module generates.

'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)
    logger = LoggingService(history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager  # ✅ import the new class\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.cs

In [ ]:
#11.05.25, 13:18 - stable version, but not using real recomendation module, dont log favorite artists v.1.0
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)
    logger = LoggingService(history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'recommend':
                recommended = recommender.recommend_songs(session['user_id'])
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager  # ✅ import the new class\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.cs

In [ ]:
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import random
import importlib
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService



def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'recommend':
                recommended = recommender.recommend_songs(session['user_id'])
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport random\nimport importlib\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\n\n\ndef start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\n\n    user_manager = UserManager(users_csv)\n    emotion_service = EmotionService()\n    recommender = RecommendationService(songs_csv, history_csv)\n\n    session = {\n        \'user_id\': None,\n        \'username\': \'\',\n        \'mode\': \'none\',\n        \'current_page\': \'start\'\n    }\n\n    # === Output area\n    output = music_recommender_output\n\n    def show_status(message, status=\'info\'):\n        colors 

In [ ]:
'''def start_music_recommender():
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import pandas as pd
    import os
    import random

    # Setup managers (reuse your modules)
    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)

    # Session state
    session = {
        'user_id': None,
        'first_name': '',
        'last_name': '',
        'age': 0,
        'gender': '',
        'mode': 'none'
    }

    # Input widgets
    user_id_input = widgets.IntText(description='User ID')
    first_name_input = widgets.Text(description='First Name')
    last_name_input = widgets.Text(description='Last Name')
    age_input = widgets.IntText(description='Age')
    gender_input = widgets.Dropdown(options=['M', 'F'], description='Gender')

    start_button = widgets.Button(description='Start Session')

    # Mode buttons
    mode_buttons = widgets.ToggleButtons(
        options=['none', 'log', 'recommend'],
        description='Mode'
    )

    # Action buttons
    play_button = widgets.Button(description='Play Song')
    sad_button = widgets.Button(description='Feel Sad')
    happy_button = widgets.Button(description='Feel Happy')
    exit_button = widgets.Button(description='Exit')

    output = widgets.Output()

    def start_session(b):
        session['user_id'] = user_id_input.value
        session['first_name'] = first_name_input.value
        session['last_name'] = last_name_input.value
        session['age'] = age_input.value
        session['gender'] = gender_input.value
        user_manager.ensure_user(session['user_id'], session['first_name'], session['last_name'], session['age'], session['gender'])
        with output:
            clear_output()
            print(f"✅ Session started for {session['first_name']} {session['last_name']} in mode {session['mode']}")

    def change_mode(change):
        session['mode'] = change['new']
        with output:
            clear_output()
            print(f"🔄 Mode changed to: {session['mode']}")

    def play_song(b):
        with output:
            clear_output()
            if session['mode'] == 'none':
                print("🔒 Privacy mode → not logging song play.")
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            print(f"🎧 Played: {song['artist']} - {song['song']}")

    def feel_sad(b):
        with output:
            clear_output()
            if session['mode'] == 'recommend':
                recommended = recommender.recommend_songs(session['user_id'])
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                print(f"😢 Sad mood → recommended playlist:\n", recommended[['artist', 'song']])
            else:
                print("⚠ Not in recommend mode — no recommendations made.")

    def feel_happy(b):
        with output:
            clear_output()
            print("😊 Mood noted: happy (no recommendations triggered).")

    def exit_session(b):
        with output:
            clear_output()
            print("👋 Session ended.")

    # Attach handlers
    start_button.on_click(start_session)
    mode_buttons.observe(change_mode, names='value')
    play_button.on_click(play_song)
    sad_button.on_click(feel_sad)
    happy_button.on_click(feel_happy)
    exit_button.on_click(exit_session)

    # Display interface
    display(widgets.VBox([
        widgets.HBox([user_id_input, first_name_input, last_name_input, age_input, gender_input, start_button]),
        mode_buttons,
        widgets.HBox([play_button, sad_button, happy_button, exit_button]),
        output
    ]))'''


'def start_music_recommender():\n    import ipywidgets as widgets\n    from IPython.display import display, clear_output\n    import pandas as pd\n    import os\n    import random\n\n    # Setup managers (reuse your modules)\n    user_manager = UserManager(users_csv)\n    emotion_service = EmotionService()\n    recommender = RecommendationService(songs_csv, history_csv)\n\n    # Session state\n    session = {\n        \'user_id\': None,\n        \'first_name\': \'\',\n        \'last_name\': \'\',\n        \'age\': 0,\n        \'gender\': \'\',\n        \'mode\': \'none\'\n    }\n\n    # Input widgets\n    user_id_input = widgets.IntText(description=\'User ID\')\n    first_name_input = widgets.Text(description=\'First Name\')\n    last_name_input = widgets.Text(description=\'Last Name\')\n    age_input = widgets.IntText(description=\'Age\')\n    gender_input = widgets.Dropdown(options=[\'M\', \'F\'], description=\'Gender\')\n\n    start_button = widgets.Button(description=\'Start Sessio

In [ ]:
#doing something on hrv
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import recommend_for_user
import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class


def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # ✅ Initialize emotion detection service
    emotion_service = EmotionService()

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, si_username, si_password, si_button, back_button, output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_fav_artists = widgets.Text(
            description='Fav Artists', placeholder='e.g., Taylor Swift, Drake, Beyoncé'
        )
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                fav_artists_str = su_fav_artists.value.strip()
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': fav_artists_str
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_fav_artists, su_button, back_button, output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(options=['none', 'log', 'recommend'], description='Mode')
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            hrv_features = {'mean_rr': 900, 'median_rr': 910, 'hr': 72, 'pnn25': 18}
            mood_before = emotion_service.detect(hrv_features)
            mood_after = emotion_service.detect(hrv_features)
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] == 'recommend':
                users_df = pd.read_csv(data_dir + 'users.csv')
                history_df = pd.read_csv(data_dir + 'listening_history.csv')
                songs_df = pd.read_csv(data_dir + 'songs.csv')
                recommended = recommend_for_user(session['user_id'], users_df, history_df, songs_df)
                recommended_song = recommended.sample(1).iloc[0]
                hrv_features = {'mean_rr': 850, 'median_rr': 860, 'hr': 95, 'pnn25': 9}
                mood_before = emotion_service.detect(hrv_features)
                mood_after = emotion_service.detect(hrv_features)
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label, mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button, output
        ]))

    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import recommend_for_user\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService  # ✅ import the new class\n\n\ndef start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\n\n    # ✅ Initialize emotion detection service\n    emotion_service = EmotionService()\n\n    session = {\n        \'user_id\': None,\n        \'username\': \'\',\n        \'mode\': \'none\',\n        \'current_page\': \'start\'\n    }\n\n    output = music_recommender_output\n\n    def show_status(message, status=\'i

In [ ]:
#using RecommendationService attribute from the recomender, not currently having, dont know what else
'''project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import random
import importlib
import sys
sys.path.append(project_dir)
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService
import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv=data_dir+'songs.csv', history_csv=data_dir+'listening_history.csv',users_csv=data_dir+'users.csv')

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        # NEW: Replace multiselect with text box input
        su_fav_artists = widgets.Text(
            description='Fav Artists',
            placeholder='e.g., Taylor Swift, Drake, Beyoncé'
        )

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                fav_artists_str = su_fav_artists.value.strip()  # store as plain text

                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': fav_artists_str
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_fav_artists,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'recommend':
                recommended = recommender.recommend_songs(session['user_id'])
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'project_dir = \'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/\'\ndata_dir = project_dir + \'data/\'\nimport ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport random\nimport importlib\nimport sys\nsys.path.append(project_dir)\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.csv"\n\ndef start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\n\n    user_manager = UserManager(users_csv)\n    emot

In [ ]:
#something fishy about this one
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import recommend_for_user

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, si_username, si_password, si_button, back_button, output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_fav_artists = widgets.Text(
            description='Fav Artists', placeholder='e.g., Taylor Swift, Drake, Beyoncé'
        )
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                fav_artists_str = su_fav_artists.value.strip()
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': fav_artists_str
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_fav_artists, su_button, back_button, output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(options=['none', 'log', 'recommend'], description='Mode')
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] == 'recommend':
                # ✅ Use the new recommend_for_user function
                users_df = pd.read_csv(data_dir + 'users.csv')
                history_df = pd.read_csv(data_dir + 'listening_history.csv')
                songs_df = pd.read_csv(data_dir + 'songs.csv')
                recommended = recommend_for_user(session['user_id'], users_df, history_df, songs_df)
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label, mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button, output
        ]))
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import recommend_for_user

# Ensure the log_listening function and emotion_service are available
# Replace this with your actual import or definition
def log_listening(user_id, song_id, mood_before, mood_after, is_recommended):
    print(f"Logged listening: user={user_id}, song={song_id}, mood_before={mood_before}, mood_after={mood_after}, recommended={is_recommended}")

# Replace this with your actual import or definition
class MockEmotionService:
    def detect(self):
        # Simulate emotion detection
        import random
        return random.choice(['neutral', 'happy', 'sad'])

emotion_service = MockEmotionService()


# Define data file paths (replace with your actual paths)
data_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/'
users_csv = data_dir + 'users.csv'
songs_csv = data_dir + 'songs.csv'
listening_history_csv = data_dir + 'listening_history.csv'


def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, si_username, si_password, si_button, back_button, output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_fav_artists = widgets.Text(
            description='Fav Artists', placeholder='e.g., Taylor Swift, Drake, Beyoncé'
        )
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                fav_artists_str = su_fav_artists.value.strip()
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': fav_artists_str
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label, su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_fav_artists, su_button, back_button, output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(options=['none', 'log', 'recommend'], description='Mode')
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] == 'recommend':
                # ✅ Use the new recommend_for_user function
                users_df = pd.read_csv(data_dir + 'users.csv')
                history_df = pd.read_csv(data_dir + 'listening_history.csv')
                songs_df = pd.read_csv(data_dir + 'songs.csv')
                recommended = recommend_for_user(session['user_id'], users_df, history_df, songs_df)
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label, mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button, output
        ]))

    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")
start_music_recommender()'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import recommend_for_user\n\ndef start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\n\n    session = {\n        \'user_id\': None,\n        \'username\': \'\',\n        \'mode\': \'none\',\n        \'current_page\': \'start\'\n    }\n\n    output = music_recommender_output\n\n    def show_status(message, status=\'info\'):\n        colors = {\'info\': \'orange\', \'success\': \'green\', \'error\': \'red\'}\n        with output:\n       

In [ ]:
#another one that uses old RecommendationService attribute
'''project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import random
import importlib
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService
import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv)

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # === Output area
    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")  # clear status on page change
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    # === Start Page ===
    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    # === Sign In Page ===
    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    # === Sign Up Page ===
    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")  # clear on success
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            su_button,
            back_button,
            output
        ]))

    # === Session Page ===
    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            log_listening(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")  # clear before valid action
            if session['mode'] == 'recommend':
                recommended = recommender.recommend_songs(session['user_id'])
                recommended_song = recommended.sample(1).iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                log_listening(session['user_id'], recommended_song['song_id'], mood_before, mood_after, is_recommended=1)
                show_status(f"😢 Recommended: 🎵 {recommended_song['artist']} - {recommended_song['song']}", 'success')
            else:
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')

        def feel_happy(b):
            show_status("")  # clear before valid action
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")  # clear on logout
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            exit_button,
            output
        ]))

    # Start at the start page
    switch_to_page('start')
    print("DEBUG: Multi-page interface with full error clearing loaded")'''

'project_dir = \'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/\'\ndata_dir = project_dir + \'data/\'\nimport ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport random\nimport importlib\nimport sys\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.csv"\n\ndef start_music_recommender():\n    print("DEBUG: Starting full multi-page recommender with full error clearing")\n\n    global music_recommender_output\n    try:\n        music_recommender_output.close()\n    except:\n        pass\n    music_recommender_output = widgets.Output()\

In [ ]:
#ui design that didnt work that good
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
data_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/'
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with Admin UI support")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    #def app_header():
    #    return widgets.HTML("<h2 style='color:'green';'>🎵 <b>My Music Recommender</b></h2>")
    def app_header(text_color='white', font='Verdana', bg_color='#000000'):
      return widgets.HTML(
          value=f"""
          <div style="
          background-color: {bg_color};
          color: {text_color};
          font-family: {font};
          padding: 15px;
          border-radius: 10px;
          text-align: center;
          box-shadow: 0 2px 6px rgba(0,0,0,0.1);
          ">
          🎵 <b>My Music Recommender</b>
          </div>

          )

    def app_header(color='navy', font='Arial'):
      return widgets.HTML(
          value=f"<h2 style='color:{color}; font-family:{font};'>🎵 <b>My Music Recommender</b></h2>"
          )

    def app_footer():
        return widgets.HTML("<hr><p style='color:gray;font-size:12px;text-align:center;'>Made with 💙 in Colab · Music for your mood</p>")

    def wrap_page(content_widgets, page='default'):
        style = page_styles.get(page, page_styles['default'])
        style_html = f"""
        <style>
            .custom-page {{
                background-color: {style['bg']};
                color: {style['text']};
                font-family: {style['font']};
                padding: 15px;
                border-radius: 12px;
                margin-bottom: 20px;
            }}
            button {{
                font-family: {style['font']};
            }}
        </style>
        """
        return widgets.VBox([
            widgets.HTML(style_html), # Wrap the HTML output in a widgets.HTML widget
            app_header(),
            widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
            app_footer()
        ])

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        style = page_styles.get('start', page_styles['default'])
        start_label = widgets.HTML(
            value=f"<h3 style='color:{style['text']}; font-family:{style['font']};'>🎵 Welcome to the Music Recommender</h3>"
            )
        #start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In', layout=widgets.Layout(margin='5px'))
        signup_button = widgets.Button(description='Sign Up', layout=widgets.Layout(margin='5px'))

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(wrap_page([
            start_label,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], page='start'))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        # Wrap the page content
        display(wrap_page([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ], page='signin')) # Specify page for styling


    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        # Wrap the page content
        display(wrap_page([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ], page='signup')) # Specify page for styling


    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        # Wrap the page content
        display(wrap_page([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ], page='edit_favorites')) # Specify page for styling


    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Wrap the page content
        display(wrap_page([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ], page='session')) # Specify page for styling

    # Start the application on the 'start' page
    switch_to_page('start')
    print("✅ Multi-page interface with updated UI polish loaded.")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\ndata_dir = \'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/\'\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=

# UI DRAFTS

In [406]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Function to switch between pages
def switchToPage(page_name):
    """Handles page transitions and displays the appropriate page."""
    clear_output()  # Clear the previous page
    page_widget = get_page_widget(page_name, style)  # Get the page widget with the proper content
    display(page_widget)  # Display the new page

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        # Welcome page with Sign In / Sign Up buttons
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button onclick="switchToPage('signin')" style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button onclick="switchToPage('signup')" style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        # Sign-in form page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button onclick="switchToPage('session')" style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button onclick="switchToPage('start')" style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        # Sign-up / Create account form
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128274;', title='Create Account')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:6px;">
                    Username: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Password: <input type="password" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    First Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Last Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Age: <input type="number" style="width:80px; padding:4px; margin:3px 0;" value=0>
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Gender:
                    <select style="padding:4px; margin:3px 0; width:100px;">
                        <option value="M">M</option><option value="F">F</option>
                    </select>
                </label>
                <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                <label style="display:block; margin-bottom:6px; font-style:italic;">
                    🎵 Favorite Artists:
                </label>
                <div style="margin-bottom:6px;">
                    Add artist: <input type="text" style="width:60%; padding:4px;">
                    <button onclick="alert('Add artist clicked!')" style="padding:6px 12px; margin-left:4px;">Add</button>
                </div>
                <div style="text-align:center; margin-top:10px;">
                    <button onclick="switchToPage('session')" style="margin:5px 10px; padding:8px 16px;">Submit</button>
                    <button onclick="switchToPage('start')" style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "session":
        # Session page with user info and mode options
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127911;', title='Your Music Dashboard')}
            <div style="padding:10px;">
                <p>Welcome back! Choose a genre or mood to get recommendations.</p>
                <button onclick="alert('Rock clicked!')" style="margin:5px 10px; padding:8px 16px;">Rock</button>
                <button onclick="alert('Jazz clicked!')" style="margin:5px 10px; padding:8px 16px;">Jazz</button>
                <button onclick="alert('Pop clicked!')" style="margin:5px 10px; padding:8px 16px;">Pop</button>
            </div>
        </div>
        {footer_html}
        """
    else:
        # Fallback: unknown page
        page_html = f"<div style='{container_style}'>Unknown page: {page}</div>"

    # Return as an HTML widget
    return widgets.HTML(page_html)


In [458]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import json

# Placeholder - adjust this path to match your system
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')

# Import all necessary modules
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService

# Define styles for HTML UI
style = {
    "background_color": "#f8f9fa",
    "text_color": "#212529",
    "font_family": "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif",
    "header_color": "#6c5ce7",
    "max_width": "600",
    "border_radius": "8",
    "padding": "15",
    "box_shadow": "0 4px 8px rgba(0,0,0,0.1)",
    "footer_message": "© Music Recommender System"
}

# Placeholder paths - adjust these to match your data paths
data_dir = "/content/drive/MyDrive/Colab Notebooks/project/Music/data/"
users_csv = f"{data_dir}users.csv"
songs_csv = f"{data_dir}songs.csv"
history_csv = f"{data_dir}listening_history.csv"

# Main function to start the music recommender
def start_music_recommender():
    print("Starting music recommender with HTML UI...")

    # Create global recommender output to hold all UI
    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # Initialize services
    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    # Load artist list for dropdown
    try:
        artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())
    except:
        artist_list = []
        print("Warning: Could not load artist list")

    # Session state
    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # Create status message display
    status_output = widgets.Output()

    def show_status(message, status='info'):
        colors = {'info': '#e67e22', 'success': '#2ecc71', 'error': '#e74c3c'}
        with status_output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    # Define function to get HTML for a specific page
    def get_page_html(page_name):
        # Common style snippets
        bg = style["background_color"]
        text = style["text_color"]
        font = style["font_family"]
        header_bg = style["header_color"]
        mw = style["max_width"]
        br = style["border_radius"]
        pad = style["padding"]
        shadow = style["box_shadow"]
        footer_text = style["footer_message"]

        # Common elements
        header_html = f"""
        <div style="background-color:{header_bg}; padding:10px;
                    border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                    color:white; font-family:{font}; font-size:20px;">
            {{icon}} <strong>{{title}}</strong>
        </div>"""

        container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                        f"font-family:{font}; border-radius:{br}px; padding:0px; box-shadow:{shadow};")

        footer_html = f"""
        <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                    font-family:{font};">
            {footer_text}
        </div>"""

        # Status bar for showing messages
        status_bar = """
        <div id="status-container" style="padding:5px 15px; min-height:30px;">
        </div>
        """

        # Build page HTML based on the page name
        if page_name == "start":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🎵', title='Welcome to the Music Recommender')}
                {status_bar}
                <div style="padding:15px; text-align:center;">
                    <button id="signin-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign In</button>
                    <button id="signup-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign Up</button>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signin":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔑', title='Sign In')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:8px;">
                        Username: <input type="text" id="si-username" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <label style="display:block; margin-bottom:8px;">
                        Password: <input type="password" id="si-password" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signin-confirm-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Confirm</button>
                        <button id="signin-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signup":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔐', title='Sign Up')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:6px;">
                        Username: <input type="text" id="su-username" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Password: <input type="password" id="su-password" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        First Name: <input type="text" id="su-firstname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Last Name: <input type="text" id="su-lastname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Age: <input type="number" id="su-age" style="width:80px; padding:4px; margin:3px 0;" value="0">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Gender:
                        <select id="su-gender" style="padding:4px; margin:3px 0; width:100px;">
                            <option value="M">M</option><option value="F">F</option>
                        </select>
                    </label>
                    <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                    <label style="display:block; margin-bottom:6px; font-weight:bold;">
                        🎵 Favorite Artists:
                    </label>
                    <div style="margin-bottom:6px;">
                        <input type="text" id="artist-input" list="artist-list" style="width:60%; padding:4px;">
                        <datalist id="artist-list">
                            ${artist_datalist_options}
                        </datalist>
                        <button id="add-artist-btn" style="padding:6px 12px; margin-left:4px; background-color:#27ae60; color:white; border:none; border-radius:4px; cursor:pointer;">Add</button>
                    </div>
                    <div id="selected-artists" style="margin-bottom:10px; max-height:150px; overflow-y:auto; padding:5px; border:1px solid #ddd; border-radius:4px;">
                        <!-- Selected artists will be listed here -->
                    </div>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signup-submit-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Submit</button>
                        <button id="signup-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "session":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🎧', title='Your Music Dashboard')}
                {status_bar}
                <div style="padding:15px;">
                    <div style="margin-bottom:10px;">
                        <strong>👤 Signed in as: </strong><span id="username-display">{session['username']}</span>
                    </div>

                    <div style="margin-bottom:15px;">
                        <strong>Mode: </strong>
                        <select id="mode-selector" style="padding:5px; width:150px;">
                            <option value="none">None (Privacy)</option>
                            <option value="log">Log</option>
                            <option value="recommend">Recommend</option>
                        </select>
                    </div>

                    <div style="margin-bottom:15px;">
                        <button id="play-song-btn" style="margin:5px; padding:8px 12px; background-color:#3498db; color:white; border:none; border-radius:4px; cursor:pointer;">Play Song</button>
                        <button id="feel-sad-btn" style="margin:5px; padding:8px 12px; background-color:#9b59b6; color:white; border:none; border-radius:4px; cursor:pointer;">Feel Sad</button>
                        <button id="feel-happy-btn" style="margin:5px; padding:8px 12px; background-color:#f39c12; color:white; border:none; border-radius:4px; cursor:pointer;">Feel Happy</button>
                    </div>

                    <div style="margin-bottom:15px;">
                        <button id="edit-favorites-btn" style="margin:5px; padding:8px 12px; background-color:#2ecc71; color:white; border:none; border-radius:4px; cursor:pointer;">Edit Favorites</button>
                        <button id="logout-btn" style="margin:5px; padding:8px 12px; background-color:#e74c3c; color:white; border:none; border-radius:4px; cursor:pointer;">Log Out</button>
                    </div>

                    <div id="recommendation-area" style="margin-top:15px;">
                        <!-- Recommendation results will be displayed here -->
                    </div>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "edit_favorites":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🎨', title='Edit Favorite Artists')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:6px; font-weight:bold;">
                        🎵 Favorite Artists:
                    </label>
                    <div style="margin-bottom:6px;">
                        <input type="text" id="artist-input" list="artist-list" style="width:60%; padding:4px;">
                        <datalist id="artist-list">
                            ${artist_datalist_options}
                        </datalist>
                        <button id="add-artist-btn" style="padding:6px 12px; margin-left:4px; background-color:#27ae60; color:white; border:none; border-radius:4px; cursor:pointer;">Add</button>
                    </div>
                    <div id="selected-artists" style="margin-bottom:10px; max-height:150px; overflow-y:auto; padding:5px; border:1px solid #ddd; border-radius:4px;">
                        <!-- Selected artists will be listed here -->
                    </div>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="save-favorites-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Save</button>
                        <button id="favorites-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """
        else:
            return f"<div style='{container_style}'>Unknown page: {page_name}</div>"

    # Create HTML UI buttons that trigger the actual Python widgets
    def create_hidden_widgets():
        # Sign in page widgets
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')

        # Sign up page widgets
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        # Session page widgets
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')

        # Create dictionary to store selected artists for signup
        selected_artists_dict = {'artists': []}

        # ========== Event handlers for sign in page ==========
        def sign_in_action():
            username = document.getElementById('si-username').value
            password = document.getElementById('si-password').value

            if username.strip() == "" or password.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == username]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != password:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = username
                show_status("")
                switch_to_page('session')

        # ========== Event handlers for sign up page ==========
        def sign_up_action():
            username = document.getElementById('su-username').value
            password = document.getElementById('su-password').value
            firstname = document.getElementById('su-firstname').value
            lastname = document.getElementById('su-lastname').value
            age = document.getElementById('su-age').value
            gender = document.getElementById('su-gender').value

            if username.strip() == "" or password.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if username in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': username,
                    'password': password,
                    'first_name': firstname,
                    'last_name': lastname,
                    'age': int(age),
                    'gender': gender,
                    'favorite_artists': ', '.join(selected_artists_dict['artists']),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists_dict['artists'])
                session['user_id'] = new_user_id
                session['username'] = username
                show_status("")
                switch_to_page('session')

        # ========== Event handlers for session page ==========
        def change_mode():
            mode = document.getElementById('mode-selector').value
            session['mode'] = mode
            show_status(f"🔄 Mode changed to: {mode}", 'info')

        def play_song():
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad():
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            # Create HTML for song selection dropdown
            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            select_html = f"""
            <div style="margin-top:15px; padding:10px; border:1px solid #ddd; border-radius:4px; background-color:#f9f9f9;">
                <p><strong>Recommended songs to improve your mood:</strong></p>
                <select id="song-selector" style="width:100%; padding:8px; margin:5px 0;">
                    {"".join([f'<option value="{i}">{opt}</option>' for i, opt in enumerate(options)])}
                </select>
                <button id="confirm-song-btn" style="padding:6px 12px; margin-top:8px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Confirm Choice</button>
            </div>
            """

            # Display the recommendation options
            display(HTML(f"""
            <script>
                document.getElementById('recommendation-area').innerHTML = `{select_html}`;

                document.getElementById('confirm-song-btn').onclick = function() {{
                    var selectElement = document.getElementById('song-selector');
                    var selectedIndex = selectElement.value;
                    var selectedLabel = selectElement.options[selectElement.selectedIndex].text;

                    // Send the selection to Python
                    google.colab.kernel.invokeFunction('confirm_song_selection', [selectedIndex, selectedLabel], {{}});
                }}
            </script>
            """))

            # Register callback to handle song selection
            def on_confirm_song(selected_index, selected_label):
                selected_index = int(selected_index)
                selected_row = recommended.iloc[selected_index]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

                # Clear recommendation area
                display(HTML("""
                <script>
                    document.getElementById('recommendation-area').innerHTML = '';
                </script>
                """))

            # Register the Python callback function
            output.register_callback('confirm_song_selection', on_confirm_song)

        def feel_happy():
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        # ========== Event handlers for Edit Favorites page ==========
        def save_favorites():
            users_df = pd.read_csv(users_csv)
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists_dict['artists'])
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists_dict['artists'])
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        # Function to add artist to selection
        def add_artist(artist_name=None):
            if artist_name is None:
                artist_input = document.getElementById('artist-input')
                artist_name = artist_input.value.strip()

            if artist_name and artist_name in artist_list and artist_name not in selected_artists_dict['artists']:
                selected_artists_dict['artists'].append(artist_name)
                refresh_artist_list()
                # Clear input field
                display(HTML("""
                <script>
                    document.getElementById('artist-input').value = '';
                </script>
                """))

        # Function to remove artist from selection
        def remove_artist(artist_name):
            if artist_name in selected_artists_dict['artists']:
                selected_artists_dict['artists'].remove(artist_name)
                refresh_artist_list()

        # Refresh the visual list of selected artists
        def refresh_artist_list():
            artists_html = ""
            for artist in selected_artists_dict['artists']:
                artists_html += f"""
                <div style="display:flex; justify-content:space-between; margin:2px 0; padding:4px; background-color:#f1f1f1; border-radius:3px;">
                    <span>{artist}</span>
                    <button
                        onclick="google.colab.kernel.invokeFunction('remove_artist', ['{artist}'], {{}})"
                        style="background:none; border:none; color:#e74c3c; cursor:pointer; font-weight:bold;"
                    >✕</button>
                </div>
                """

            display(HTML(f"""
            <script>
                document.getElementById('selected-artists').innerHTML = `{artists_html}`;
            </script>
            """))

        # Register Python callbacks for artist management
        output.register_callback('add_artist', add_artist)
        output.register_callback('remove_artist', remove_artist)

        # Function to load favorite artists for edit page
        def load_favorite_artists():
            selected_artists_dict['artists'] = []

            try:
                users_df = pd.read_csv(users_csv)
                user_row = users_df[users_df['user_id'] == session['user_id']]

                if not user_row.empty:
                    fav_str = user_row.iloc[0].get('favorite_artists', '')
                    if not pd.isna(fav_str):
                        selected_artists_dict['artists'] = [a.strip() for a in fav_str.split(',') if a.strip()]

                refresh_artist_list()
            except Exception as e:
                print(f"Error loading favorite artists: {e}")

        # Function to handle logging out
        def exit_session():
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        # Register all callbacks to make them accessible from JavaScript
        output.register_callback('sign_in_action', sign_in_action)
        output.register_callback('sign_up_action', sign_up_action)
        output.register_callback('change_mode', change_mode)
        output.register_callback('play_song', play_song)
        output.register_callback('feel_sad', feel_sad)
        output.register_callback('feel_happy', feel_happy)
        output.register_callback('save_favorites', save_favorites)
        output.register_callback('exit_session', exit_session)
        output.register_callback('load_favorite_artists', load_favorite_artists)

        return {
            'si_username': si_username,
            'si_password': si_password,
            'si_button': si_button,
            'su_username': su_username,
            'su_password': su_password,
            'su_firstname': su_firstname,
            'su_lastname': su_lastname,
            'su_age': su_age,
            'su_gender': su_gender,
            'mode_selector': mode_selector,
            'play_button': play_button,
            'sad_button': sad_button,
            'happy_button': happy_button
        }

    # Function to switch between pages
    def switch_to_page(page_name):
        session['current_page'] = page_name

        # Create datalist options for artists
        global artist_datalist_options
        artist_datalist_options = "".join([f'<option value="{artist}">' for artist in artist_list])

        # Display the HTML page
        html_content = get_page_html(page_name)
        with music_recommender_output:
            clear_output()
            display(HTML(html_content))

        # Add JavaScript to handle button clicks
        if page_name == "start":
            display(HTML("""
            <script>
                document.getElementById('signin-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signin'], {});
                }
                document.getElementById('signup-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signup'], {});
                }
            </script>
            """))

        elif page_name == "signin":
            display(HTML("""
            <script>
                document.getElementById('signin-confirm-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_in_action', [], {});
                }
                document.getElementById('signin-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
            </script>
            """))

        elif page_name == "signup":
            display(HTML("""
            <script>
                document.getElementById('signup-submit-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_up_action', [], {});
                }
                document.getElementById('signup-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
            </script>
            """))

        elif page_name == "session":
            display(HTML("""
            <script>
                document.getElementById('mode-selector').onchange = function() {
                    google.colab.kernel.invokeFunction('change_mode', [], {});
                }
                document.getElementById('play-song-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('play_song', [], {});
                }
                document.getElementById('feel-sad-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_sad', [], {});
                }
                document.getElementById('feel-happy-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_happy', [], {});
                }
                document.getElementById('edit-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['edit_favorites'], {});
                }
                document.getElementById('logout-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('exit_session', [], {});
                }

                // Update username display
                document.getElementById('username-display').textContent = '${session["username"]}';
            </script>
            """))

        elif page_name == "edit_favorites":
            # First load the user's favorite artists
            output.register_callback('load_favorite_artists', load_favorite_artists)
            display(HTML("""
            <script>
                google.colab.kernel.invokeFunction('load_favorite_artists', [], {});

                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
                document.getElementById('save-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('save_favorites', [], {});
                }
                document.getElementById('favorites-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['session'], {});
                }
            </script>
            """))

    # Create output widget for displaying messages
    output = widgets.Output()
    display(output)

    # Register our main callback to switch pages
    output.register_callback('switch_page', switch_to_page)

    # Create hidden widgets
    widgets_dict = create_hidden_widgets()

    # Set up status messaging
    def update_status_message(message, status_type='info'):
        colors = {'info': '#e67e22', 'success': '#2ecc71', 'error': '#e74c3c'}
        color = colors.get(status_type, colors['info'])

        display(HTML(f"""
        <script>
            var statusContainer = document.getElementById('status-container');
            if (statusContainer) {{
                statusContainer.innerHTML = '<span style="color:{color}; font-weight:bold">{message}</span>';
            }}
        </script>
        """))

    # Override show_status to use our HTML UI
    def show_status(message, status='info'):
        update_status_message(message, status)

    # Start the application
    switch_to_page('start')
    print("✅ Music recommender with HTML UI loaded successfully.")

# Function to actually launch the application
def launch_music_recommender():
    # This is the entry point for Colab notebook
    # Make sure you've set up the correct paths for data files

    # Ensure data files exist - if not, create sample data
    check_and_create_sample_data()

    # Start the music recommender application
    start_music_recommender()

# Helper function to check if data files exist and create them if they don't
def check_and_create_sample_data():
    try:
        # Check if data directory exists
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
            print(f"Created data directory: {data_dir}")

        # Check if users.csv exists
        if not os.path.exists(users_csv):
            # Create sample users data
            users_df = pd.DataFrame({
                'user_id': [1, 2],
                'username': ['user1', 'user2'],
                'password': ['pass1', 'pass2'],
                'first_name': ['John', 'Jane'],
                'last_name': ['Doe', 'Smith'],
                'age': [25, 30],
                'gender': ['M', 'F'],
                'favorite_artists': ['Taylor Swift, Ed Sheeran', 'Beyoncé, Drake'],
                'recommended_artists': ['', '']
            })
            users_df.to_csv(users_csv, index=False)
            print(f"Created sample users data: {users_csv}")

        # Check if songs.csv exists
        if not os.path.exists(songs_csv):
            # Create sample songs data
            songs_df = pd.DataFrame({
                'song_id': range(1, 11),
                'song': [
                    'Shape of You', 'Blank Space', 'Halo', 'God\'s Plan',
                    'Someone Like You', 'Uptown Funk', 'Blinding Lights',
                    'Bad Guy', 'Shallow', 'Happy'
                ],
                'artist': [
                    'Ed Sheeran', 'Taylor Swift', 'Beyoncé', 'Drake',
                    'Adele', 'Bruno Mars', 'The Weeknd',
                    'Billie Eilish', 'Lady Gaga', 'Pharrell Williams'
                ],
                'genre': [
                    'Pop', 'Pop', 'R&B', 'Hip-hop',
                    'Pop', 'Funk', 'Synth-pop',
                    'Electropop', 'Pop', 'Soul'
                ]
            })
            songs_df.to_csv(songs_csv, index=False)
            print(f"Created sample songs data: {songs_csv}")

        # Check if listening_history.csv exists
        if not os.path.exists(history_csv):
            # Create sample listening history data
            history_df = pd.DataFrame({
                'history_id': range(1, 5),
                'user_id': [1, 1, 2, 2],
                'song_id': [1, 3, 2, 4],
                'timestamp': pd.date_range(start='2023-01-01', periods=4),
                'mood_before': ['neutral', 'sad', 'happy', 'neutral'],
                'mood_after': ['happy', 'neutral', 'happy', 'happy'],
                'is_recommended': [0, 0, 1, 1]
            })
            history_df.to_csv(history_csv, index=False)
            print(f"Created sample listening history data: {history_csv}")

    except Exception as e:
        print(f"Error creating sample data: {e}")

# This code would normally be executed when the module is run
if __name__ == "__main__":
    launch_music_recommender()

Starting music recommender with HTML UI...
✅ Health-based Dummy EmotionService initialized


Output()

AttributeError: 'Output' object has no attribute 'register_callback'

In [461]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

# Define paths for data files
data_dir = "/content/drive/MyDrive/Colab Notebooks/project/Music/data/"
users_csv = f"{data_dir}users.csv"
songs_csv = f"{data_dir}songs.csv"
history_csv = f"{data_dir}listening_history.csv"

# Placeholder paths - adjust these to match your data paths
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')

# Import all necessary modules
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService

# Define styles for HTML UI
style = {
    "background_color": "#f8f9fa",
    "text_color": "#212529",
    "font_family": "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif",
    "header_color": "#6c5ce7",
    "max_width": "600",
    "border_radius": "8",
    "padding": "15",
    "box_shadow": "0 4px 8px rgba(0,0,0,0.1)",
    "footer_message": "© Music Recommender System"
}

# Function to start the music recommender
def start_music_recommender():
    print("Starting music recommender with HTML UI...")

    # Initialize output widget
    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # Initialize services
    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    # Load artist list for dropdown
    try:
        artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())
    except:
        artist_list = []
        print("Warning: Could not load artist list")

    # Session state
    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # Create status message display
    status_output = widgets.Output()

    def show_status(message, status='info'):
        colors = {'info': '#e67e22', 'success': '#2ecc71', 'error': '#e74c3c'}
        with status_output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    # Define function to get HTML for a specific page
    def get_page_html(page_name):
        # Common style snippets
        bg = style["background_color"]
        text = style["text_color"]
        font = style["font_family"]
        header_bg = style["header_color"]
        mw = style["max_width"]
        br = style["border_radius"]
        pad = style["padding"]
        shadow = style["box_shadow"]
        footer_text = style["footer_message"]

        # Common elements
        header_html = f"""
        <div style="background-color:{header_bg}; padding:10px;
                    border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                    color:white; font-family:{font}; font-size:20px;">
            {{icon}} <strong>{{title}}</strong>
        </div>"""

        container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                        f"font-family:{font}; border-radius:{br}px; padding:0px; box-shadow:{shadow};")

        footer_html = f"""
        <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                    font-family:{font};">
            {footer_text}
        </div>"""

        # Status bar for showing messages
        status_bar = """
        <div id="status-container" style="padding:5px 15px; min-height:30px;">
        </div>
        """

        # Build page HTML based on the page name
        if page_name == "start":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🎵', title='Welcome to the Music Recommender')}
                {status_bar}
                <div style="padding:15px; text-align:center;">
                    <button id="signin-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign In</button>
                    <button id="signup-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign Up</button>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signin":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔑', title='Sign In')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:8px;">
                        Username: <input type="text" id="si-username" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <label style="display:block; margin-bottom:8px;">
                        Password: <input type="password" id="si-password" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signin-confirm-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Confirm</button>
                        <button id="signin-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signup":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔐', title='Sign Up')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:6px;">
                        Username: <input type="text" id="su-username" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Password: <input type="password" id="su-password" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        First Name: <input type="text" id="su-firstname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Last Name: <input type="text" id="su-lastname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Age: <input type="number" id="su-age" style="width:80px; padding:4px; margin:3px 0;" value="0">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Gender:
                        <select id="su-gender" style="padding:4px; margin:3px 0; width:100px;">
                            <option value="M">M</option><option value="F">F</option>
                        </select>
                    </label>
                    <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                    <label style="display:block; margin-bottom:6px; font-weight:bold;">
                        🎵 Favorite Artists:
                    </label>
                    <div style="margin-bottom:6px;">
                        <input type="text" id="artist-input" list="artist-list" style="width:60%; padding:4px;">
                        <datalist id="artist-list">
                            ${artist_datalist_options}
                        </datalist>
                        <button id="add-artist-btn" style="padding:6px 12px; margin-left:4px; background-color:#27ae60; color:white; border:none; border-radius:4px; cursor:pointer;">Add</button>
                    </div>
                    <div id="selected-artists" style="margin-bottom:10px; max-height:150px; overflow-y:auto; padding:5px; border:1px solid #ddd; border-radius:4px;">
                        <!-- Selected artists will be listed here -->
                    </div>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signup-submit-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Submit</button>
                        <button id="signup-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        else:
            return f"<div style='{container_style}'>Unknown page: {page_name}</div>"

    # Create HTML UI buttons that trigger the actual Python widgets
    def create_hidden_widgets():
        # Sign in page widgets
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')

        # Sign up page widgets
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        # Session page widgets
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')

        # Create dictionary to store selected artists for signup
        selected_artists_dict = {'artists': []}

        return {
            'si_username': si_username,
            'si_password': si_password,
            'si_button': si_button,
            'su_username': su_username,
            'su_password': su_password,
            'su_firstname': su_firstname,
            'su_lastname': su_lastname,
            'su_age': su_age,
            'su_gender': su_gender,
            'mode_selector': mode_selector,
            'play_button': play_button,
            'sad_button': sad_button,
            'happy_button': happy_button
        }

    # Function to switch between pages
    def switch_to_page(page_name):
        session['current_page'] = page_name

        # Create datalist options for artists
        global artist_datalist_options
        artist_datalist_options = "".join([f'<option value="{artist}">' for artist in artist_list])

        # Display the HTML page
        html_content = get_page_html(page_name)
        with music_recommender_output:
            clear_output()
            display(HTML(html_content))

        # Add JavaScript to handle button clicks
        if page_name == "start":
            display(HTML("""
            <script>
                document.getElementById('signin-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signin'], {});
                }
                document.getElementById('signup-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signup'], {});
                }
            </script>
            """))

        elif page_name == "signin":
            display(HTML("""
            <script>
                document.getElementById('signin-confirm-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_in_action', [], {});
                }
                document.getElementById('signin-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
            </script>
            """))

        elif page_name == "signup":
            display(HTML("""
            <script>
                document.getElementById('signup-submit-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_up_action', [], {});
                }
                document.getElementById('signup-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
            </script>
            """))

        elif page_name == "session":
            display(HTML("""
            <script>
                document.getElementById('mode-selector').onchange = function() {
                    google.colab.kernel.invokeFunction('change_mode', [], {});
                }
                document.getElementById('play-song-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('play_song', [], {});
                }
                document.getElementById('feel-sad-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_sad', [], {});
                }
                document.getElementById('feel-happy-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_happy', [], {});
                }
                document.getElementById('edit-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['edit_favorites'], {});
                }
                document.getElementById('logout-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('exit_session', [], {});
                }

                // Update username display
                document.getElementById('username-display').textContent = '${session["username"]}';
            </script>
            """))

        elif page_name == "edit_favorites":
            # First load the user's favorite artists
            output.register_callback('load_favorite_artists', load_favorite_artists)
            display(HTML("""
            <script>
                google.colab.kernel.invokeFunction('load_favorite_artists', [], {});

                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
                document.getElementById('save-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('save_favorites', [], {});
                }
                document.getElementById('favorites-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['session'], {});
                }
            </script>
            """))

    # Create output widget for displaying messages
    output = widgets.Output()
    display(output)

    # Register our main callback to switch pages
    output.register_callback('switch_page', switch_to_page)

    # Create hidden widgets
    widgets_dict = create_hidden_widgets()

    # Start the application
    switch_to_page('start')
    print("✅ Music recommender with HTML UI loaded successfully.")


In [465]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

# Define paths for data files
data_dir = "/content/drive/MyDrive/Colab Notebooks/project/Music/data/"
users_csv = f"{data_dir}users.csv"
songs_csv = f"{data_dir}songs.csv"
history_csv = f"{data_dir}listening_history.csv"

# Placeholder paths - adjust these to match your data paths
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')

# Import all necessary modules
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService

# Define styles for HTML UI
style = {
    "background_color": "#f8f9fa",
    "text_color": "#212529",
    "font_family": "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif",
    "header_color": "#6c5ce7",
    "max_width": "600",
    "border_radius": "8",
    "padding": "15",
    "box_shadow": "0 4px 8px rgba(0,0,0,0.1)",
    "footer_message": "© Music Recommender System"
}

# Function to start the music recommender
def start_music_recommender():
    print("Starting music recommender with HTML UI...")

    # Initialize output widget
    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # Initialize services
    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    # Load artist list for dropdown
    try:
        artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())
    except:
        artist_list = []
        print("Warning: Could not load artist list")

    # Session state
    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    # Create status message display
    status_output = widgets.Output()

    def show_status(message, status='info'):
        colors = {'info': '#e67e22', 'success': '#2ecc71', 'error': '#e74c3c'}
        with status_output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    # Define function to get HTML for a specific page
    def get_page_html(page_name):
        # Common style snippets
        bg = style["background_color"]
        text = style["text_color"]
        font = style["font_family"]
        header_bg = style["header_color"]
        mw = style["max_width"]
        br = style["border_radius"]
        pad = style["padding"]
        shadow = style["box_shadow"]
        footer_text = style["footer_message"]

        # Common elements
        header_html = f"""
        <div style="background-color:{header_bg}; padding:10px;
                    border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                    color:white; font-family:{font}; font-size:20px;">
            {{icon}} <strong>{{title}}</strong>
        </div>"""

        container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                        f"font-family:{font}; border-radius:{br}px; padding:0px; box-shadow:{shadow};")

        footer_html = f"""
        <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                    font-family:{font};">
            {footer_text}
        </div>"""

        # Status bar for showing messages
        status_bar = """
        <div id="status-container" style="padding:5px 15px; min-height:30px;">
        </div>
        """

        # Build page HTML based on the page name
        if page_name == "start":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🎵', title='Welcome to the Music Recommender')}
                {status_bar}
                <div style="padding:15px; text-align:center;">
                    <button id="signin-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign In</button>
                    <button id="signup-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Sign Up</button>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signin":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔑', title='Sign In')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:8px;">
                        Username: <input type="text" id="si-username" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <label style="display:block; margin-bottom:8px;">
                        Password: <input type="password" id="si-password" style="width:60%; padding:4px; margin:4px 0;">
                    </label>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signin-confirm-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Confirm</button>
                        <button id="signin-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        elif page_name == "signup":
            return f"""
            <div style="{container_style}" id="music-recommender-container">
                {header_html.format(icon='🔐', title='Sign Up')}
                {status_bar}
                <div style="padding:15px;">
                    <label style="display:block; margin-bottom:6px;">
                        Username: <input type="text" id="su-username" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Password: <input type="password" id="su-password" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        First Name: <input type="text" id="su-firstname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Last Name: <input type="text" id="su-lastname" style="width:80%; padding:4px; margin:3px 0;">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Age: <input type="number" id="su-age" style="width:80px; padding:4px; margin:3px 0;" value="0">
                    </label>
                    <label style="display:block; margin-bottom:6px;">
                        Gender:
                        <select id="su-gender" style="padding:4px; margin:3px 0; width:100px;">
                            <option value="M">M</option><option value="F">F</option>
                        </select>
                    </label>
                    <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                    <label style="display:block; margin-bottom:6px; font-weight:bold;">
                        🎵 Favorite Artists:
                    </label>
                    <div style="margin-bottom:6px;">
                        <input type="text" id="artist-input" list="artist-list" style="width:60%; padding:4px;">
                        <datalist id="artist-list">
                            ${artist_datalist_options}
                        </datalist>
                        <button id="add-artist-btn" style="padding:6px 12px; margin-left:4px; background-color:#27ae60; color:white; border:none; border-radius:4px; cursor:pointer;">Add</button>
                    </div>
                    <div id="selected-artists" style="margin-bottom:10px; max-height:150px; overflow-y:auto; padding:5px; border:1px solid #ddd; border-radius:4px;">
                        <!-- Selected artists will be listed here -->
                    </div>
                    <div style="text-align:center; margin-top:10px;">
                        <button id="signup-submit-btn" style="margin:5px 10px; padding:8px 16px; background-color:#6c5ce7; color:white; border:none; border-radius:4px; cursor:pointer;">Submit</button>
                        <button id="signup-back-btn" style="margin:5px 10px; padding:8px 16px; background-color:#95a5a6; color:white; border:none; border-radius:4px; cursor:pointer;">Back</button>
                    </div>
                </div>
            </div>
            {footer_html}
            """

        else:
            return f"<div style='{container_style}'>Unknown page: {page_name}</div>"

    # Create hidden widgets for page switching
    def create_hidden_widgets():
        # Sign in page widgets
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')

        # Sign up page widgets
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        # Session page widgets
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')

        # Return widgets
        return {
            'si_username': si_username,
            'si_password': si_password,
            'si_button': si_button,
            'su_username': su_username,
            'su_password': su_password,
            'su_firstname': su_firstname,
            'su_lastname': su_lastname,
            'su_age': su_age,
            'su_gender': su_gender,
            'mode_selector': mode_selector,
            'play_button': play_button,
            'sad_button': sad_button,
            'happy_button': happy_button
        }

    # Function to switch between pages
    def switch_to_page(page_name):
        session['current_page'] = page_name
        global artist_datalist_options
        artist_datalist_options = "".join([f'<option value="{artist}">' for artist in artist_list])

        # Display the HTML page
        html_content = get_page_html(page_name)
        with music_recommender_output:
            clear_output()
            display(HTML(html_content))

        # Add JavaScript to handle button clicks for each page
        if page_name == "start":
            display(HTML("""
            <script>
                document.getElementById('signin-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signin'], {});
                }
                document.getElementById('signup-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['signup'], {});
                }
            </script>
            """))
        elif page_name == "signin":
            display(HTML("""
            <script>
                document.getElementById('signin-confirm-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_in_action', [], {});
                }
                document.getElementById('signin-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
            </script>
            """))
        elif page_name == "signup":
            display(HTML("""
            <script>
                document.getElementById('signup-submit-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('sign_up_action', [], {});
                }
                document.getElementById('signup-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['start'], {});
                }
                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
            </script>
            """))
        elif page_name == "session":
            display(HTML("""
            <script>
                document.getElementById('mode-selector').onchange = function() {
                    google.colab.kernel.invokeFunction('change_mode', [], {});
                }
                document.getElementById('play-song-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('play_song', [], {});
                }
                document.getElementById('feel-sad-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_sad', [], {});
                }
                document.getElementById('feel-happy-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('feel_happy', [], {});
                }
                document.getElementById('edit-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['edit_favorites'], {});
                }
                document.getElementById('logout-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('exit_session', [], {});
                }

                // Update username display
                document.getElementById('username-display').textContent = '${session["username"]}';
            </script>
            """))
        elif page_name == "edit_favorites":
            display(HTML("""
            <script>
                google.colab.kernel.invokeFunction('load_favorite_artists', [], {});

                document.getElementById('add-artist-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('add_artist', [], {});
                }
                document.getElementById('save-favorites-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('save_favorites', [], {});
                }
                document.getElementById('favorites-back-btn').onclick = function() {
                    google.colab.kernel.invokeFunction('switch_page', ['session'], {});
                }
            </script>
            """))

    # Initialize the app
    switch_to_page('start')



In [468]:
switch_to_page('start')


In [455]:
import pandas as pd
import numpy as np

class RecommendationService:
    """
    Service for providing song recommendations based on user preferences and listening history.
    """

    def __init__(self, songs_csv, history_csv, users_csv):
        """
        Initialize the recommendation service with paths to data files.

        Args:
            songs_csv (str): Path to the songs database CSV file
            history_csv (str): Path to the listening history CSV file
            users_csv (str): Path to the users database CSV file
        """
        self.songs_csv = songs_csv
        self.history_csv = history_csv
        self.users_csv = users_csv

    def _load_data(self):
        """Load all necessary data for making recommendations."""
        try:
            self.songs_df = pd.read_csv(self.songs_csv)
            self.history_df = pd.read_csv(self.history_csv) if pd.io.common.file_exists(self.history_csv) else pd.DataFrame()
            self.users_df = pd.read_csv(self.users_csv)
            return True
        except Exception as e:
            print(f"Error loading data: {e}")
            return False

    def recommend_songs(self, user_id, n=5):
        """
        Recommend songs for a specific user.

        Args:
            user_id (int): User ID to recommend songs for
            n (int): Number of songs to recommend

        Returns:
            pandas.DataFrame: DataFrame containing recommended songs
        """
        if not self._load_data():
            return pd.DataFrame()  # Return empty DataFrame if data loading fails

        # Get user's favorite artists
        user_row = self.users_df[self.users_df['user_id'] == user_id]
        if user_row.empty:
            return pd.DataFrame()

        # Get user's past listening history
        user_history = self.history_df[self.history_df['user_id'] == user_id] if not self.history_df.empty else pd.DataFrame()

        # Get list of songs the user has already listened to
        listened_songs = set()
        if not user_history.empty:
            listened_songs = set(user_history['song_id'].tolist())

        # Get favorite artists from user data
        favorite_artists = []
        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if not pd.isna(fav_str) and fav_str.strip():
            favorite_artists = [artist.strip() for artist in fav_str.split(',') if artist.strip()]

        # Get recommended artists from user data
        recommended_artists = []
        rec_str = user_row.iloc[0].get('recommended_artists', '')
        if not pd.isna(rec_str) and rec_str.strip():
            recommended_artists = [artist.strip() for artist in rec_str.split(',') if artist.strip()]

        # Strategy 1: Find songs by favorite artists
        favorite_songs = self.songs_df[self.songs_df['artist'].isin(favorite_artists)]

        # Strategy 2: Find songs by recommended artists
        recommended_songs = self.songs_df[self.songs_df['artist'].isin(recommended_artists)]

        # Strategy 3: If we have listening history and mood data, find songs that improved mood
        mood_improvers = pd.DataFrame()
        if not self.history_df.empty and 'mood_before' in self.history_df.columns and 'mood_after' in self.history_df.columns:
            # Find songs that improved mood from sad to better
            improved_history = self.history_df[(self.history_df['mood_before'] == 'sad') &
                                            (self.history_df['mood_after'].isin(['neutral', 'happy']))]

            if not improved_history.empty:
                improved_songs = improved_history['song_id'].unique()
                mood_improvers = self.songs_df[self.songs_df['song_id'].isin(improved_songs)]

        # Combine all recommendation sources with priority
        candidates = pd.concat([
            # First priority: Songs that improved mood
            mood_improvers.assign(priority=1),
            # Second priority: Songs by favorite artists
            favorite_songs.assign(priority=2),
            # Third priority: Songs by recommended artists
            recommended_songs.assign(priority=3),
            # Last resort: Random songs
            self.songs_df.sample(min(10, len(self.songs_df))).assign(priority=4)
        ]).drop_duplicates(subset=['song_id'])

        # Filter out songs the user has already listened to
        if listened_songs:
            candidates = candidates[~candidates['song_id'].isin(listened_songs)]

        # Sort by priority and select top n
        recommended = candidates.sort_values('priority').head(n)

        # If we don't have enough recommendations, add some random songs
        if len(recommended) < n:
            more_songs = self.songs_df[~self.songs_df['song_id'].isin(recommended['song_id'])]
            more_songs = more_songs[~more_songs['song_id'].isin(listened_songs)]
            if not more_songs.empty:
                more_recommendations = more_songs.sample(min(n - len(recommended), len(more_songs)))
                recommended = pd.concat([recommended, more_recommendations])

        # Drop the priority column before returning
        if 'priority' in recommended.columns:
            recommended = recommended.drop('priority', axis=1)

        return recommended.head(n)

In [456]:
import random

class EmotionService:
    """
    Service for detecting and tracking user emotions.

    In a real application, this might use sentiment analysis of user input,
    facial recognition, or sensors to detect emotions. For this simulation,
    we will generate random emotions.
    """

    def __init__(self):
        """Initialize the emotion detection service."""
        self.mood_states = ['sad', 'neutral', 'happy']

    def detect(self):
        """
        Detect the current emotion of the user.

        In this simulation, returns a random mood state.

        Returns:
            str: Current mood state ('sad', 'neutral', or 'happy')
        """
        # Simulate emotion detection - in a real app this would use ML, cameras, etc.
        return random.choice(self.mood_states)

    def analyze_song_impact(self, mood_before, mood_after):
        """
        Analyze how a song impacted the user's mood.

        Args:
            mood_before (str): Mood before listening to the song
            mood_after (str): Mood after listening to the song

        Returns:
            int: Impact score (-1: negative impact, 0: neutral, 1: positive impact)
        """
        # Map mood states to numerical values
        mood_values = {'sad': 0, 'neutral': 1, 'happy': 2}

        # Calculate the mood change
        if mood_before in mood_values and mood_after in mood_values:
            return mood_values[mood_after] - mood_values[mood_before]

        return 0  # Default to no change if mood values are invalid

In [470]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, HBox, VBox

# Enable in Colab (already done above, shown here for completeness)
# from google.colab import output
# output.enable_custom_widget_manager()

# -- Define style/layout constants for consistency --
btn_layout = Layout(width='100px', height='40px', margin='5px')
input_layout = Layout(width='200px', margin='5px')
header_style = {'font_size': '20px', 'font_weight': 'bold', 'text_align': 'center'}

# -- Start Page Widgets --
start_title = widgets.HTML("<h1 style='font-family:Arial; color:#333;'>Welcome!</h1>")
sign_in_btn = Button(description="Sign In", layout=btn_layout)
sign_up_btn = Button(description="Sign Up", layout=btn_layout)
# Example styling: set colors (green for Sign Up, blue for Sign In)
sign_in_btn.style.button_color = '#007bff'   # blue
sign_in_btn.style.text_color = 'white'
sign_up_btn.style.button_color = '#28a745'   # green
sign_up_btn.style.text_color = 'white'
start_page = VBox([start_title, HBox([sign_in_btn, sign_up_btn], layout=Layout(justify_content='center'))],
                  layout=Layout(align_items='center', padding='20px'))

# -- Sign In Page Widgets --
signin_title = widgets.HTML("<h2 style='font-family:Arial; color:#333;'>Sign In</h2>")
username_in = widgets.Text(placeholder='Username', layout=input_layout)
password_in = widgets.Password(placeholder='Password', layout=input_layout)
submit_in = Button(description="Submit", layout=btn_layout, button_style='primary')
back_from_signin = Button(description="Back", layout=btn_layout, button_style='info')
signin_page = VBox([signin_title, username_in, password_in, HBox([submit_in, back_from_signin])],
                   layout=Layout(align_items='center', padding='20px'))

# -- Sign Up Page Widgets --
signup_title = widgets.HTML("<h2 style='font-family:Arial; color:#333;'>Sign Up</h2>")
username_up = widgets.Text(placeholder='Username', layout=input_layout)
password_up = widgets.Password(placeholder='Password', layout=input_layout)
email_up = widgets.Text(placeholder='Email', layout=input_layout)
submit_up = Button(description="Register", layout=btn_layout, button_style='success')
back_from_signup = Button(description="Back", layout=btn_layout, button_style='info')
signup_page = VBox([signup_title, username_up, email_up, password_up, HBox([submit_up, back_from_signup])],
                   layout=Layout(align_items='center', padding='20px'))

# -- Hidden routing buttons (not visible to the user) --
hidden_to_signin = Button(layout=Layout(visibility='hidden'))
hidden_to_signup = Button(layout=Layout(visibility='hidden'))
hidden_to_start = Button(layout=Layout(visibility='hidden'))

# Define page-switching callbacks for hidden buttons
def show_start_page(_=None):
    main_box.children = [start_page]
def show_signin_page(_=None):
    main_box.children = [signin_page]
def show_signup_page(_=None):
    main_box.children = [signup_page]

hidden_to_start.on_click(show_start_page)
hidden_to_signin.on_click(show_signin_page)
hidden_to_signup.on_click(show_signup_page)

# Link visible buttons to hidden triggers
sign_in_btn.on_click(lambda b: hidden_to_signin.click())
sign_up_btn.on_click(lambda b: hidden_to_signup.click())
back_from_signin.on_click(lambda b: hidden_to_start.click())
back_from_signup.on_click(lambda b: hidden_to_start.click())

# -- Display the main container and show the Start page initially --
main_box = VBox()
display(main_box)
show_start_page()


VBox()

In [469]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",      # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        400,              # Max width (px)
    "border_radius":    10,               # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# Function to switch between pages
def switchToPage(page_name):
    """Handles page transitions and displays the appropriate page."""
    clear_output()  # Clear the previous page
    page_widget = get_page_widget(page_name, style)  # Get the page widget with the proper content
    display(page_widget)  # Display the new page

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        # Welcome page with Sign In / Sign Up buttons (buttons are invisible)
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button onclick="alert('Sign In clicked!')" style="margin:5px 10px; padding:8px 16px;" id="signin_button">Sign In</button>
                <button onclick="alert('Sign Up clicked!')" style="margin:5px 10px; padding:8px 16px;" id="signup_button">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
        # Invisible buttons that trigger page switch on click
        signin_button = widgets.Button(description="Sign In", layout=widgets.Layout(visibility='hidden'))
        signup_button = widgets.Button(description="Sign Up", layout=widgets.Layout(visibility='hidden'))

        # Trigger page switches on button clicks without displaying them
        signin_button.on_click(lambda b: switchToPage('signin'))
        signup_button.on_click(lambda b: switchToPage('signup'))

        # Return as an HTML widget with invisible buttons
        return widgets.VBox([widgets.HTML(page_html), signin_button, signup_button])

    elif page == "signin":
        # Sign-in form page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;" onclick="switchToPage('start')">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
        signin_button = widgets.Button(description="Confirm", layout=widgets.Layout(visibility='hidden'))
        back_button = widgets.Button(description="Back", layout=widgets.Layout(visibility='hidden'))

        # Invisible button event handlers
        signin_button.on_click(lambda b: print("User Signed In"))
        back_button.on_click(lambda b: switchToPage('start'))

        return widgets.VBox([widgets.HTML(page_html), signin_button, back_button])

    # Other pages (signup, session) would follow the same pattern...
    # You would define the "signup", "session", etc., similarly

# Initialize the app by calling the start page
switchToPage('start')


In [433]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",      # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        400,              # Max width (px)
    "border_radius":    10,               # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# Function to switch between pages
def switchToPage(page_name):
    """Handles page transitions and displays the appropriate page."""
    clear_output()  # Clear the previous page
    page_widget = get_page_widget(page_name, style)  # Get the page widget with the proper content
    display(page_widget)  # Display the new page

In [451]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",      # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        400,              # Max width (px)
    "border_radius":    10,               # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# Function to switch between pages
def switchToPage(page_name):
    """Handles page transitions and displays the appropriate page."""
    clear_output()  # Clear the previous page
    page_widget = get_page_widget(page_name, style)  # Get the page widget with the proper content
    display(page_widget)  # Display the new page

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = """
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        🎵 <strong>{title}</strong>
    </div>""".format(header_bg=header_bg, br=br, text=text, font=font, title=page)

    # Common container style
    container_style = "max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};".format(mw=mw, bg=bg, text=text, font=font, br=br, pad=pad, shadow=shadow)

    # Footer HTML
    footer_html = """
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>""".format(mw=mw, font=font, footer_text=footer_text)

    # Build each page
    if page == "start":
        # Welcome page with Sign In / Sign Up buttons (buttons are visible)
        page_html = """
        <div style="{container_style}">
            {header_html}
            <div style="padding:10px; text-align:center;">
                <button id="signin_button" style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button id="signup_button" style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """.format(container_style=container_style, header_html=header_html, footer_html=footer_html)

        # Create invisible buttons that trigger page switch on click
        signin_button = widgets.Button(description="Sign In", layout=widgets.Layout(visibility='hidden'))
        signup_button = widgets.Button(description="Sign Up", layout=widgets.Layout(visibility='hidden'))

        # Button event handlers (invisible)
        signin_button.on_click(lambda b: switchToPage('signin'))
        signup_button.on_click(lambda b: switchToPage('signup'))

        # Return as an HTML widget with invisible buttons and HTML interaction
        return widgets.VBox([widgets.HTML(page_html), signin_button, signup_button])

    elif page == "signin":
        # Sign-in form page
        page_html = """
        <div style="{container_style}">
            {header_html}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;" onclick="google.colab.kernel.invokeFunction('notebook.python.switchToPage', ['start'], {});">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """.format(container_style=container_style, header_html=header_html, footer_html=footer_html)

        signin_button = widgets.Button(description="Confirm", layout=widgets.Layout(visibility='hidden'))
        back_button = widgets.Button(description="Back", layout=widgets.Layout(visibility='hidden'))

        # Invisible button event handlers
        signin_button.on_click(lambda b: print("User Signed In"))
        back_button.on_click(lambda b: switchToPage('start'))

        return widgets.VBox([widgets.HTML(page_html), signin_button, back_button])

    # Other pages (signup, session) would follow the same pattern...
    # You would define the "signup", "session", etc., similarly

# Initialize the app by calling the start page
get_page_widget('signin',DEFAULT_STYLE)


IndexError: Replacement index 0 out of range for positional args tuple

In [439]:
# Example of initializing the first page (start page)
switchToPage('signup')

HTML(value='<div style=\'max-width:400px; margin:auto; background-color:#00bcd4; color:#000000; font-family:mo…

In [424]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

# Default style parameters for consistent styling across all pages
DEFAULT_STYLE = {
    "background_color": "#ffffff",  # White background for pages
    "text_color": "#000000",        # Text color black
    "font_family": "Arial",         # Font for header
    "header_color": "#6200ea",      # Purple header
    "max_width": 600,               # Max width for the page container
    "border_radius": 10,            # Rounded corners
    "padding": 40,                  # Padding inside the container
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",  # Shadow for container
    "footer_message": "Made with ❤️ in Colab",  # Footer message
    "button_font_size": "13px",     # Button font size
    "button_text_color": "#ffffff",# Button text color
    "button_hover_color": "#8aff00"  # Button hover color
}

def apply_default_styles(style_dict):
    """Ensure style_dict has all the necessary keys, and fill missing ones with default values."""
    for key, value in DEFAULT_STYLE.items():
        if key not in style_dict:
            style_dict[key] = value
    return style_dict

# Helper function to create reusable components (header, footer, buttons)
def create_header(title, icon, style_dict):
    """Creates a header with title and icon."""
    header_content = widgets.HTML(f"{icon} {title}")
    header_box = widgets.VBox([header_content], layout=widgets.Layout(
        background_color=style_dict['header_color'],
        color=style_dict['text_color'],
        padding='10px',
        font_size='20px',
        font_family=style_dict['font_family'],
        border_top_left_radius=style_dict['border_radius'],
        border_top_right_radius=style_dict['border_radius'],
        text_align='center'
    ))
    return header_box

def create_footer(style_dict):
    """Creates a footer with the provided footer message."""
    footer_content = widgets.HTML(style_dict['footer_message'])
    footer_box = widgets.VBox([footer_content], layout=widgets.Layout(
        padding='10px',
        color=style_dict['text_color'],
        opacity=0.7,
        text_align='center'
    ))
    return footer_box

def create_buttons(buttons, style_dict):
    """Creates buttons (Sign In, Sign Up) with consistent styling."""
    button_widgets = []
    for button in buttons:
        button_widget = widgets.Button(description=button['label'], layout=widgets.Layout(margin="5px 10px"))
        button_widget.style.button_color = style_dict['header_color']
        button_widget.style.font_weight = 'bold'
        button_widget.style.font_size = style_dict['button_font_size']
        button_widgets.append(button_widget)
    return widgets.HBox(button_widgets, layout=widgets.Layout(justify_content='center'))

# Page Functions:
def show_start_page():
    """Show the start page with Sign In and Sign Up buttons."""
    clear_output()
    style_dict = DEFAULT_STYLE

    # Create components (header, footer, buttons)
    header = create_header("Welcome to the Music Recommender", '🎵', style_dict)
    footer = create_footer(style_dict)
    buttons = create_buttons([{'label': 'Sign In'}, {'label': 'Sign Up'}], style_dict)

    # Define actions for buttons
    def on_sign_in(b):
        switch_to_page('signin')

    def on_sign_up(b):
        switch_to_page('signup')

    # Assign button actions
    buttons.children[0].on_click(on_sign_in)  # Sign In button
    buttons.children[1].on_click(on_sign_up)  # Sign Up button

    # Combine components to show the start page
    page_layout = widgets.VBox([header, buttons, footer])
    display(page_layout)

def show_signin_page():
    """Show the sign-in page."""
    clear_output()
    style_dict = DEFAULT_STYLE

    # Create components (header, footer, form)
    header = create_header("Sign In", '🔑', style_dict)
    footer = create_footer(style_dict)

    # Create form widgets
    si_username = widgets.Text(description='Username')
    si_password = widgets.Password(description='Password')
    si_button = widgets.Button(description='Confirm')
    back_button = widgets.Button(description='Back')

    # Define actions for buttons
    def sign_in_action(b):
        print("Signing in...")
        # Add sign-in logic here (e.g., validate, update session)
        switch_to_page('session')

    def back_action(b):
        switch_to_page('start')

    si_button.on_click(sign_in_action)
    back_button.on_click(back_action)

    # Layout the form
    form_layout = widgets.VBox([si_username, si_password, si_button, back_button])

    # Combine components to show the sign-in page
    page_layout = widgets.VBox([header, form_layout, footer])
    display(page_layout)

def show_signup_page():
    """Show the sign-up page."""
    clear_output()
    style_dict = DEFAULT_STYLE

    # Create components (header, footer, form)
    header = create_header("Sign Up", '🔐', style_dict)
    footer = create_footer(style_dict)

    # Create form widgets
    su_username = widgets.Text(description='Username')
    su_password = widgets.Password(description='Password')
    su_firstname = widgets.Text(description='First Name')
    su_lastname = widgets.Text(description='Last Name')
    su_age = widgets.IntText(description='Age')
    su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

    submit_button = widgets.Button(description="Submit")
    back_button = widgets.Button(description="Back")

    # Define actions for buttons
    def sign_up_action(b):
        print("Signing up...")
        # Add sign-up logic here (e.g., validate, save user data)
        switch_to_page('session')

    def back_action(b):
        switch_to_page('start')

    submit_button.on_click(sign_up_action)
    back_button.on_click(back_action)

    # Layout the form
    form_layout = widgets.VBox([su_username, su_password, su_firstname, su_lastname, su_age, su_gender, submit_button, back_button])

    # Combine components to show the sign-up page
    page_layout = widgets.VBox([header, form_layout, footer])
    display(page_layout)

def show_session_page():
    """Show the session page after login."""
    clear_output()
    style_dict = DEFAULT_STYLE

    # Create components (header, footer)
    header = create_header(f"Welcome, {session['username']}", '🎶', style_dict)
    footer = create_footer(style_dict)

    # Create buttons for session page actions
    play_button = widgets.Button(description='Play Song')
    logout_button = widgets.Button(description='Log Out')

    # Define actions for buttons
    def play_song_action(b):
        print("🎧 Playing song...")

    def logout_action(b):
        session['user_id'] = None
        session['username'] = ''
        session['mode'] = 'none'
        switch_to_page('start')

    play_button.on_click(play_song_action)
    logout_button.on_click(logout_action)

    # Layout session page
    session_buttons = widgets.HBox([play_button, logout_button], layout=widgets.Layout(justify_content='center'))
    page_layout = widgets.VBox([header, session_buttons, footer])

    display(page_layout)

def switch_to_page(page):
    """Switch to the specified page."""
    session['current_page'] = page
    if page == 'start':
        show_start_page()
    elif page == 'signin':
        show_signin_page()
    elif page == 'signup':
        show_signup_page()
    elif page == 'session':
        show_session_page()
    else:
        print("Page not found.")

# Start the music recommender system
switch_to_page('start')


In [376]:
import ipywidgets as widgets
from IPython.display import display

# Example: Create a button and an output widget
button = widgets.Button(description="Click me!")

# Define an action when the button is clicked
def on_button_click(b):
    print("Button clicked!")

# Link the button click to the action
button.on_click(on_button_click)

# Display the button in the notebook
display(button)

Button(description='Click me!', style=ButtonStyle())

Button clicked!


In [377]:
# Create a vertical layout
vbox = widgets.VBox([button, widgets.Label("This is a label")])

# Display the layout
display(vbox)


Button clicked!
Button clicked!


In [381]:
from IPython.display import display, HTML

# Create a clickable HTML button
button_html = """
    <button onclick="alert('Button clicked!')">Click Me</button>
"""

# Display the button
display(HTML(button_html))


In [344]:
show_session_page()

In [187]:
DEFAULT_STYLE = {
    "background": "#00bcd4",
    "text_color": "#000000",
    "font": "monospace",
    "font_size": "16px",
    "header_bg": "#a6ff00",
    "max_width": 400,
    "border_radius": 10,
    "padding": 40,
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message": "Made with ❤️ in Colab",
    "button_font_size": "13px",
    "button_text_color": "#000000",
    "button_hover_color": "#8aff00"
}
def apply_default_styles(style_dict):
    for key, value in DEFAULT_STYLE.items():
        if key not in style_dict:
            style_dict[key] = value
    return style_dict

def style_widget(widget, class_name=None, layout_dict=None):
    if layout_dict:
        for key, value in layout_dict.items():
            setattr(widget.layout, key, value)
    if class_name:
        widget.add_class(class_name)

def apply_styles_to_widget_list(widgets_list, class_name="styled-button", layout_overrides=None):
    layout_overrides = layout_overrides or {}
    for widget in widgets_list:
        layout = layout_overrides.get(widget.description, {})
        style_widget(widget, class_name=class_name, layout_dict=layout)
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_styled_page(title, icon, content_widgets, style_dict=None, footer_text=None):
    style_dict = apply_default_styles(style_dict or {})

    style_html = widgets.HTML(f"""
    <style>
    .container {{
      background-color: {style_dict['background_color']};
      padding: {style_dict['padding']}px;
      border-radius: {style_dict['border_radius']}px;
      box-shadow: {style_dict['box_shadow']};
      text-align: center;
      max-width: {style_dict['max_width']}px;
      margin: auto;
    }}
    .header-box {{
      background-color: {style_dict['header_color']};
      color: {style_dict['text_color']};
      padding: 5px;
      font-family: {style_dict['font_family']};
      font-size: 15px;
    }}
    .footer-box {{
      color: {style_dict['text_color']};
      text-align: center;
      padding: 10px;
      opacity: 0.7;
      margin-top: 20px;
    }}
    .styled-button {{
      background-color: {style_dict['header_color']};
      color: {style_dict['button_text_color']};
      font-family: {style_dict['font_family']};
      font-size: {style_dict['button_font_size']};
      border-radius: 5px;
      padding: 6px 12px;
    }}
    .styled-button:hover {{
      background-color: {style_dict['button_hover_color']};
    }}
    </style>
    """)

    header = widgets.HTML(f"{icon} {title}")
    header_box = widgets.VBox([header])
    header_box.add_class("header-box")

    content_box = widgets.VBox(content_widgets)
    content_box.layout.justify_content = 'center'

    footer = widgets.HTML(footer_text or style_dict['footer_message'])
    footer_box = widgets.VBox([footer])
    footer_box.add_class("footer-box")

    container = widgets.VBox([header_box, content_box])
    container.add_class("container")

    return widgets.VBox([style_html, container, footer_box])

In [384]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

# Default style parameters for consistent styling across all pages
DEFAULT_STYLE = {
    "background_color": "#ffffff",  # White background for pages
    "text_color": "#000000",        # Text color black
    "font_family": "Arial",         # Font for header
    "header_color": "#6200ea",      # Purple header
    "max_width": 600,               # Max width for the page container
    "border_radius": 10,            # Rounded corners
    "padding": 40,                  # Padding inside the container
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",  # Shadow for container
    "footer_message": "Made with ❤️ in Colab",  # Footer message
    "button_font_size": "13px",     # Button font size
    "button_text_color": "#ffffff",# Button text color
    "button_hover_color": "#8aff00"  # Button hover color
}

def apply_default_styles(style_dict):
    """Ensure style_dict has all the necessary keys, and fill missing ones with default values."""
    for key, value in DEFAULT_STYLE.items():
        if key not in style_dict:
            style_dict[key] = value
    return style_dict

def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg = style["background_color"]
    text = style["text_color"]
    font = style["font_family"]
    header_bg = style["header_color"]
    mw = style["max_width"]
    br = style["border_radius"]
    pad = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        # Create Sign In and Sign Up buttons as ipywidgets Buttons
        signin_button = widgets.Button(
            description="Sign In",
            layout=widgets.Layout(margin="5px 10px"),
            style=widgets.ButtonStyle(
                button_color="#6200ea",  # Set background color
                font_weight="bold",  # Make the text bold
                font_size="14px"  # Adjust font size
            )
        )
        signup_button = widgets.Button(
            description="Sign Up",
            layout=widgets.Layout(margin="5px 10px"),
            style=widgets.ButtonStyle(
                button_color="#6200ea",  # Set background color
                font_weight="bold",  # Make the text bold
                font_size="14px"  # Adjust font size
            )
        )

        # Define button actions (switch pages)
        def on_signin_click(b):
            switch_to_page('signin')

        def on_signup_click(b):
            switch_to_page('signup')

        signin_button.on_click(on_signin_click)
        signup_button.on_click(on_signup_click)

        # Build the page HTML layout with the buttons
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <!-- Here we are using widgets buttons instead of HTML buttons -->
                {display(signin_button)}
                {display(signup_button)}
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128100;', title='Sign Up')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    First Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Last Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Age: <input type="number" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Gender:
                    <select style="width:60%; padding:4px; margin:4px 0;">
                        <option value="M">Male</option>
                        <option value="F">Female</option>
                    </select>
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    # Handle other pages similarly...

    return widgets.HTML(page_html)

def switch_to_page(page):
    """Switch to the specified page."""
    session['current_page'] = page
    if page == 'start':
        show_start_page()
    elif page == 'signin':
        show_signin_page()
    elif page == 'signup':
        show_signup_page()
    else:
        print("Page not found.")

def show_start_page():
    """Show the start page with Sign In and Sign Up buttons."""
    clear_output()

    # Create the page layout using the helper function
    page_widget = get_page_widget("start", DEFAULT_STYLE)
    display(page_widget)

# Assume other page functions (signin, signup, etc.) are implemented similarly...

# Start the music recommender system
switch_to_page('start')


Button(description='Sign In', layout=Layout(margin='5px 10px'), style=ButtonStyle(button_color='#6200ea', font…

Button(description='Sign Up', layout=Layout(margin='5px 10px'), style=ButtonStyle(button_color='#6200ea', font…

HTML(value='\n        <div style="max-width:600px; margin:auto; background-color:#ffffff; color:#000000; font-…

In [188]:
show_signin_page()

AttributeError: 'HTML' object has no attribute 'add_class'

In [222]:
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg = style["background_color"]
    text = style["text_color"]
    font = style["font_family"]
    header_bg = style["header_color"]
    mw = style["max_width"]
    br = style["border_radius"]
    pad = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128100;', title='Sign Up')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    First Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Last Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Age: <input type="number" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Gender:
                    <select style="width:60%; padding:4px; margin:4px 0;">
                        <option value="M">Male</option>
                        <option value="F">Female</option>
                    </select>
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "session":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128100;', title='User Session')}
            <div style="padding:10px;">
                <label>Welcome, <strong>{{session['username']}}</strong></label><br>
                <label>Mode: <strong>{{session['mode']}}</strong></label><br>
                <button style="margin:5px 10px; padding:8px 16px;">Play Song</button>
                <button style="margin:5px 10px; padding:8px 16px;">Feel Sad</button>
                <button style="margin:5px 10px; padding:8px 16px;">Feel Happy</button>
                <button style="margin:5px 10px; padding:8px 16px;">Edit Favorites</button>
                <button style="margin:5px 10px; padding:8px 16px;">Log Out</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "edit_favorites":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Edit Favorite Artists')}
            <div style="padding:10px;">
                <label>Current favorites: </label><br>
                <div style="padding:5px 0;">
                    <button style="margin:5px 10px; padding:8px 16px;">Add Artist</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Remove Artist</button>
                </div>
                <label>Confirm Changes:</label><br>
                <button style="margin:5px 10px; padding:8px 16px;">Save</button>
                <button style="margin:5px 10px; padding:8px 16px;">Back</button>
            </div>
        </div>
        {footer_html}
        """
    else:
        page_html = f"""
        <div style="{container_style}">
            <div style="text-align:center; color:red;">Page Not Found</div>
        </div>
        {footer_html}
        """
    return widgets.HTML(page_html)

In [234]:
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg = style["background_color"]
    text = style["text_color"]
    font = style["font_family"]
    header_bg = style["header_color"]
    mw = style["max_width"]
    br = style["border_radius"]
    pad = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <!-- Merged Sign In and Sign Up buttons -->
                <button style="margin:5px 10px; padding:8px 16px;" onclick="window.location.href='#'">Sign In</button>
                <button style="margin:5px 10px; padding:8px 16px;" onclick="window.location.href='#'">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128100;', title='Sign Up')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    First Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Last Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Age: <input type="number" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Gender:
                    <select style="width:60%; padding:4px; margin:4px 0;">
                        <option value="M">Male</option>
                        <option value="F">Female</option>
                    </select>
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    else:
        page_html = f"""
        <div style="{container_style}">
            <div style="text-align:center; color:red;">Page Not Found</div>
        </div>
        {footer_html}
        """

    return widgets.HTML(page_html)

In [385]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg = style["background_color"]
    text = style["text_color"]
    font = style["font_family"]
    header_bg = style["header_color"]
    mw = style["max_width"]
    br = style["border_radius"]
    pad = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <!-- Buttons are handled below in ipywidgets -->
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128100;', title='Sign Up')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    First Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Last Name: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Age: <input type="number" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Gender:
                    <select style="width:60%; padding:4px; margin:4px 0;">
                        <option value="M">Male</option>
                        <option value="F">Female</option>
                    </select>
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    else:
        page_html = f"""
        <div style="{container_style}">
            <div style="text-align:center; color:red;">Page Not Found</div>
        </div>
        {footer_html}
        """

    return widgets.HTML(page_html)

In [264]:
DEFAULT_STYLE = {
    "background_color": "#ffffff",  # Light background
    "text_color": "#000000",  # Black text
    "font": "Arial",  # Font family
    "header_color": "#6200ea",  # Purple header
    "max_width": 600,  # Max width for the page container
    "border_radius": 10,  # Rounded corners
    "padding": 20,  # Padding inside the container
    "box_shadow": "0px 4px 8px rgba(0, 0, 0, 0.1)",  # Shadow effect for the container
    "footer_message": "Enjoy your music experience!"  # Footer text
}

In [321]:
import ipywidgets as widgets

def get_page_widget(header_text="Welcome to the Music Recommender", footer_text="Enjoy your music experience!"):
    # Create the purple header inside a styled HTML div
    header_html = widgets.HTML(
        value=f"<div style='background-color: purple; color: white; font-size: 24px; "
              "font-weight: bold; text-align: center; padding: 10px;'>"
              f"🎵 {header_text}</div>"
    )

    # Placeholder VBox where the Sign In/Sign Up buttons will be injected
    button_placeholder = widgets.VBox()

    # White container VBox that holds the header and the button placeholder
    content_box = widgets.VBox([header_html, button_placeholder])
    content_box.layout = widgets.Layout(
        background_color='white',
        border='2px solid lightgray',
        border_radius='100px',
        padding='10px',
        width='80%',       # adjust as needed
        margin='20px auto' # center the white box horizontally
    )

    # Footer HTML outside the white box
    footer_html = widgets.HTML(
        value=f"<div style='text-align: center; color: gray; margin-top: 60px;'>{footer_text}</div>"
    )

    # The overall page VBox contains the white content box and then the footer
    page = widgets.VBox([content_box, footer_html])

    # Return both the page layout and the button placeholder for later injection
    return page, button_placeholder

In [322]:
def show_start_page():
    # Get the base layout and the placeholder for buttons
    page_layout, placeholder = get_page_widget()

    # Create the Sign In and Sign Up buttons with styling
    sign_in_btn = widgets.Button(
        description="Sign In",
        style={'button_color': 'purple', 'font_color': 'white'},
        layout=widgets.Layout(width='60px', margin='0 10px')
    )
    sign_up_btn = widgets.Button(
        description="Sign Up",
        style={'button_color': 'purple', 'font_color': 'white'},
        layout=widgets.Layout(width='60px', margin='0 10px')
    )

    # Put the buttons in an HBox and center them
    buttons_box = widgets.HBox([sign_in_btn, sign_up_btn])
    buttons_box.layout = widgets.Layout(justify_content='center')

    # Inject the buttons into the placeholder inside the white box
    placeholder.children = [widgets.HTML("<div style='height: 20px;'></div>"), buttons_box]
    # (Optional HTML div above buttons adds spacing if needed)

    # Display the final layout
    display(page_layout)

In [323]:
show_start_page()

In [326]:
def show_start_page():
    clear_output()

    # Define the page style
    style = {
        "background_color": "#ffffff",  # Light background for start page (white box)
        "text_color": "#000000",  # Black text
        "font_family": "Arial",  # Font family
        "header_color": "#6200ea",  # Purple header
        "max_width": 6000,  # Max width for the page container
        "border_radius": 10,  # Rounded corners
        "padding": 200,  # Padding inside the container
        "box_shadow": "0px 4px 8px rgba(0, 0, 0, 0.1)",  # Shadow effect for the container
        "footer_message": "Enjoy your music experience!"  # Footer text
    }

    # Create the HTML page using get_page_widget
    page_html = get_page_widget("start", style)

    # Display the HTML page
    display(page_html)

    # Create Sign In and Sign Up buttons manually using ipywidgets with CSS injected
    signin_button = widgets.Button(
        description="Sign In",
        layout=widgets.Layout(margin="0px 0px"),
        style=widgets.ButtonStyle(
            button_color="#ffffff",  # Set background color
            font_weight="bold",  # Make the text bold
            font_size="14px"  # Adjust font size
        )
    )
    signup_button = widgets.Button(
        description="Sign Up",
        layout=widgets.Layout(margin="5px 10px"),
        style=widgets.ButtonStyle(
            button_color="#6200ea",  # Set background color
            font_weight="bold",  # Make the text bold
            font_size="14px"  # Adjust font size
        )
    )

    # Define button actions (switch pages)
    signin_button.on_click(lambda b: switch_to_page('signin'))
    signup_button.on_click(lambda b: switch_to_page('signup'))

    # Create a container to place the buttons (centered)
    button_container = widgets.HBox([signin_button, signup_button])

    # Create a box for the buttons inside the white content area (below the purple header)
    button_box = widgets.VBox([button_container], layout=widgets.Layout(
        justify_content='center', align_items='center', width='100%', padding='20px 0'  # Add spacing around the buttons
    ))

    # Display the button container directly inside the white content box, below the header
    display(button_box)

In [327]:
get_page_widget("edit_favorites", DEFAULT_STYLE)

(VBox(children=(VBox(children=(HTML(value="<div style='background-color: purple; color: white; font-size: 24px; font-weight: bold; text-align: center; padding: 10px;'>🎵 edit_favorites</div>"), VBox()), layout=Layout(border='2px solid lightgray', margin='20px auto', padding='10px', width='80%')), HTML(value="<div style='text-align: center; color: gray; margin-top: 60px;'>{'background_color': '#ffffff', 'text_color': '#000000', 'font': 'Arial', 'header_color': '#6200ea', 'max_width': 600, 'border_radius': 10, 'padding': 20, 'box_shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'footer_message': 'Enjoy your music experience!'}</div>"))),
 VBox())

In [200]:
def update_preview(change=None):
    # Gather the current style settings from the widgets
    current_style = {
        "background_color": bg_picker.value,
        "text_color": text_picker.value,
        "font_family": font_dropdown.value,
        "header_color": header_picker.value,
        "max_width": width_slider.value,
        "border_radius": radius_slider.value,
        "padding": padding_slider.value,
        "box_shadow": f"{shadow_x.value}px {shadow_y.value}px {shadow_blur.value}px {shadow_color.value}",
        "footer_message": footer_text.value
    }

    # Get the selected page and generate the corresponding page widget
    page = page_dropdown.value
    page_widget = get_page_widget(page, current_style)

    # Display the updated preview
    with preview_out:
        clear_output()
        display(page_widget)

# Attach the update_preview function to the widgets
bg_picker.observe(update_preview, names='value')
text_picker.observe(update_preview, names='value')
header_picker.observe(update_preview, names='value')
font_dropdown.observe(update_preview, names='value')
width_slider.observe(update_preview, names='value')
radius_slider.observe(update_preview, names='value')
padding_slider.observe(update_preview, names='value')
shadow_x.observe(update_preview, names='value')
shadow_y.observe(update_preview, names='value')
shadow_blur.observe(update_preview, names='value')
shadow_color.observe(update_preview, names='value')
footer_text.observe(update_preview, names='value')
page_dropdown.observe(update_preview, names='value')

# Add actions for buttons
refresh_button.on_click(update_preview)

def reset_defaults(change):
    # Reset the style values to defaults
    bg_picker.value = DEFAULT_STYLE["background_color"]
    text_picker.value = DEFAULT_STYLE["text_color"]
    header_picker.value = DEFAULT_STYLE["header_color"]
    font_dropdown.value = DEFAULT_STYLE["font_family"]
    width_slider.value = DEFAULT_STYLE["max_width"]
    radius_slider.value = DEFAULT_STYLE["border_radius"]
    padding_slider.value = DEFAULT_STYLE["padding"]
    shadow_x.value = 2
    shadow_y.value = 2
    shadow_blur.value = 6
    shadow_color.value = "#000000"
    footer_text.value = DEFAULT_STYLE["footer_message"]

# Attach reset function to reset button
reset_button.on_click(reset_defaults)

# Display the UI components alongside the page preview
display(controls_panel, preview_out)

# Initialize the preview
update_preview()

# Now, you can display the music recommender page as usual
#start_music_recommender()


Output()

In [125]:
start_music_recommender()

In [386]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

# Add path for imports (keep as is)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService

# CSV files for users, songs, and history (keep as is)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"


# Page-wide CSS for styling
page_style = """
    <style>
        body {
            font-family: 'monospace';
            background-color: #e0f7fa;
            margin: 0;
            padding: 0;
        }
        .page-container {
            width: 100%;
            max-width: 600px;
            margin: 50px auto;
            padding: 30px;
            background-color: white;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
        }
        .page-header {
            font-size: 32px;
            color: #004d40;
            text-align: center;
            margin-bottom: 20px;
            font-weight: bold;
        }
        .button-style {
            width: 100%;
            height: 50px;
            font-size: 16px;
            background-color: #00796b;
            color: white;
            border: none;
            margin-top: 15px;
            cursor: pointer;
            border-radius: 5px;
        }
        .button-style:hover {
            background-color: #004d40;
        }
        .status-message {
            font-weight: bold;
            color: #d32f2f;
            text-align: center;
        }
        .form-label {
            font-size: 18px;
            color: #004d40;
            margin-bottom: 10px;
        }
        .form-input {
            width: 100%;
            padding: 12px;
            margin-bottom: 10px;
            border-radius: 8px;
            border: 1px solid #ddd;
            font-size: 16px;
        }
        .form-input:focus {
            border-color: #00796b;
            outline: none;
        }
        .form-section {
            margin-bottom: 25px;
        }
        .combobox-style {
            width: 100%;
            padding: 10px;
            margin-top: 10px;
            border-radius: 5px;
        }
    </style>
"""

# Show page-wide style
display(HTML(page_style))

def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3 class='page-header'>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In', layout=widgets.Layout(width='100%', height='10px', margin='2px'), style={'button_color': 'green'})
        signup_button = widgets.Button(description='Sign Up', layout=widgets.Layout(width='100%', height='10px', margin='2px'), style={'button_color': 'green'})

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3 class='page-header'>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        si_password = widgets.Password(description='Password', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        si_button = widgets.Button(description='Confirm', layout=widgets.Layout(width='100%', height='50px'))
        back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100%', height='50px'))

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3 class='page-header'>🔐 Create Account</h3>")
        su_username = widgets.Text(description='Username', layout=widgets.Layout(width='100%'), style={'description_width': '50%'})
        su_password = widgets.Password(description='Password', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        su_firstname = widgets.Text(description='First Name', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        su_lastname = widgets.Text(description='Last Name', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        su_age = widgets.IntText(description='Age', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True,
            layout=widgets.Layout(width='100%')
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='100%'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit', layout=widgets.Layout(width='100%', height='50px'))
        back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100%', height='50px'))

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode',
            layout=widgets.Layout(width='100%')
        )
        play_button = widgets.Button(description='Play Song', layout=widgets.Layout(width='100%'))
        sad_button = widgets.Button(description='Feel Sad', layout=widgets.Layout(width='100%'))
        happy_button = widgets.Button(description='Feel Happy', layout=widgets.Layout(width='100%'))
        fav_edit_button = widgets.Button(description='Edit Favorites', layout=widgets.Layout(width='100%'))
        exit_button = widgets.Button(description='Log Out', layout=widgets.Layout(width='100%'))

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='100%')
            )
            confirm_button = widgets.Button(description='Confirm Choice', layout=widgets.Layout(width='100%'))

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")

In [387]:
start_music_recommender()

✅ Multi-page interface with updated favorites-recommendation link loaded.


# Celled Main

In [345]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


# Default style parameters for consistent styling across all pages
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color": "#000000",               # Text color
    "font_family": "monospace",            # Font for header
    "header_color": "#a6ff00",             # Header background
    "max_width": 400,                      # Max width (px)
    "border_radius": 10,                   # Rounded corners
    "padding": 40,                         # Padding inside container
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",  # Shadow
    "footer_message": "Made with ❤️ in Colab",
    "button_font_size": "13px",            # Button font size
    "button_text_color": "#000000",        # Button text color
    "button_hover_color": "#8aff00"        # Button hover color
}

def apply_default_styles(style_dict):
    """Ensure style_dict has all the necessary keys, and fill missing ones with default values."""
    for key, value in DEFAULT_STYLE.items():
        if key not in style_dict:
            style_dict[key] = value
    return style_dict

def create_styled_page(title, icon, content_widgets, style_dict=None):
    """Create a styled page with a header, content, and footer using ipywidgets."""
    # Ensure style_dict contains all necessary keys, using the default values if missing
    if style_dict is None:
        style_dict = {}
    style_dict = apply_default_styles(style_dict)

    # Inject CSS classes via an HTML widget, forcing header text to stay black
    style_html = widgets.HTML(f"""
    <style>
    .container {{
      background-color: {style_dict['background_color']};
      padding: {style_dict['padding']};
      border-radius: {style_dict['border_radius']};
      box-shadow: {style_dict['box_shadow']};
      text-align: center;
      max-width: 800px;
      margin: auto;
    }}
    .header-box {{
      background-color: {style_dict['header_color']};
      color: {style_dict['text_color']};
      padding: 5px;
      font-family: {style_dict['font_family']};
      font-size: 15px;
      border-top-left-radius: {style_dict['border_radius']};
      border-top-right-radius: {style_dict['border_radius']};
    }}
    .footer-box {{
      text-align: center;
      padding: 10px;
      color: {style_dict['text_color']};
      opacity: 0.7;
      margin-top: 20px;
    }}
    .styled-button {{
      background-color: {style_dict['header_color']};
      color: {style_dict['button_text_color']};
      font-family: {style_dict['font_family']};
      font-size: {style_dict['button_font_size']};  /* Reduced font size */
      padding: 0px 0px;
      border-radius: 5px;
      border: 2px solid transparent;
    }}
    .styled-button:hover {{
      background-color: {style_dict['button_hover_color']};
    }}
    /* Force header text to be black */
    .widget-html-content, .widget-html-content * {{
      color: black !important;
    }}
    </style>
    """)

    # Header content within an HTML widget
    header_content = widgets.HTML(f"{icon} {title}")

    # Header VBox to apply styling
    header_box = widgets.VBox([header_content])
    header_box.add_class('header-box')  # Apply CSS .header-box styling to the VBox

    # Create buttons with the class "styled-button" to apply our custom styles
    sign_in_button = widgets.Button(description="Sign In", layout=widgets.Layout(margin="5px 10px"))
    sign_up_button = widgets.Button(description="Sign Up", layout=widgets.Layout(margin="5px 10px"))

    # Apply custom class to the buttons
    sign_in_button.add_class("styled-button")
    sign_up_button.add_class("styled-button")

    # Define button click callbacks
    def on_sign_in(b):
        print("DEBUG: Sign In button clicked")
        switch_to_page('signin')

    def on_sign_up(b):
        print("DEBUG: Sign Up button clicked")
        switch_to_page('signup')

    sign_in_button.on_click(on_sign_in)
    sign_up_button.on_click(on_sign_up)

    # Content area (e.g. buttons) centered in an HBox
    content_box = widgets.HBox([sign_in_button, sign_up_button], layout=widgets.Layout(justify_content='center', padding='10px 0'))

    # Container VBox for header + content
    main_container = widgets.VBox([header_box, content_box])
    main_container.add_class('container')  # Apply CSS .container styling

    # Footer content
    footer_content = widgets.HTML(style_dict.get('footer_message', ''))
    footer_box = widgets.VBox([footer_content])
    footer_box.add_class('footer-box')

    # Combine style, main_container, and footer_box in one VBox for final display
    return widgets.VBox([style_html, main_container, footer_box])

def show_start_page():
    clear_output()

    # Content for the start page (buttons for sign in and sign up)
    sign_in_button = widgets.Button(description="Sign In", layout=widgets.Layout(margin="10px 10px"))
    sign_up_button = widgets.Button(description="Sign Up", layout=widgets.Layout(margin="10px 10px"))

    # Apply custom button styles
    sign_in_button.add_class("styled-button")
    sign_up_button.add_class("styled-button")

    # Define button click callbacks
    def on_sign_in(b):
        print("DEBUG: Sign In button clicked")
        switch_to_page('signin')

    def on_sign_up(b):
        print("DEBUG: Sign Up button clicked")
        switch_to_page('signup')

    sign_in_button.on_click(on_sign_in)
    sign_up_button.on_click(on_sign_up)

    # Create and display the styled page
    page = create_styled_page(
        title="Welcome to the Music Recommender",
        icon='🎵',
        content_widgets=[sign_in_button, sign_up_button],  # Pass buttons to content area
        style_dict=DEFAULT_STYLE
    )
    display(page)

# Assuming `switch_to_page()` is defined and works with your system
# Call the function to display the start page
show_start_page()


In [346]:
#imports + fixing:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

In [347]:
from IPython.display import display, HTML
import ipywidgets as widgets

# Global CSS injected once at startup
display(HTML("""
<style>
  .header-text {
      color: black !important;  /* Ensure header text stays black */
  }
  .styled-button {
      background-color: #4CAF50;
      color: white;
      border-radius: 8px;
      border: none;
      padding: 8px 16px;
      font-size: 14px;
  }
  .styled-button:hover {
      background-color: #45a049;
  }
  .container {
      background-color: #00bcd4;
      padding: 20px;
      border-radius: 10px;
      box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
      text-align: center;
      max-width: 800px;
      margin: auto;
  }
  .footer-box {
      text-align: center;
      padding: 10px;
      color: #000;
      opacity: 0.7;
      margin-top: 20px;
  }
</style>
"""))

In [348]:
def create_styled_button(label, callback, size="medium"):
    """Create a styled button with options for different sizes."""
    btn = widgets.Button(description=label)
    btn.add_class("styled-button")
    btn.on_click(callback)

    # Customize button size
    if size == "small":
        btn.layout.width = "120px"
        btn.layout.height = "30px"
    elif size == "large":
        btn.layout.width = "200px"
        btn.layout.height = "50px"
    else:
        btn.layout.width = "150px"
        btn.layout.height = "40px"

    return btn

def create_styled_page(title, icon, content_widgets, style_dict=None):
    """Create a styled page with a header, content, and footer using ipywidgets."""
    if style_dict is None:
        style_dict = DEFAULT_STYLE

    # Header content within an HTML widget
    header_content = widgets.HTML(f"<h2 class='header-text'>{icon} {title}</h2>")

    # Header VBox to apply styling
    header_box = widgets.VBox([header_content])
    header_box.add_class('header-box')

    # Content area (e.g. buttons) centered in an HBox
    content_box = widgets.HBox(content_widgets, layout=widgets.Layout(justify_content='center', padding='10px 0'))

    # Container VBox for header + content
    main_container = widgets.VBox([header_box, content_box])
    main_container.add_class('container')  # Apply global container style

    # Footer content
    footer_content = widgets.HTML(style_dict.get('footer_msg', ''))
    footer_box = widgets.VBox([footer_content])
    footer_box.add_class('footer-box')

    # Combine style, main_container, and footer_box in one VBox for final display
    return widgets.VBox([header_box, content_box, footer_box])

In [349]:
user_manager = UserManager(users_csv)
emotion_service = EmotionService()
recommender = RecommendationService(songs_csv, history_csv, users_csv)
logger = LoggingService(history_csv)

artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

session = {
    'user_id': None,
    'username': '',
    'mode': 'none',
    'current_page': 'start'
}

music_recommender_output = widgets.Output()

✅ Health-based Dummy EmotionService initialized


In [350]:
def show_status(message, status='info'):
    colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
    with music_recommender_output:
        clear_output()
        if message:
            display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

def switch_to_page(page):
    session['current_page'] = page
    show_status("")
    if page == 'start':
        show_start_page()
    elif page == 'signin':
        show_signin_page()
    elif page == 'signup':
        show_signup_page()
    elif page == 'session':
        show_session_page()
    elif page == 'edit_favorites':
        show_edit_favorites_page()
    else:
        with music_recommender_output:
            clear_output()
            print(f"❌ Unknown page: {page}")

In [351]:
def show_start_page():
    # Create and style buttons
    sign_in = Button(description='Sign In')
    sign_up = Button(description='Sign Up')
    sign_in.style.button_color = DEFAULT_STYLE['header_bg']
    sign_up.style.button_color = DEFAULT_STYLE['header_bg']
    sign_in.style.font_weight = 'bold'
    sign_up.style.font_weight = 'bold'
    sign_in.style.font_size = '15px'
    sign_up.style.font_size = '15px'

    # Define button click callbacks
    def on_sign_in(b):
        switch_to_page('signin')
    def on_sign_up(b):
        switch_to_page('signup')
    sign_in.on_click(on_sign_in)
    sign_up.on_click(on_sign_up)

    # Build and display the styled page
    page = styled_widget_page(
        title="Welcome to the Music Recommender",
        icon='🎵',
        content_widgets=[sign_in, sign_up],
        style_dict=DEFAULT_STYLE
    )
    display(page)

# Display the styled start page
#show_start_page()

In [352]:
def show_signin_page():
    clear_output()

    # Create the title for the page
    label = widgets.HTML(
        f"<h3 style='color:{DEFAULT_STYLE['text_color']}; font-family:{DEFAULT_STYLE['font']}'>🔑 Sign In</h3>"
    )

    # Create username and password input fields
    si_username = widgets.Text(description='Username:', layout=widgets.Layout(width='60%', padding='4px'))
    si_password = widgets.Password(description='Password:', layout=widgets.Layout(width='60%', padding='4px'))

    # Create buttons with applied styles
    si_button = widgets.Button(
        description='Confirm',
        layout=widgets.Layout(margin="5px 10px", padding="8px 16px"),
        style={'button_color': DEFAULT_STYLE['header_bg'], 'font_weight': 'bold', 'color': 'black'}
    )
    back_button = widgets.Button(
        description='Back',
        layout=widgets.Layout(margin="5px 10px", padding="8px 16px"),
        style={'button_color': DEFAULT_STYLE['header_bg'], 'font_weight': 'bold', 'color': 'black'}
    )

    # Define the action for the Sign In button
    def sign_in_action(b):
        if si_username.value.strip() == "" or si_password.value.strip() == "":
            show_status("❌ Please enter both username and password.", 'error')
            return

        users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
        user_row = users_df[users_df['username'] == si_username.value]
        if user_row.empty:
            show_status("❌ Username not found.", 'error')
        elif user_row.iloc[0]['password'] != si_password.value:
            show_status("❌ Incorrect password.", 'error')
        else:
            session['user_id'] = user_row.iloc[0]['user_id']
            session['username'] = si_username.value
            show_status("")
            switch_to_page('session')

    si_button.on_click(sign_in_action)
    back_button.on_click(lambda b: switch_to_page('start'))

    # Use the styled page function to create the full page layout
    page = styled_widget_page(
        title="Sign In",
        icon="🔑",
        content_widgets=[label, si_username, si_password, si_button, back_button],
        style_dict=DEFAULT_STYLE
    )

    display(page)

In [353]:
def show_signup_page():
    header = widgets.HTML(
        f"<h1 style='background-color:{DEFAULT_STYLE['header_color']}; "
        f"color:{DEFAULT_STYLE['text_color']}; padding:5px 10px; "
        f"border-radius:5px;'>🔒 Create Account</h1>"
    )
    # Form fields (username, password, etc.)
    username = widgets.Text(description='Username:')
    password = widgets.Password(description='Password:')
    age = widgets.IntSlider(description='Age:', min=0, max=100)
    # Favorite artists section
    add_artist = widgets.Text(description='Add artist:')
    add_btn = widgets.Button(description='Add')
    # Layout for form elements
    form = widgets.VBox([
        username, password, age,
        widgets.HTML("<b>🎵 Favorite Artists:</b>"),
        widgets.HBox([add_artist, add_btn])
    ], layout=widgets.Layout(gap='10px'))
    # Action buttons
    submit_btn = widgets.Button(description='Submit')
    back_btn = widgets.Button(description='Back')
    actions = widgets.HBox([submit_btn, back_btn],
                           layout=widgets.Layout(justify_content='center', padding='10px'))
    # Combine all into the styled container
    page = styled_container([header, form, actions])
    display(page)

In [354]:
start_music_recommender()

✅ Multi-page interface with updated favorites-recommendation link loaded.


## Draft control

In [ ]:
'''# ✅ UI Style Configuration and Utilities

page_styles = {
    'start': {
        'bg': '#50c5d3', 'text': '#ffffff', 'font': 'Verdana', 'header_bg': '#bbdefb'
    },
    'signin': {
        'bg': '#fce4ec', 'text': '#880e4f', 'font': 'Georgia', 'header_bg': '#f8bbd0'
    },
    'signup': {
        'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana', 'header_bg': '#c8e6c9'
    },
    'session': {
        'bg': '#fffde7', 'text': '#f57f17', 'font': 'Courier New', 'header_bg': '#ffe082'
    },
    'edit_favorites': {
        'bg': '#ede7f6', 'text': '#4a148c', 'font': 'Tahoma', 'header_bg': '#d1c4e9'
    },
    'default': {
        'bg': '#ffffff', 'text': '#000000', 'font': 'sans-serif', 'header_bg': '#f0f0f0'
    }
}

def app_header(text_color='white', font='Verdana', bg_color='#000000'):
    return widgets.HTML(
        value=f"""
        <div style="
            background-color: {bg_color};
            color: {text_color};
            font-family: {font};
            padding: 15px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 2px 6px rgba(0,0,0,0.1);
        ">
            🎵 <b>My Music Recommender</b>
        </div>
        """
    )

def app_footer():
    return widgets.HTML("<hr><p style='color:gray;font-size:12px;text-align:center;'>Made with 💙 in Colab · Music for your mood</p>")

def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: 15px;
            border-radius: 12px;
            margin-bottom: 20px;
        }}
        button {{
            font-family: {style['font']};
        }}
    </style>
    """
    return widgets.VBox([
        widgets.HTML(value=style_html),
        app_header(
            text_color=style['text'],
            font=style['font'],
            bg_color=style.get('header_bg', '#f0f0f0')
        ),
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])'''

In [ ]:
'''import ipywidgets as widgets

# ✅ UI Style Configuration and Utilities
page_styles = {
    'start': {
        'bg': '#50c5d3', 'text': '#ffffff', 'font': 'Verdana', 'header_bg': '#bbdefb'
    },
    'signin': {
        'bg': '#fce4ec', 'text': '#880e4f', 'font': 'Georgia', 'header_bg': '#f8bbd0'
    },
    'signup': {
        'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana', 'header_bg': '#c8e6c9'
    },
    'session': {
        'bg': '#fffde7', 'text': '#f57f17', 'font': 'Courier New', 'header_bg': '#ffe082'
    },
    'edit_favorites': {
        'bg': '#ede7f6', 'text': '#4a148c', 'font': 'Tahoma', 'header_bg': '#d1c4e9'
    },
    'default': {
        'bg': '#ffffff', 'text': '#000000', 'font': 'sans-serif', 'header_bg': '#f0f0f0'
    }
}

# Header creation function with applied styles
def app_header(text_color='white', font='Verdana', bg_color='#000000'):
    return widgets.HTML(
        value=f"""
        <div style="
            background-color: {bg_color};
            color: {text_color};
            font-family: {font};
            padding: 15px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 2px 6px rgba(0,0,0,0.1);
        ">
            🎵 <b>My Music Recommender</b>
        </div>
        """
    )

# Footer creation function with styling
def app_footer():
    return widgets.HTML("<hr><p style='color:gray;font-size:12px;text-align:center;'>Made with 💙 in Colab · Music for your mood</p>")

# Main function to wrap page content with style
def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: 15px;
            border-radius: 12px;
            margin-bottom: 20px;
        }}
        button {{
            font-family: {style['font']};
            background-color: {style['header_bg']};
            color: {style['text']};
            border: none;
            padding: 8px 16px;
            border-radius: 6px;
        }}
        button:hover {{
            background-color: {style['text']};
            color: {style['bg']};
        }}
    </style>
    """
    # Return the full page layout
    return widgets.VBox([
        widgets.HTML(value=style_html),  # Apply the styles
        app_header(
            text_color=style['text'],
            font=style['font'],
            bg_color=style.get('header_bg', '#f0f0f0')
        ),
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])
'''

In [ ]:
'''import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

# Default styling configuration
DEFAULT_STYLE = {
    "background_color": "#00bcd4",         # Page background
    "text_color": "#000000",               # Text color
    "font_family": "monospace",            # Font family
    "header_color": "#a6ff00",             # Header background
    "max_width": 1200,                     # Max width (px)
    "border_radius": 10,                   # Border radius (px)
    "padding": 40,                         # Padding inside container (px)
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message": "Made with ❤️ in Colab",
    "button_bg": "#ffffff",                # Button background
    "button_text": "#000000",              # Button text color
    "button_hover": "#f0f0f0",             # Button hover background
    "button_padding": "8px 16px",          # Button padding
    "button_margin": "5px 10px",           # Button margin
    "button_border": "1px solid #cccccc",  # Button border
    "button_border_radius": "4px",         # Button border radius
    "input_bg": "#ffffff",                 # Input background
    "input_padding": "6px 8px",            # Input padding
    "input_margin": "4px 0",               # Input margin
    "input_border": "1px solid #cccccc",   # Input border
    "input_border_radius": "4px",          # Input border radius
}

# Use the default style
style = DEFAULT_STYLE.copy()

def get_styled_button(text, onclick="#", extra_style=""):
    """Generate HTML for a styled button"""
    button_style = (
        f"background-color:{style['button_bg']}; "
        f"color:{style['button_text']}; "
        f"padding:{style['button_padding']}; "
        f"margin:{style['button_margin']}; "
        f"border:{style['button_border']}; "
        f"border-radius:{style['button_border_radius']}; "
        f"cursor:pointer; "
        f"transition:background-color 0.3s; "
        f"{extra_style}"
    )
    return f"""<button style="{button_style}"
                      onclick="{onclick}"
                      onmouseover="this.style.backgroundColor='{style['button_hover']}';"
                      onmouseout="this.style.backgroundColor='{style['button_bg']}';">
                {text}
              </button>"""

def get_styled_input(input_type="text", placeholder="", value="", width="80%", extra_style=""):
    """Generate HTML for a styled input field"""
    input_style = (
        f"background-color:{style['input_bg']}; "
        f"padding:{style['input_padding']}; "
        f"margin:{style['input_margin']}; "
        f"border:{style['input_border']}; "
        f"border-radius:{style['input_border_radius']}; "
        f"width:{width}; "
        f"{extra_style}"
    )
    return f"""<input type="{input_type}"
                     style="{input_style}"
                     placeholder="{placeholder}"
                     value="{value}">"""

def get_page_widget(page_name, content_html):
    """Return an HTML widget for a page with the given content and applied styles"""
    bg = style["background_color"]
    text = style["text_color"]
    font = style["font_family"]
    header_bg = style["header_color"]
    mw = style["max_width"]
    br = style["border_radius"]
    pad = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Set icon and title based on page name
    icon_map = {
        'start': '&#127925;',          # Music note
        'signin': '&#128273;',         # Key
        'signup': '&#128274;',         # Lock
        'session': '&#127911;',        # Headphones
        'edit_favorites': '&#9733;',   # Star
    }

    title_map = {
        'start': 'Welcome to the Music Recommender',
        'signin': 'Sign In',
        'signup': 'Create Account',
        'session': 'Your Music Dashboard',
        'edit_favorites': 'Edit Favorite Artists',
    }

    icon = icon_map.get(page_name, '&#128196;')    # Default: document icon
    title = title_map.get(page_name, 'Music Recommender')

    # Header HTML
    header_html = f"""
    <div style="background-color:{header_bg}; padding:15px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px; font-weight:bold;">
        {icon} {title}
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; box-shadow:{shadow}; overflow:hidden;")

    # Footer HTML
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:10px;
                font-family:{font}; font-size:12px; margin-top:10px;">
        {footer_text}
    </div>"""

    # Build the complete page
    page_html = f"""
    <div style="{container_style}">
        {header_html}
        <div style="padding:{pad}px;">
            {content_html}
        </div>
    </div>
    {footer_html}
    """

    return widgets.HTML(page_html)

def start_music_recommender():
    print("DEBUG: Starting styled multi-page recommender with HTML widgets")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # Initialize services
    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    # Session management
    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start',
        'status_message': '',
        'status_type': 'info'
    }

    output = music_recommender_output

    # Helper function to show status messages
    def show_status(message, status_type='info'):
        session['status_message'] = message
        session['status_type'] = status_type
        with output:
            clear_output()
            if message:
                colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
                display(HTML(f"<span style='color:{colors[status_type]}; font-weight:bold'>{message}</span>"))

    # Switch pages and manage UI state
    def switch_to_page(page):
        session['current_page'] = page
        with output:
            clear_output()

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                print(f"❌ Unknown page: {page}")

    # Handle callback for button clicks
    def handle_button_click(b):
        action = b.action if hasattr(b, 'action') else None
        if action == 'goto_signin':
            switch_to_page('signin')
        elif action == 'goto_signup':
            switch_to_page('signup')
        elif action == 'goto_start':
            switch_to_page('start')
        elif action == 'goto_session':
            switch_to_page('session')
        elif action == 'goto_edit_favorites':
            switch_to_page('edit_favorites')
        elif action == 'sign_in':
            perform_signin(b)
        elif action == 'sign_up':
            perform_signup(b)
        elif action == 'logout':
            perform_logout()
        elif action == 'save_favorites':
            save_favorites()
        elif action == 'play_song':
            play_song()
        elif action == 'feel_sad':
            feel_sad()
        elif action == 'feel_happy':
            feel_happy()

    def create_button(text, action):
        """Create a button with the specified action"""
        button = widgets.Button(description=text)
        button.action = action
        button.on_click(handle_button_click)
        return button

    def show_start_page():
        """Display the start page"""
        content_html = """
        <div style="text-align:center; padding:20px;">
            <p style="font-size:18px; margin-bottom:30px;">
                Welcome to your personalized music recommendation system!
                Please sign in or create a new account to continue.
            </p>
            <div id="button-container">
                <!-- Buttons will be added here via Python -->
            </div>
        </div>
        """

        page_widget = get_page_widget('start', content_html)
        display(page_widget)

        # Add buttons programmatically
        signin_button = create_button('Sign In', 'goto_signin')
        signup_button = create_button('Sign Up', 'goto_signup')

        display(widgets.HBox([signin_button, signup_button],
                           layout=widgets.Layout(display='flex',
                                               justify_content='center',
                                               width='100%')))

        # Display status message if any
        if session['status_message']:
            show_status(session['status_message'], session['status_type'])

    def show_signin_page():
        """Display the sign in page"""
        content_html = """
        <div style="max-width:600px; margin:0 auto;">
            <form id="signin-form">
                <div style="margin-bottom:15px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">Username:</label>
                    <div id="username-input"></div>
                </div>
                <div style="margin-bottom:25px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">Password:</label>
                    <div id="password-input"></div>
                </div>
                <div style="text-align:center; margin-top:20px;" id="signin-buttons"></div>
            </form>
        </div>
        """

        page_widget = get_page_widget('signin', content_html)
        display(page_widget)

        # Add interactive elements
        username_input = widgets.Text(description='', placeholder='Enter username')
        password_input = widgets.Password(description='', placeholder='Enter password')

        # Add buttons
        signin_button = create_button('Confirm', 'sign_in')
        back_button = create_button('Back', 'goto_start')

        display(username_input)
        display(password_input)
        display(widgets.HBox([signin_button, back_button],
                           layout=widgets.Layout(display='flex',
                                               justify_content='center',
                                               width='100%')))

        # Store references for the callback functions
        signin_button.username_input = username_input
        signin_button.password_input = password_input

        # Display status message if any
        if session['status_message']:
            show_status(session['status_message'], session['status_type'])

    def perform_signin(button):
        """Handle sign in logic"""
        username = button.username_input.value.strip()
        password = button.password_input.value.strip()

        if username == "" or password == "":
            show_status("❌ Please enter both username and password.", 'error')
            return

        users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
        user_row = users_df[users_df['username'] == username]

        if user_row.empty:
            show_status("❌ Username not found.", 'error')
        elif user_row.iloc[0]['password'] != password:
            show_status("❌ Incorrect password.", 'error')
        else:
            session['user_id'] = user_row.iloc[0]['user_id']
            session['username'] = username
            show_status("✅ Signed in successfully!", 'success')
            switch_to_page('session')

    def show_signup_page():
        """Display the sign up page"""
        content_html = """
        <div style="max-width:600px; margin:0 auto;">
            <form id="signup-form">
                <div style="margin-bottom:15px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">Username:</label>
                    <div id="su-username-input"></div>
                </div>
                <div style="margin-bottom:15px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">Password:</label>
                    <div id="su-password-input"></div>
                </div>
                <div style="margin-bottom:15px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">First Name:</label>
                    <div id="firstname-input"></div>
                </div>
                <div style="margin-bottom:15px;">
                    <label style="display:block; margin-bottom:5px; font-weight:bold;">Last Name:</label>
                    <div id="lastname-input"></div>
                </div>
                <div style="display:flex; gap:20px;">
                    <div style="margin-bottom:15px; flex:1;">
                        <label style="display:block; margin-bottom:5px; font-weight:bold;">Age:</label>
                        <div id="age-input"></div>
                    </div>
                    <div style="margin-bottom:15px; flex:1;">
                        <label style="display:block; margin-bottom:5px; font-weight:bold;">Gender:</label>
                        <div id="gender-input"></div>
                    </div>
                </div>

                <div style="margin-top:20px; border-top:1px solid #ccc; padding-top:20px;">
                    <label style="display:block; margin-bottom:10px; font-weight:bold;">🎵 Favorite Artists:</label>
                    <div id="artist-inputs"></div>
                    <div id="selected-artists"></div>
                </div>

                <div style="text-align:center; margin-top:30px;" id="signup-buttons"></div>
            </form>
        </div>
        """

        page_widget = get_page_widget('signup', content_html)
        display(page_widget)

        # Add interactive form elements
        username_input = widgets.Text(description='', placeholder='Choose a username')
        password_input = widgets.Password(description='', placeholder='Choose a password')
        firstname_input = widgets.Text(description='', placeholder='Your first name')
        lastname_input = widgets.Text(description='', placeholder='Your last name')
        age_input = widgets.IntText(description='', value=0)
        gender_input = widgets.Dropdown(options=['M', 'F'], description='')

        # Artist selection widgets
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='',
            ensure_option=True
        )
        add_artist_button = widgets.Button(description='Add')
        selected_artists_output = widgets.Output()

        # Create a list to store selected artists
        selected_artists = []

        def refresh_artist_list():
            with selected_artists_output:
                clear_output()
                if not selected_artists:
                    display(HTML("<p><i>No artists selected yet</i></p>"))
                else:
                    for i, artist in enumerate(selected_artists):
                        artist_row = widgets.HBox([
                            widgets.Label(artist),
                            widgets.Button(description='Remove', layout=widgets.Layout(width='80px'))
                        ])
                        artist_row.children[1].index = i
                        artist_row.children[1].on_click(lambda b: (
                            selected_artists.pop(b.index),
                            refresh_artist_list()
                        ))
                        display(artist_row)

        def on_add_artist(_):
            artist = artist_input.value.strip()
            if artist and artist in artist_list and artist not in selected_artists:
                selected_artists.append(artist)
                artist_input.value = ''
                refresh_artist_list()
            elif artist and artist not in artist_list:
                with selected_artists_output:
                    print("Artist not found in database")

        add_artist_button.on_click(on_add_artist)

        # Submit and back buttons
        signup_button = create_button('Submit', 'sign_up')
        back_button = create_button('Back', 'goto_start')

        # Display all widgets
        display(username_input)
        display(password_input)
        display(firstname_input)
        display(lastname_input)
        display(widgets.HBox([age_input, gender_input]))
        display(HTML("<p style='font-weight:bold; margin-top:20px'>🎵 Favorite Artists:</p>"))
        display(widgets.HBox([artist_input, add_artist_button]))
        display(selected_artists_output)
        display(widgets.HBox([signup_button, back_button],
                           layout=widgets.Layout(display='flex',
                                               justify_content='center',
                                               width='100%',
                                               margin='20px 0 0 0')))

        # Initialize the artist list
        refresh_artist_list()

        # Store references for the callback functions
        signup_button.username_input = username_input
        signup_button.password_input = password_input
        signup_button.firstname_input = firstname_input
        signup_button.lastname_input = lastname_input
        signup_button.age_input = age_input
        signup_button.gender_input = gender_input
        signup_button.selected_artists = selected_artists

        # Display status message if any
        if session['status_message']:
            show_status(session['status_message'], session['status_type'])

    def perform_signup(button):
        """Handle sign up logic"""
        username = button.username_input.value.strip()
        password = button.password_input.value.strip()
        first_name = button.firstname_input.value.strip()
        last_name = button.lastname_input.value.strip()
        age = button.age_input.value
        gender = button.gender_input.value
        selected_artists = button.selected_artists

        if username == "" or password == "":
            show_status("❌ Please enter both username and password.", 'error')
            return

        users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
        if username in users_df['username'].values:
            show_status("❌ Username already taken.", 'error')
        else:
            new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
            new_user = pd.DataFrame([{
                'user_id': new_user_id,
                'username': username,
                'password': password,
                'first_name': first_name,
                'last_name': last_name,
                'age': age,
                'gender': gender,
                'favorite_artists': ', '.join(selected_artists),
                'recommended_artists': ''
            }])
            users_df = pd.concat([users_df, new_user], ignore_index=True)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(new_user_id, selected_artists)
            session['user_id'] = new_user_id
            session['username'] = username
            show_status("✅ Account created successfully!", 'success')
            switch_to_page('session')

    def show_session_page():
        """Display the user's session page"""

        # Get user info
        if session['user_id'] is None:
            switch_to_page('start')
            return

        # Get user data
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]
        if user_row.empty:
            show_status("❌ User not found. Please log in again.", 'error')
            switch_to_page('start')
            return

        content_html = f"""
        <div style="max-width:800px; margin:0 auto;">
            <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:20px;">
                <h3 style="margin:0;">👤 Welcome, {session['username']}!</h3>
                <div id="mode-selector"></div>
            </div>

            <div style="background-color:rgba(255,255,255,0.2); padding:15px; border-radius:8px; margin-bottom:20px;">
                <h4 style="margin-top:0;">Your Music Controls</h4>
                <div id="music-controls"></div>
            </div>

            <div style="background-color:rgba(255,255,255,0.2); padding:15px; border-radius:8px; margin-bottom:20px;">
                <h4 style="margin-top:0;">How are you feeling today?</h4>
                <div id="mood-controls"></div>
            </div>

            <div style="margin-top:30px; text-align:center;">
                <div id="profile-controls"></div>
            </div>

            <div id="status-output" style="margin-top:20px;"></div>
            <div id="recommendation-output" style="margin-top:20px;"></div>
        </div>
        """

        page_widget = get_page_widget('session', content_html)
        display(page_widget)

        # Mode selector
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode:',
            style={'description_width': 'initial'},
            button_style=''
        )
        mode_selector.value = session['mode']

        def on_mode_change(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        mode_selector.observe(on_mode_change, names='value')

        # Music control buttons
        play_button = create_button('Play Song', 'play_song')

        # Mood control buttons
        sad_button = create_button('Feel Sad', 'feel_sad')
        happy_button = create_button('Feel Happy', 'feel_happy')

        # Profile controls
        edit_favorites_button = create_button('Edit Favorites', 'goto_edit_favorites')
        logout_button = create_button('Log Out', 'logout')

        # Recommendation output area
        recommendation_output = widgets.Output()

        # Display all widgets
        display(mode_selector)
        display(play_button)
        display(widgets.HBox([sad_button, happy_button]))
        display(widgets.HBox([edit_favorites_button, logout_button]))
        display(recommendation_output)

        # Store references
        play_button.recommendation_output = recommendation_output
        sad_button.recommendation_output = recommendation_output
        happy_button.recommendation_output = recommendation_output

        # Display status message if any
        if session['status_message']:
            show_status(session['status_message'], session['status_type'])

    def perform_logout():
        """Handle logout logic"""
        session['user_id'] = None
        session['username'] = ''
        session['mode'] = 'none'
        show_status("✅ Logged out successfully.", 'success')
        switch_to_page('start')

    def play_song():
        """Handle play song action"""
        if session['mode'] == 'none':
            show_status("🔒 Privacy mode → not logging song play.", 'info')
            return

        songs_df = pd.read_csv(songs_csv)
        song = songs_df.sample(1).iloc[0]
        mood_before = emotion_service.detect()
        mood_after = emotion_service.detect()
        logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
        show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

    def feel_sad():
        """Handle feeling sad action"""
        if session['mode'] != 'recommend':
            show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
            return

        recommended = recommender.recommend_songs(session['user_id'])

        if recommended.empty:
            show_status("⚠ No songs found to recommend.", 'error')
            return

        # Create a recommendation interface
        options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
        song_selector = widgets.Dropdown(
            options=options,
            description='Pick one:',
            layout=widgets.Layout(width='auto')
        )
        confirm_button = widgets.Button(description='Confirm Choice')

        def on_confirm_click(_):
            selected_label = song_selector.value
            selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
            mood_before = 'sad'
            mood_after = emotion_service.detect()
            logger.log(
                session['user_id'],
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )
            show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

        confirm_button.on_click(on_confirm_click)

        with output:
            clear_output()
            display(widgets.VBox([
                widgets.HTML("<h4>🎵 Recommended Songs</h4>"),
                song_selector,
                confirm_button
            ]))

    def feel_happy():
        """Handle feeling happy action"""
        show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

    def show_edit_favorites_page():
        """Display the edit favorites page"""
        if session['user_id'] is None:
            switch_to_page('start')
            return

        # Get user's favorite artists
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        if user_row.empty:
            show_status("❌ User not found. Please log in again.", 'error')
            switch_to_page('start')
            return

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        content_html = """
        <div style="max-width:700px; margin:0 auto;">
            <p style="margin-bottom:20px;">
                Update your favorite artists below. Your recommendations will be based on these selections.
            </p>

            <div style="margin-bottom:20px;">
                <div id="artist-search"></div>
            </div>

            <div style="background-color:rgba(255,255,255,0.2); padding:15px; border-radius:8px; margin-bottom:20px;">
                <h4 style="margin-top:0;">Your Selected Artists</h4>
                <div id="selected-artists-list"></div>
            </div>

            <div style="text-align:center; margin-top:30px;">
                <div id="edit-buttons"></div>
            </div>
        </div>
        """

        page_widget = get_page_widget('edit_favorites', content_html)
        display(page_widget)

        # Artist selection widgets
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_artist_button = widgets.Button(description='Add')
        selected_artists_output = widgets.Output()

        def refresh_artist_list():
            with selected_artists_output:
                clear_output()
                if not selected_artists:
                    display(HTML("<p><i>No artists selected yet</i></p>"))
                else:
                    for i, artist in enumerate(selected_artists):
                        artist_row = widgets.HBox([
                            widgets.Label(artist),
                            widgets.Button(description='Remove', layout=widgets.Layout(width='80px'))
                        ])
                        artist_row.children[1].index = i
                        artist_row.children[1].on_click(lambda b: (
                            selected_artists.pop(b.index),
                            refresh_artist_list()
                        ))
                        display(artist_row)

        def on_add_artist(_):
            artist = artist_input.value.strip()
            if artist and artist in artist_list and artist not in selected_artists:
                selected_artists.append(artist)
                artist_input.value = ''
                refresh_artist_list()
            elif artist and artist not in artist_list:
                with selected_artists_output:
                    print("Artist not found in database")

        add_artist_button.on_click(on_add_artist)

        # Submit and back buttons
        save_button = create_button('Save Changes', 'save_favorites')
        back_button = create_button('Back to Session', 'goto_session')

        # Display all widgets
        display(artist_input)
        display(add_artist_button)
        display(selected_artists_output)
        display(widgets.HBox([save_button, back_button],
                           layout=widgets.Layout(display='flex',
                                               justify_content='center',
                                               width='100%')))

        # Initialize the artist list
        refresh_artist_list()

    def save_favorites():
        """Save the updated favorite artists list"""
        users_df = pd.read_csv(users_csv)
        users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
        users_df.to_csv(users_csv,index=False)
        user_manager.update_recommended_artists(session['user_id'], selected_artists)
        show_status("✅ Favorites updated successfully.", 'success')
        switch_to_page('session')

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")'''

'import ipywidgets as widgets\nfrom IPython.display import display, clear_output, HTML\nimport pandas as pd\nimport os\nimport importlib\nimport sys\n##################################################################################\nsys.path.append(\'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton\')\nimport recommendation_service\nimportlib.reload(recommendation_service)\nfrom recommendation_service import RecommendationService\n\nimport emotion_service\nimportlib.reload(emotion_service)\nfrom emotion_service import EmotionService\n\nimport user_manager\nimportlib.reload(user_manager)\nfrom user_manager import UserManager\n\nimport logging_service\nimportlib.reload(logging_service)\nfrom logging_service import LoggingService\n##################################################################################\nusers_csv=f"{data_dir}users.csv"\nsongs_csv=f"{data_dir}songs.csv"\nhistory_csv=f"{data_dir}listening_history.csv"\n################################################

In [ ]:
#start_music_recommender()

✅ UI launched.
Sign In
Sign Up


In [ ]:
'''# ✅ FULL UI CONTROL CELL

# 🎨 Per-page styling config
page_styles = {
    'start':        {'bg': '#e3f2fd', 'text': '#000000', 'font': 'Verdana',        'header_bg': '#bbdefb'},
    'signin':       {'bg': '#fce4ec', 'text': '#880e4f', 'font': 'Georgia',      'header_bg': '#f8bbd0'},
    'signup':       {'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'session':      {'bg': '#fffde7', 'text': '#f57f17', 'font': 'Courier New',  'header_bg': '#ffe082'},
    'edit_favorites': {'bg': '#ede7f6','text': '#4a148c','font': 'Tahoma',       'header_bg': '#d1c4e9'},
    'default':      {'bg': '#ffffff', 'text': '#000000', 'font': 'sans-serif',   'header_bg': '#f0f0f0'}
}

# 🧱 Header content & layout config
page_display_config = {
    'start':          {'header_text': '🎵 Welcome to the Music Recommender', 'header_align': 'center', 'header_size': '24px'},
    'signin':         {'header_text': '🔑 Sign In',                          'header_align': 'center', 'header_size': '24px'},
    'signup':         {'header_text': '🔐 Create Account',                   'header_align': 'center', 'header_size': '22px'},
    'session':        {'header_text': '🧠 Session Mode',                     'header_align': 'center', 'header_size': '22px'},
    'edit_favorites': {'header_text': '🎨 Edit Favorites',                   'header_align': 'left',   'header_size': '22px'},
    'default':        {'header_text': '🎶 Music App',                        'header_align': 'center', 'header_size': '22px'}
}

# 🧩 Per-page widget appearance config
page_widget_styles = {
    'default': {
        'button_color': '#333333',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '6px',
        'font_size': '14px',
    },
    'signin': {
        'button_color': '#880e4f',
        'button_text_color': '#ffffff',
        'input_width': '350px',
        'input_border_radius': '8px',
        'font_size': '16px',
    },
    'signup': {
        'button_color': '#1b5e20',
        'button_text_color': '#ffffff',
        'input_width': '400px',
        'input_border_radius': '10px',
        'font_size': '15px',
    }
    # Add more per-page overrides as needed
}

# 🦶 Footer message (change here to affect all pages)
footer_message = "Made with 💙 in Colab · Music for your mood"

# 🚀 Header builder
def app_header(text_color='white', font='Verdana', bg_color='#000000', text='🎵 My Music Recommender', align='center', size='24px'):
    return widgets.HTML(
        value=f"""
        <div style="
            background-color: {bg_color};
            color: {text_color};
            font-family: {font};
            font-size: {size};
            text-align: {align};
            padding: 15px;
            border-radius: 0px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.1);">
            {text}
        </div>
        """
    )

# 🦶 Footer builder
def app_footer():
    return widgets.HTML(
        f"<hr><p style='color:gray;font-size:12px;text-align:center;'>{footer_message}</p>"
    )

# 🧱 Full wrapper function
def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    header_cfg = page_display_config.get(page, page_display_config['default'])

    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: 15px;
            border-radius: 0px;
            margin-bottom: 20px;
        }}
        button {{
            font-family: {style['font']};
        }}
    </style>
    """

    header = app_header(
        text_color=style['text'],
        font=style['font'],
        bg_color=style.get('header_bg', '#f0f0f0'),
        text=header_cfg['header_text'],
        align=header_cfg['header_align'],
        size=header_cfg['header_size']
    )

    return widgets.VBox([
        widgets.HTML(value=style_html),
        header,
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])

def style_widgets(widgets_list, page='default'):
    config = page_widget_styles.get(page, page_widget_styles['default'])

    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = config.get('button_color', '#333')
            widget.style.font_weight = 'bold'
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px 12px',
                border_radius=config.get('input_border_radius', '6px'),
            )
        elif isinstance(widget, (widgets.Text, widgets.Password, widgets.Combobox, widgets.IntText, widgets.Dropdown)):
            widget.layout = widgets.Layout(
                width=config.get('input_width', '300px'),
                padding='6px',
                border_radius=config.get('input_border_radius', '6px'),
                font_size=config.get('font_size', '14px')
            )'''

'# ✅ FULL UI CONTROL CELL\n\n# 🎨 Per-page styling config\npage_styles = {\n    \'start\':        {\'bg\': \'#e3f2fd\', \'text\': \'#000000\', \'font\': \'Verdana\',        \'header_bg\': \'#bbdefb\'},\n    \'signin\':       {\'bg\': \'#fce4ec\', \'text\': \'#880e4f\', \'font\': \'Georgia\',      \'header_bg\': \'#f8bbd0\'},\n    \'signup\':       {\'bg\': \'#e8f5e9\', \'text\': \'#1b5e20\', \'font\': \'Verdana\',      \'header_bg\': \'#c8e6c9\'},\n    \'session\':      {\'bg\': \'#fffde7\', \'text\': \'#f57f17\', \'font\': \'Courier New\',  \'header_bg\': \'#ffe082\'},\n    \'edit_favorites\': {\'bg\': \'#ede7f6\',\'text\': \'#4a148c\',\'font\': \'Tahoma\',       \'header_bg\': \'#d1c4e9\'},\n    \'default\':      {\'bg\': \'#ffffff\', \'text\': \'#000000\', \'font\': \'sans-serif\',   \'header_bg\': \'#f0f0f0\'}\n}\n\n# 🧱 Header content & layout config\npage_display_config = {\n    \'start\':          {\'header_text\': \'🎵 Welcome to the Music Recommender\', \'header_align\': \'center

In [ ]:
'''import ipywidgets as widgets
from IPython.display import display, HTML

# Define default style parameters
DEFAULT_STYLE = {
    'background': '#00bcd4',   # Container background
    'text_color': '#000000',   # Text color
    'header_color': '#a6ff00', # Header background
    'font': 'monospace',       # Font family
    'border_radius': 10,       # Border radius in px
    'padding': 20,             # Padding in px
    'shadow': '2px 2px 8px #888888',  # Box-shadow
    'max_width': 600           # Max width for container
}

# Inject CSS for the styled-container class
css = f"""
<style>
.styled-container {{
  background-color: {DEFAULT_STYLE['background']};
  color: {DEFAULT_STYLE['text_color']};
  font-family: {DEFAULT_STYLE['font']};
  border-radius: {DEFAULT_STYLE['border_radius']}px;
  padding: {DEFAULT_STYLE['padding']}px;
  max-width: {DEFAULT_STYLE['max_width']}px;
  margin: auto;
  box-shadow: {DEFAULT_STYLE['shadow']};
}}
</style>
"""
display(HTML(css))

def styled_container(children):
    """
    Wraps the given list of widgets in a styled container (VBox)
    using the predefined DEFAULT_STYLE. All style attributes
    (background, font, padding, etc.) are applied via the 'styled-container' CSS class.
    """
    container = widgets.VBox(children, layout=widgets.Layout(align_items='center'))
    container.add_class('styled-container')
    return container'''

In [ ]:
'''import ipywidgets as widgets
from IPython.display import display, clear_output

# Define default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",         # Page background
    "text_color": "#000000",                # Text color
    "font_family": "monospace",             # Font family
    "header_color": "#a6ff00",              # Header background
    "max_width": 1200,                      # Max width (px)
    "border_radius": 10,                    # Border radius (px)
    "padding": 40,                          # Padding inside container (px)
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",  # Box shadow
    "footer_message": "Made with ❤️ in Colab",     # Footer message
    # Button specific styles
    "button_bg": "#ffffff",                 # Button background
    "button_text": "#000000",               # Button text color
    "button_hover": "#f0f0f0",              # Button hover background
    "button_padding": "8px 16px",           # Button padding
    "button_margin": "5px 10px",            # Button margin
    "button_border": "1px solid #cccccc",   # Button border
    "button_border_radius": "4px",          # Button border radius
    # Input specific styles
    "input_bg": "#ffffff",                  # Input background
    "input_padding": "6px 8px",             # Input padding
    "input_margin": "4px 0",                # Input margin
    "input_border": "1px solid #cccccc",    # Input border
    "input_border_radius": "4px",           # Input border radius
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

def apply_styles_to_widgets(widgets_list, style):
    """
    This function applies the provided style to a list of widgets.
    :param widgets_list: List of widgets to be styled.
    :param style: A dictionary containing the style properties.
    """
    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = style['button_bg']
            widget.style.font_weight = 'bold'
            widget.layout.padding = style['button_padding']
            widget.layout.margin = style['button_margin']
            widget.layout.border = style['button_border']
            widget.layout.border_radius = style['button_border_radius']
        elif isinstance(widget, widgets.Label):
            widget.style.font_family = style['font_family']
            widget.style.font_size = '16px'
            widget.style.font_weight = 'normal'
        elif isinstance(widget, widgets.Text):
            widget.style.background_color = style['input_bg']
            widget.style.padding = style['input_padding']
            widget.layout.margin = style['input_margin']
            widget.layout.border = style['input_border']
            widget.layout.border_radius = style['input_border_radius']
        elif isinstance(widget, widgets.Password):
            widget.style.background_color = style['input_bg']
            widget.style.padding = style['input_padding']
            widget.layout.margin = style['input_margin']
            widget.layout.border = style['input_border']
            widget.layout.border_radius = style['input_border_radius']
        elif isinstance(widget, widgets.Dropdown):
            widget.style.background_color = style['input_bg']
            widget.style.padding = style['input_padding']
            widget.layout.margin = style['input_margin']
            widget.layout.border = style['input_border']
            widget.layout.border_radius = style['input_border_radius']

def wrap_page(content):
    """
    Wrap the given content in a styled container.
    :param content: List of widgets to display inside the page.
    :return: A container widget with applied styles.
    """
    # Create the container and set its layout
    container = widgets.VBox(content, layout=widgets.Layout(
        width=str(style["max_width"]) + "px",
        background_color=style["background_color"],
        padding=str(style["padding"]) + "px",
        border_radius=str(style["border_radius"]) + "px",
        box_shadow=style["box_shadow"],
        align_items='center'
    ))

    return container'''

## Control

In [388]:
import ipywidgets as widgets
from IPython.display import display

# Define default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",  # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        400,              # Max width (px)
    "border_radius":    10,                # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}
# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        # Welcome page with Sign In / Sign Up buttons
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        # Sign-in form page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        # Sign-up / Create account form
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128274;', title='Create Account')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:6px;">
                    Username: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Password: <input type="password" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    First Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Last Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Age: <input type="number" style="width:80px; padding:4px; margin:3px 0;" value=0>
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Gender:
                    <select style="padding:4px; margin:3px 0; width:100px;">
                        <option value="M">M</option><option value="F">F</option>
                    </select>
                </label>
                <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                <label style="display:block; margin-bottom:6px; font-style:italic;">
                    🎵 Favorite Artists:
                </label>
                <div style="margin-bottom:6px;">
                    Add artist: <input type="text" style="width:60%; padding:4px;">
                    <button style="padding:6px 12px; margin-left:4px;">Add</button>
                </div>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Submit</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "session":
        # Dummy session/home page after login
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127911;', title='Your Music Dashboard')}
            <div style="padding:10px;">
                <p>Welcome back! Choose a genre or mood to get recommendations.</p>
                <button style="margin:5px 10px; padding:8px 16px;">Rock</button>
                <button style="margin:5px 10px; padding:8px 16px;">Jazz</button>
                <button style="margin:5px 10px; padding:8px 16px;">Pop</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "edit_favorites":
        # Edit favorites list page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Edit Favorite Artists')}
            <div style="padding:10px;">
                <p>Here are your favorite artists. Remove or add new ones.</p>
                <ul style="text-align:left;">
                    <li>Artist A <button style="margin-left:10px;">Remove</button></li>
                    <li>Artist B <button style="margin-left:10px;">Remove</button></li>
                </ul>
                <div style="margin-top:10px;">
                    Add artist: <input type="text" style="padding:4px;">
                    <button style="padding:6px 12px;">Add</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    else:
        # Fallback: unknown page
        page_html = f"<div style='{container_style}'>Unknown page: {page}</div>"

    # Return as an HTML widget
    return widgets.HTML(page_html)

In [390]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Define default style parameters
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",      # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        400,              # Max width (px)
    "border_radius":    10,               # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        {{icon}} <strong>{{title}}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        # Welcome page with Sign In / Sign Up buttons
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button onclick="alert('Sign In clicked!')" style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button onclick="alert('Sign Up clicked!')" style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        # Sign-in form page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button onclick="alert('Confirm clicked!')" style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button onclick="alert('Back clicked!')" style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        # Sign-up / Create account form
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128274;', title='Create Account')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:6px;">
                    Username: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Password: <input type="password" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    First Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Last Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Age: <input type="number" style="width:80px; padding:4px; margin:3px 0;" value=0>
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Gender:
                    <select style="padding:4px; margin:3px 0; width:100px;">
                        <option value="M">M</option><option value="F">F</option>
                    </select>
                </label>
                <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                <label style="display:block; margin-bottom:6px; font-style:italic;">
                    🎵 Favorite Artists:
                </label>
                <div style="margin-bottom:6px;">
                    Add artist: <input type="text" style="width:60%; padding:4px;">
                    <button onclick="alert('Add clicked!')" style="padding:6px 12px; margin-left:4px;">Add</button>
                </div>
                <div style="text-align:center; margin-top:10px;">
                    <button onclick="alert('Submit clicked!')" style="margin:5px 10px; padding:8px 16px;">Submit</button>
                    <button onclick="alert('Back clicked!')" style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "session":
        # Dummy session/home page after login
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127911;', title='Your Music Dashboard')}
            <div style="padding:10px;">
                <p>Welcome back! Choose a genre or mood to get recommendations.</p>
                <button onclick="alert('Rock clicked!')" style="margin:5px 10px; padding:8px 16px;">Rock</button>
                <button onclick="alert('Jazz clicked!')" style="margin:5px 10px; padding:8px 16px;">Jazz</button>
                <button onclick="alert('Pop clicked!')" style="margin:5px 10px; padding:8px 16px;">Pop</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "edit_favorites":
        # Edit favorites list page
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Edit Favorite Artists')}
            <div style="padding:10px;">
                <p>Here are your favorite artists. Remove or add new ones.</p>
                <ul style="text-align:left;">
                    <li>Artist A <button onclick="alert('Remove clicked!')" style="margin-left:10px;">Remove</button></li>
                    <li>Artist B <button onclick="alert('Remove clicked!')" style="margin-left:10px;">Remove</button></li>
                </ul>
                <div style="margin-top:10px;">
                    Add artist: <input type="text" style="padding:4px;">
                    <button onclick="alert('Add clicked!')" style="padding:6px 12px;">Add</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    else:
        # Fallback: unknown page
        page_html = f"<div style='{container_style}'>Unknown page: {page}</div>"

    # Return as an HTML widget
    return widgets.HTML(page_html)


In [395]:
get_page_widget("edit_favorites", DEFAULT_STYLE)

HTML(value='\n        <div style="max-width:400px; margin:auto; background-color:#00bcd4; color:#000000; font-…

In [357]:
# Page selector dropdown
page_dropdown = widgets.Dropdown(
    options=["start", "signin", "signup", "session", "edit_favorites"],
    value="start", description="Page:")
# Color pickers
bg_picker = widgets.ColorPicker(value=DEFAULT_STYLE["background_color"], description="Background:")
text_picker = widgets.ColorPicker(value=DEFAULT_STYLE["text_color"], description="Text:")
header_picker = widgets.ColorPicker(value=DEFAULT_STYLE["header_color"], description="Header:")
# Font family dropdown (common web-safe fonts)
font_dropdown = widgets.Dropdown(
    options=["Arial, sans-serif", "Helvetica", "Times New Roman", "Courier New", "Verdana, sans-serif", "serif", "sans-serif", "monospace"],
    value=DEFAULT_STYLE["font_family"], description="Font:")
# Numeric sliders for layout
width_slider = widgets.IntSlider(value=DEFAULT_STYLE["max_width"], min=200, max=1200, step=50, description="Max Width (px):")
radius_slider = widgets.IntSlider(value=DEFAULT_STYLE["border_radius"], min=0, max=50, step=1, description="Border Radius (px):")
padding_slider = widgets.IntSlider(value=DEFAULT_STYLE["padding"], min=0, max=50, step=1, description="Padding (px):")
# Box-shadow inputs (to ensure valid formatting)
shadow_x = widgets.IntText(value=2, description="Shadow X (px):")
shadow_y = widgets.IntText(value=2, description="Shadow Y (px):")
shadow_blur = widgets.IntText(value=6, description="Shadow Blur (px):")
shadow_color = widgets.ColorPicker(value="#000000", description="Shadow Color:")
# Footer message text
footer_text = widgets.Text(value=DEFAULT_STYLE["footer_message"], description="Footer Msg:")

# Buttons for actions
refresh_button = widgets.Button(description="🔄 Refresh Preview", button_style="info")
reset_button   = widgets.Button(description="♻️ Reset Defaults", button_style="warning")

# Output area for preview
preview_out = widgets.Output()

In [358]:
# Layout containers for grouping
color_box = widgets.VBox([bg_picker, text_picker, header_picker])
font_box = widgets.VBox([font_dropdown])
layout_box = widgets.VBox([width_slider, radius_slider, padding_slider])
shadow_box = widgets.VBox([widgets.HBox([shadow_x, shadow_y, shadow_blur]), shadow_color])
buttons_box = widgets.HBox([refresh_button, reset_button])

# Final control panel (left column)
controls_panel = widgets.VBox([
    widgets.HTML("<h3>Style Control Panel</h3>"),
    widgets.HTML("<b>Page Preview:</b>"), page_dropdown,
    widgets.HTML("<b>Colors:</b>"), color_box,
    widgets.HTML("<b>Font Family:</b>"), font_box,
    widgets.HTML("<b>Layout:</b>"), layout_box,
    widgets.HTML("<b>Box Shadow:</b>"), shadow_box,
    widgets.HTML("<b>Footer Message:</b>"), footer_text,
    buttons_box
], layout=widgets.Layout(width='35%', border='solid 1px gray', padding='10px', margin='10px'))

In [359]:
def refresh_preview(_=None):
    """Read widget values into the style dict and update the preview output."""
    # Update global style dictionary from widget values
    style["background_color"] = bg_picker.value
    style["text_color"]       = text_picker.value
    style["font_family"]      = font_dropdown.value
    style["header_color"]     = header_picker.value
    style["max_width"]        = width_slider.value
    style["border_radius"]    = radius_slider.value
    style["padding"]          = padding_slider.value
    # Construct box-shadow string from parts
    style["box_shadow"] = f"{shadow_x.value}px {shadow_y.value}px {shadow_blur.value}px {shadow_color.value}"
    style["footer_message"] = footer_text.value

    # Display the selected page with updated style
    selected_page = page_dropdown.value
    with preview_out:
        preview_out.clear_output()
        display(get_page_widget(selected_page, style))

def reset_defaults(_=None):
    """Reset all controls to default values and refresh preview."""
    # Reset widget values to defaults
    page_dropdown.value = "start"
    bg_picker.value     = DEFAULT_STYLE["background_color"]
    text_picker.value   = DEFAULT_STYLE["text_color"]
    font_dropdown.value = DEFAULT_STYLE["font_family"]
    header_picker.value = DEFAULT_STYLE["header_color"]
    width_slider.value  = DEFAULT_STYLE["max_width"]
    radius_slider.value = DEFAULT_STYLE["border_radius"]
    padding_slider.value= DEFAULT_STYLE["padding"]
    shadow_x.value      = 2
    shadow_y.value      = 2
    shadow_blur.value   = 6
    shadow_color.value  = "#000000"
    footer_text.value   = DEFAULT_STYLE["footer_message"]
    # Also update the style dict and preview
    refresh_preview()

# Bind buttons to callbacks
refresh_button.on_click(refresh_preview)
reset_button.on_click(reset_defaults)

In [360]:
# Combine panel and preview output side by side
ui = widgets.HBox([controls_panel, preview_out], layout=widgets.Layout(width='100%'))

# Display the complete UI
display(ui)

# Initialize the preview with defaults
refresh_preview()

In [361]:
import ipywidgets as widgets
from IPython.display import display, HTML

# 🎨 Default style parameters (for any page)
DEFAULT_STYLE = {
    "background_color": "#00bcd4",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "monospace",  # Font family
    "header_color":     "#a6ff00",        # Header background
    "max_width":        600,              # Max width (px)
    "border_radius":    10,                # Border radius (px)
    "padding":          40,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab"
}

# Function to construct an HTML widget for each page, applying the style
def get_page_widget(page: str, style: dict) -> widgets.HTML:
    """Return an ipywidgets.HTML representing the given page with applied styles."""
    # Common style snippets
    bg    = style["background_color"]
    text  = style["text_color"]
    font  = style["font_family"]
    header_bg = style["header_color"]
    mw    = style["max_width"]
    br    = style["border_radius"]
    pad   = style["padding"]
    shadow = style["box_shadow"]
    footer_text = style["footer_message"]

    # Header HTML (shared by pages)
    header_html = f"""
    <div style="background-color:{header_bg}; padding:10px;
                border-top-left-radius:{br}px; border-top-right-radius:{br}px;
                color:{text}; font-family:{font}; font-size:20px;">
        🎵 <strong>{page}</strong>
    </div>"""

    # Common container style
    container_style = (f"max-width:{mw}px; margin:auto; background-color:{bg}; color:{text}; "
                       f"font-family:{font}; border-radius:{br}px; padding:{pad}px; box-shadow:{shadow};")
    footer_html = f"""
    <div style="max-width:{mw}px; margin:auto; text-align:center; color:gray; padding:5px;
                font-family:{font};">
        {footer_text}
    </div>"""

    # Build each page
    if page == "start":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Welcome to the Music Recommender')}
            <div style="padding:10px; text-align:center;">
                <button style="margin:5px 10px; padding:8px 16px;">Sign In</button>
                <button style="margin:5px 10px; padding:8px 16px;">Sign Up</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signin":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128273;', title='Sign In')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:8px;">
                    Username: <input type="text" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <label style="display:block; margin-bottom:8px;">
                    Password: <input type="password" style="width:60%; padding:4px; margin:4px 0;">
                </label>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Confirm</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "signup":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#128274;', title='Create Account')}
            <div style="padding:10px;">
                <label style="display:block; margin-bottom:6px;">
                    Username: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Password: <input type="password" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    First Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Last Name: <input type="text" style="width:80%; padding:4px; margin:3px 0;">
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Age: <input type="number" style="width:80px; padding:4px; margin:3px 0;" value=0>
                </label>
                <label style="display:block; margin-bottom:6px;">
                    Gender:
                    <select style="padding:4px; margin:3px 0; width:100px;">
                        <option value="M">M</option><option value="F">F</option>
                    </select>
                </label>
                <div style="border-top:1px solid #ccc; margin:10px 0;"></div>
                <label style="display:block; margin-bottom:6px; font-style:italic;">
                    🎵 Favorite Artists:
                </label>
                <div style="margin-bottom:6px;">
                    Add artist: <input type="text" style="width:60%; padding:4px;">
                    <button style="padding:6px 12px; margin-left:4px;">Add</button>
                </div>
                <div style="text-align:center; margin-top:10px;">
                    <button style="margin:5px 10px; padding:8px 16px;">Submit</button>
                    <button style="margin:5px 10px; padding:8px 16px;">Back</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    elif page == "session":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127911;', title='Your Music Dashboard')}
            <div style="padding:10px;">
                <p>Welcome back! Choose a genre or mood to get recommendations.</p>
                <button style="margin:5px 10px; padding:8px 16px;">Rock</button>
                <button style="margin:5px 10px; padding:8px 16px;">Jazz</button>
                <button style="margin:5px 10px; padding:8px 16px;">Pop</button>
            </div>
        </div>
        {footer_html}
        """
    elif page == "edit_favorites":
        page_html = f"""
        <div style="{container_style}">
            {header_html.format(icon='&#127925;', title='Edit Favorite Artists')}
            <div style="padding:10px;">
                <p>Here are your favorite artists. Remove or add new ones.</p>
                <ul style="text-align:left;">
                    <li>Artist A <button style="margin-left:10px;">Remove</button></li>
                    <li>Artist B <button style="margin-left:10px;">Remove</button></li>
                </ul>
                <div style="margin-top:10px;">
                    Add artist: <input type="text" style="padding:4px;">
                    <button style="padding:6px 12px;">Add</button>
                </div>
            </div>
        </div>
        {footer_html}
        """
    else:
        page_html = f"<div style='{container_style}'>Unknown page: {page}</div>"

    return widgets.HTML(page_html)

# To display the page
page_widget = get_page_widget('signup', DEFAULT_STYLE)  # Pass 'start', 'signin', etc.
display(page_widget)

HTML(value='\n        <div style="max-width:600px; margin:auto; background-color:#00bcd4; color:#000000; font-…

In [362]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Default style settings
DEFAULT_STYLE = {
    "background_color": "#ffffff",        # Page background
    "text_color":       "#000000",        # Text color
    "font_family":      "Arial, sans-serif",  # Font family
    "header_color":     "#eeeeee",        # Header background
    "max_width":        600,              # Max width (px)
    "border_radius":    8,                # Border radius (px)
    "padding":          10,               # Padding inside container (px)
    # Box shadow: x-offset, y-offset, blur, color (RGBA)
    "box_shadow":       "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message":   "Made with ❤️ in Colab",
    "margin":           "20px auto" # Added missing margin from wrap_page
}

# Copy defaults to current style dict
style = DEFAULT_STYLE.copy()

# ============================
# 🧩 Control Widgets (Style Update)
# ============================

def update_style(change):
    """Update the global style dictionary from the control panel widgets."""
    global style
    style['background_color'] = bg_color_picker.value
    style['text_color'] = text_color_picker.value
    style['font_family'] = font_picker.value
    style['header_color'] = header_bg_picker.value
    style['max_width'] = max_width_picker.value
    style['border_radius'] = border_radius_picker.value
    style['padding'] = padding_picker.value
    # Ensure the box shadow components are combined correctly
    style['box_shadow'] = f"{shadow_x.value}px {shadow_y.value}px {shadow_blur.value}px {shadow_color.value}"
    style['footer_message'] = footer_text.value
    # No need to call refresh_preview here, the observation on widget values will trigger it
    pass # Or remove this function entirely if updates are handled directly by observing widget changes

# Let's remove the explicit update_style function and rely on refreshing the preview
# when widget values change. This simplifies the logic.

def reset_style(_=None):
    """Reset all styles to default values and update the widgets."""
    global style
    style = DEFAULT_STYLE.copy()
    # Update the widget values to reflect the reset defaults
    bg_color_picker.value = style['background_color']
    text_color_picker.value = style['text_color']
    header_bg_picker.value = style['header_color']
    # Ensure the font picker value is valid after reset
    if style['font_family'] not in font_picker.options:
         font_picker.options = list(font_picker.options) + [style['font_family']] # Add if missing
    font_picker.value = style['font_family']

    max_width_picker.value = style['max_width']
    border_radius_picker.value = style['border_radius']
    padding_picker.value = style['padding']
    # Note: DEFAULT_STYLE had a single string for box_shadow, but the widgets
    # split it into components. Resetting these will require parsing the default.
    # A simpler approach is to define default shadow values separately or
    # set them explicitly here. Let's set them explicitly based on the original error.
    shadow_x.value = 2
    shadow_y.value = 2
    shadow_blur.value = 6
    shadow_color.value = "#000000" # Assuming this was the intended default color part
    footer_text.value = style['footer_message']

    refresh_preview()

# ============================
# 🧱 Header builder
# ============================

# This header builder uses the global 'style' dictionary directly,
# which is good.

def app_header(text_color='white', font='Verdana', bg_color='#000000',
               text='🎵 My Music Recommender', align='center', size='24px'):
    # Use values from the global style dict unless overridden by function args
    return widgets.HTML(
        value=f"""
        <div style="display: flex; justify-content: center; margin-bottom: 10px;">
            <div style="
                background-color: {bg_color or style.get('header_color', '#eeeeee')}; /* Use style if arg is None or '' */
                color: {text_color or style.get('text_color', '#000000')};
                font-family: {font or style.get('font_family', 'Arial, sans-serif')};
                font-size: {size};
                text-align: {align};
                padding: 12px;
                border-radius: {style.get('border_radius', 8)}px;
                max-width: {style.get('max_width', 600)}px;
                box-shadow: {style.get('box_shadow', '2px 2px 6px rgba(0,0,0,0.2)')};
                margin: {style.get('margin', '20px auto')};
                width: 100%; /* Ensure it takes full width within max-width */
                box-sizing: border-box; /* Include padding in width calculation */
                ">
                {text}
            </div>
        </div>
        """
    )

# ============================
# 🧱 Footer builder
# ============================

# Need an app_footer function used in wrap_page
def app_footer():
     return widgets.HTML(
        value=f"""
        <div style="max-width:{style.get('max_width', 600)}px; margin:auto; text-align:center; color:gray; padding:5px;
                    font-family:{style.get('font_family', 'Arial, sans-serif')}; font-size: small;">
            {style.get('footer_message', 'Made with ❤️ in Colab')}
        </div>
        """
     )


# ============================
# 🧱 Page Wrapper with dynamic preview
# ============================

def wrap_page(content_widgets):
    """Wrap content in a page with the applied styles."""
    # The style_html is added directly to the page. This is less ideal
    # than applying styles to the VBox directly or using layout properties.
    # However, sticking to the original pattern for now.
    # Using a CSS class might be more robust for applying styles.
    style_html = f"""
    <style>
        .custom-page-container {{ /* Container for content */
            background-color: {style.get('background_color', '#ffffff')};
            color: {style.get('text_color', '#000000')};
            font-family: {style.get('font_family', 'Arial, sans-serif')};
            padding: {style.get('padding', 10)}px;
            border-radius: {style.get('border_radius', 8)}px;
            max-width: {style.get('max_width', 600)}px;
            box-shadow: {style.get('box_shadow', '2px 2px 6px rgba(0,0,0,0.2)')};
            margin: {style.get('margin', '20px auto')};
            box-sizing: border-box; /* Include padding in width calculation */
        }}
        /* Ensure internal widgets inherit font/color */
        .custom-page-container .widget-label,
        .custom-page-container .widget-button {{
             font-family: inherit !important; /* Use font from container */
             color: inherit !important; /* Use text color from container */
        }}

        /* Optional: Style buttons/inputs a bit */
        .custom-page-container .widget-button {{
             padding: 8px 15px;
             margin: 5px;
             border-radius: 4px;
             border: 1px solid #ccc;
             background-color: #f0f0f0;
             cursor: pointer;
        }}
         .custom-page-container .widget-button:hover {{
            background-color: #e0e0e0;
         }}
        .custom-page-container .widget-text,
        .custom-page-container .widget-password,
        .custom-page-container .widget-dropdown,
        .custom-page-container .widget-combobox,
        .custom-page-container .widget-inttext,
        .custom-page-container .widget-intslider {{
            margin: 5px 0;
            padding: 5px;
            border-radius: 4px;
            border: 1px solid #ccc;
        }}
    </style>
    """

    header = app_header(
        text_color=style.get('text_color', '#000000'), # Pass color from style
        font=style.get('font_family', 'Arial, sans-serif'), # Pass font from style
        bg_color=style.get('header_color', '#eeeeee'), # Pass header color from style
        text='🎵 Welcome to the Music Recommender',
        align='center',
        size='20px'
    )

    # Wrap content in a VBox and apply the CSS class via _dom_classes
    content_vbox = widgets.VBox(content_widgets, layout=widgets.Layout(width='100%')) # Ensure VBox takes full width
    content_container = widgets.VBox([content_vbox], _dom_classes=['custom-page-container'])

    return widgets.VBox([
        widgets.HTML(value=style_html), # Inject CSS
        header,
        content_container,
        app_footer()
    ])

# ============================
# Mock Page Widgets for Preview
# ============================

# Create mock widget structures for each page type for the *preview*
# This is separate from the actual application pages defined earlier
# (show_start_page, show_signin_page, etc.)

def get_page_widget(page: str, style: dict) -> widgets.VBox:
    """Generate a mock widget structure for each page type based on current style."""
    # This function should return ipywidgets objects, not HTML strings directly,
    # so wrap_page can apply the styling.

    if page == "start":
        label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender (Preview)</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')
        # Apply styles to individual widgets within the page structure
        # This is done via the style_widgets function or direct layout/style attributes if not using CSS classes
        # For the preview, let's rely on the CSS class injected by wrap_page for now.
        content = [
            label,
            widgets.HBox([signin_button, signup_button])
        ]
        # wrap_page will add the header, footer, and apply the main container styles
        return wrap_page(content)

    elif page == "signin":
        label = widgets.HTML("<h3>🔑 Sign In (Preview)</h3>")
        username = widgets.Text(description='Username')
        password = widgets.Password(description='Password')
        confirm = widgets.Button(description='Confirm')
        back = widgets.Button(description='Back')
        content = [label, username, password, confirm, back]
        return wrap_page(content)

    elif page == "signup":
        label = widgets.HTML("<h3>🔐 Create Account (Preview)</h3>")
        username = widgets.Text(description='Username')
        password = widgets.Password(description='Password')
        firstname = widgets.Text(description='First Name')
        age = widgets.IntText(description='Age')
        gender = widgets.Dropdown(options=['M', 'F'], description='Gender')
        artist_input = widgets.Combobox(placeholder='Artist...')
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox([widgets.HBox([widgets.Label("Artist 1"), widgets.Button(description='Remove')]),
                                   widgets.HBox([widgets.Label("Artist 2"), widgets.Button(description='Remove')])]) # Mock artists
        content = [
            label, username, password, firstname, age, gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            widgets.Button(description='Submit'), widgets.Button(description='Back')
        ]
        return wrap_page(content)

    elif page == "session":
        profile_label = widgets.HTML("<b>👤 Signed in as: preview_user</b>")
        mode_selector = widgets.ToggleButtons(options=['none', 'log', 'recommend'], description='Mode')
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        edit_button = widgets.Button(description='Edit Favorites')
        logout_button = widgets.Button(description='Log Out')

        content = [
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            edit_button,
            logout_button,
            widgets.Output() # Placeholder output for session messages
        ]
        return wrap_page(content)

    elif page == "edit_favorites":
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists (Preview)</h3>")
        artist_input = widgets.Combobox(placeholder='Artist...')
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox([widgets.HBox([widgets.Label("Artist A"), widgets.Button(description='Remove')]),
                                   widgets.HBox([widgets.Label("Artist B"), widgets.Button(description='Remove')])]) # Mock artists
        save_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')
        content = [
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            save_button,
            back_button
        ]
        return wrap_page(content)

    else:
        return widgets.HTML(f"❌ Unknown page for preview: {page}")

# ============================
# 🎯 Control Panel for Dynamic Changes
# ============================

# Create control widgets for each style property
page_dropdown = widgets.Dropdown(options=["start", "signin", "signup", "session", "edit_favorites"], value="start", description="Page:")
bg_color_picker = widgets.ColorPicker(value=style['background_color'], description="Background:")
text_color_picker = widgets.ColorPicker(value=style['text_color'], description="Text color:")
header_bg_picker = widgets.ColorPicker(value=style['header_color'], description="Header BG:")

# Modified: Added "Arial, sans-serif" to the options list
font_options = ["Verdana", "Arial", "Courier New", "Georgia", "Arial, sans-serif"]
font_picker = widgets.Dropdown(options=font_options, value=style['font_family'] if style['font_family'] in font_options else font_options[0], description="Font:")

max_width_picker = widgets.IntSlider(value=style['max_width'], min=200, max=1200, description="Max Width:")
border_radius_picker = widgets.IntSlider(value=style['border_radius'], min=0, max=50, description="Border Radius:")
padding_picker = widgets.IntSlider(value=style['padding'], min=0, max=50, description="Padding:")
# Note: Initializing shadow widgets from the string requires parsing or hardcoding defaults
shadow_x = widgets.IntText(value=2, description="Shadow X:") # Changed to IntText to match original error source
shadow_y = widgets.IntText(value=2, description="Shadow Y:") # Changed to IntText
shadow_blur = widgets.IntText(value=6, description="Shadow Blur:") # Changed to IntText
shadow_color = widgets.ColorPicker(value="#000000", description="Shadow Color:") # Assuming default color

footer_text = widgets.Text(value=style.get('footer_message', ''), description="Footer Message:") # Use .get for safety

# Buttons for actions
refresh_button = widgets.Button(description="🔄 Refresh Preview", button_style='info') # Added button_style
reset_button = widgets.Button(description="♻️ Reset Defaults", button_style='warning') # Added button_style

# ============================
# 🧩 Create Control Panel Layout
# ============================

# Group widgets into logical sections
color_box = widgets.VBox([bg_color_picker, text_color_picker, header_bg_picker])
font_box = widgets.VBox([font_picker])
layout_box = widgets.VBox([max_width_picker, border_radius_picker, padding_picker])
# Combined shadow inputs as in the original setup
shadow_box = widgets.VBox([widgets.HBox([shadow_x, shadow_y, shadow_blur]), shadow_color])
footer_box = widgets.VBox([footer_text]) # Wrapped footer_text in a VBox

# Layout for the control panel
controls_panel = widgets.VBox([
    widgets.HTML("<h3>Style Control Panel</h3>"),
    widgets.HTML("<b>Page Preview:</b>"), page_dropdown, # Added label
    widgets.HTML("<b>Colors:</b>"), color_box,
    widgets.HTML("<b>Font Family:</b>"), font_box,
    widgets.HTML("<b>Layout:</b>"), layout_box,
    widgets.HTML("<b>Box Shadow:</b>"), shadow_box,
    widgets.HTML("<b>Footer Message:</b>"), footer_box, # Use footer_box
    widgets.HBox([refresh_button, reset_button])
], layout=widgets.Layout(width='35%', border='solid 1px gray', padding='10px', margin='10px')) # Added layout from original

# Preview output area
preview_out = widgets.Output(layout=widgets.Layout(width='60%')) # Added layout to control preview width

# Bind button actions to functions
refresh_button.on_click(refresh_preview)
reset_button.on_click(reset_style)

# Observe changes on style-related widgets to automatically refresh the preview
# This is a more interactive approach than only using a refresh button.
# Let's make a list of style widgets to observe.
style_widgets_to_observe = [
    bg_color_picker, text_color_picker, header_bg_picker, font_picker,
    max_width_picker, border_radius_picker, padding_picker,
    shadow_x, shadow_y, shadow_blur, shadow_color, footer_text, page_dropdown
]

def on_style_widget_change(change):
    """Update the style dictionary and refresh the preview."""
    # Update style dictionary from current widget values
    style['background_color'] = bg_color_picker.value
    style['text_color'] = text_color_picker.value
    style['font_family'] = font_picker.value
    style['header_color'] = header_bg_picker.value
    style['max_width'] = max_width_picker.value
    style['border_radius'] = border_radius_picker.value
    style['padding'] = padding_picker.value
    style['box_shadow'] = f"{shadow_x.value}px {shadow_y.value}px {shadow_blur.value}px {shadow_color.value}"
    style['footer_message'] = footer_text.value
    # Only refresh the preview, no need to re-set widget values
    refresh_preview()

# Observe 'value' changes on the style widgets
for widget in style_widgets_to_observe:
    widget.observe(on_style_widget_change, names='value')

# Display the entire control panel and preview side-by-side
ui = widgets.HBox([controls_panel, preview_out], layout=widgets.Layout(width='100%', flex_flow='row wrap')) # Wrap to handle smaller screens

display(ui)

# Initial preview
refresh_preview()

In [363]:
import ipywidgets as widgets
from IPython.display import display, HTML

# ================================
# 🎨 Global Styles Configuration
# ================================

# Global page styling (background, text, font)
page_style = {
    'bg': '#00bcd4',           # Background color
    'text': '#000000',         # Text color
    'font': 'Monospace',         # Font family
    'header_bg': '#a6ff00',    # Header background color
    'max_width': '1200px',      # Max width of the content box
    'border_radius': '10px',   # Border radius for content box
    'padding': '40px',         # Padding inside the content box
    'box_shadow': '0 2px 6px rgba(0,0,0,0.1)',  # Box shadow
    'margin': '0 auto'         # Center horizontally, no top/bottom margin
}

# Global footer message (shown on all pages)
footer_message = "Made with 💙 in Colab · Music for your mood"

# ================================
# 🧱 Header and Footer Builders
# ================================

# 🚀 Header builder (dynamically customizable)
def app_header(text_color='white', font='Verdana', bg_color='#000000',
               text='🎵 My Music Recommender', align='center', size='24px'):
    return widgets.HTML(
        value=f"""
        <div style="display: flex; justify-content: center; margin-bottom: 10px;">
            <div style="
                background-color: {bg_color};
                color: {text_color};
                font-family: {font};
                font-size: {size};
                text-align: {align};
                padding: 12px;
                border-radius: {page_style['border_radius']};
                max-width: {page_style['max_width']};
                box-shadow: {page_style['box_shadow']};
                margin: {page_style['margin']};
                width: 100%;">
                {text}
            </div>
        </div>
        """
    )

# 🦶 Footer builder
def app_footer():
    return widgets.HTML(
        f"<hr><p style='color:gray;font-size:12px;text-align:center;'>{footer_message}</p>"
    )

# ================================
# 🧩 Page Layout Wrapping
# ================================

# 🧱 Full page wrapper (wraps content, header, footer with global styles)
def wrap_page(content_widgets):
    style_html = f"""
    <style>
        .custom-page {{
            background-color: {page_style['bg']};
            color: {page_style['text']};
            font-family: {page_style['font']};
            padding: {page_style['padding']};
            border-radius: {page_style['border_radius']};
            margin: {page_style['margin']};
            max-width: {page_style['max_width']};
            box-shadow: {page_style['box_shadow']};
        }}
    </style>
    """

    # Global header
    #header = app_header(
    #    text_color=page_style['text'],
    #    font=page_style['font'],
    #    bg_color=page_style['header_bg'],
    #    text='🎵 Welcome to the Music Recommender',
    #    align='center',
    #    size='20px'
    #)

    return widgets.VBox([
        widgets.HTML(value=style_html),
        #header,
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])

# ================================
# 🎯 Apply Styles to Widgets
# ================================

# 🎯 Apply styles to individual widgets (Button, Text, Dropdown, etc.)
def style_widgets(widgets_list):
    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = page_style['text']
            widget.style.font_weight = 'bold'
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px 12px',
                border_radius=page_style['border_radius']
            )
        elif isinstance(widget, widgets.Text):
            widget.layout = widgets.Layout(
                width='300px',
                padding='6px',
                border_radius=page_style['border_radius'],
                font_size='30px'
            )
        elif isinstance(widget, widgets.Password):
            widget.layout = widgets.Layout(
                width='300px',
                padding='6px',
                border_radius=page_style['border_radius'],
                font_size='14px'
            )
        elif isinstance(widget, widgets.Combobox):
            widget.layout = widgets.Layout(
                width='300px',
                padding='6px',
                border_radius=page_style['border_radius'],
                font_size='14px'
            )
        elif isinstance(widget, widgets.Dropdown):
            widget.layout = widgets.Layout(
                width='300px',
                padding='6px',
                border_radius=page_style['border_radius'],
                font_size='14px'
            )
        elif isinstance(widget, widgets.Checkbox):
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px',
                border_radius=page_style['border_radius']
            )

# 🎧 Global Footer Section
def app_footer():
    return widgets.HTML(value=f"<footer style='text-align: center; padding: 10px; color: #555;'>{footer_message}</footer>")

In [364]:
import ipywidgets as widgets
from IPython.display import display, HTML

# 🎨 Per-page background/text/font styles
page_styles = {
    'start':         {'bg': '#e3f2fd', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'signin':        {'bg': '#fce4ec', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'signup':        {'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'session':       {'bg': '#fffde7', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'edit_favorites':{'bg': '#ede7f6', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'default':       {'bg': '#ffffff', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'}
}

# 🧱 Header bar content per page
page_display_config = {
    'start':          {'header_text': '🎵 Welcome to the Music Recommender', 'header_align': 'center', 'header_size': '20px'},
    'signin':         {'header_text': '🔑 Sign In',                          'header_align': 'center', 'header_size': '20px'},
    'signup':         {'header_text': '🔐 Create Account',                   'header_align': 'center', 'header_size': '20px'},
    'session':        {'header_text': '🧠 Session Mode',                     'header_align': 'center', 'header_size': '20px'},
    'edit_favorites': {'header_text': '🎨 Edit Favorites',                   'header_align': 'center', 'header_size': '20px'},
    'default':        {'header_text': '🎶 Music App',                        'header_align': 'center', 'header_size': '20px'}
}

# 🧩 Optional inline box headers per page
page_box_header_config = {
    'signin':         {'show': True, 'text': '🔑 Sign In'},
    'signup':         {'show': True, 'text': '🔐 Create Account'},
    'edit_favorites': {'show': True,  'text': '🎨 Your Artists'},
    'session':        {'show': True, 'text': '🧠 Dashboard'},
    'default':        {'show': True, 'text': '🎶 Music UI'}
}

# 🧰 Widget appearance per page
page_widget_styles = {
    'default': {
        'button_color': '#333333',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '6px',
        'font_size': '14px',
        'input_padding': '10px',
    },
    'signin': {
        'button_color': '#333333',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '6px',
        'font_size': '14px',
        'input_padding': '10px',
    },
    'signup': {
        'button_color': '#333333',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '60px',
        'font_size': '14px',
        'input_padding': '10px',
    }
}

# 📦 Page box size + layout styling
page_box_shapes = {
    'default': {
        'max_width': '800px',
        'border_radius': '10px',
        'padding': '20px',
        'box_shadow': '0 2px 6px rgba(0,0,0,0.1)',
        'margin': 'auto'
    },
    'signin': {
        'max_width': '800px',
        'border_radius': '12px',
        'padding': '20px',
        'box_shadow': '0 2px 6px rgba(0,0,0,0.1)',
        'margin': 'auto'
    },
    'signup': {
        'max_width': '800px',
        'border_radius': '12px',
        'padding': '20px',
        'box_shadow': '0 0px 6px rgba(0,0,0,0.1)',
        'margin': 'auto'
    }
}

# 🦶 Global footer message
footer_message = "Made with 💙 in Colab · Music for your mood"

# 🚀 Top-wide header bar with box-aligned layout
def app_header(text_color='white', font='Verdana', bg_color='#000000',
               text='🎵 My Music Recommender', align='center', size='24px', page='default'):
    shape = page_box_shapes.get(page, page_box_shapes['default'])

    return widgets.HTML(
        value=f"""
        <div style="display: flex; justify-content: center; margin-bottom: 10px;">
            <div style="
                background-color: {bg_color};
                color: {text_color};
                font-family: {font};
                font-size: {size};
                text-align: {align};
                padding: 12px;
                border-radius: {shape['border_radius']};
                max-width: {shape['max_width']};
                box-shadow: {shape['box_shadow']};
                margin: {shape['margin']};
                width: 100%;">
                {text}
            </div>
        </div>
        """
    )

# 🧩 Optional inline header inside the content box
def get_box_header_widget(page='default'):
    config = page_box_header_config.get(page, {})
    if not config.get('show'):
        return None
    style = page_styles.get(page, page_styles['default'])
    return widgets.HTML(
        value=f"<h3 style='color:{style['text']}; font-family:{style['font']}; margin-top:0;'>{config['text']}</h3>"
    )

# 🧱 Full page wrapper
def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    header_cfg = page_display_config.get(page, page_display_config['default'])
    shape = page_box_shapes.get(page, page_box_shapes['default'])

    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: {shape['padding']};
            border-radius: {shape['border_radius']};
            margin: {shape['margin']};
            max-width: {shape['max_width']};
            box-shadow: {shape['box_shadow']};
        }}
        button {{
            font-family: {style['font']};
        }}
        input, select, textarea {{
            font-family: {style['font']};
            font-size: {page_widget_styles.get(page, page_widget_styles['default']).get('font_size', '14px')};
        }}
    </style>
    """

    header = app_header(
        text_color=style['text'],
        font=style['font'],
        bg_color=style.get('header_bg', '#ffffff'),
        text=header_cfg['header_text'],
        align=header_cfg['header_align'],
        size=header_cfg['header_size'],
        page=page
    )

    return widgets.VBox([
        widgets.HTML(value=style_html),
        header,
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])

# 🎯 Apply per-page styles to individual widgets
def style_widgets(widgets_list, page='default'):
    config = page_widget_styles.get(page, page_widget_styles['default'])

    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = config.get('button_color', '#333')
            widget.style.font_weight = 'bold'
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px 12px',
                border_radius=config.get('input_border_radius', '6px')
            )
        elif isinstance(widget, (widgets.Text, widgets.Password, widgets.Combobox, widgets.IntText, widgets.Dropdown)):
            widget.layout = widgets.Layout(
                width=config.get('input_width', '300px'),
                padding='6px',
                border_radius=config.get('input_border_radius', '6px'),
                font_size=config.get('font_size', '14px')
            )

# 🧩 Global Footer Section
def app_footer():
    return widgets.HTML(value=f"<footer style='text-align: center; padding: 10px; color: #555;'>{footer_message}</footer>")


In [365]:
import ipywidgets as widgets
from IPython.display import display, HTML

# 🎨 Background/text/font colors per page
page_styles = {
    'start':         {'bg': '#e3f2fd', 'text': '#000000', 'font': 'Verdana',      'header_bg': '#bbdefb'},
    'signin':        {'bg': '#fce4ec', 'text': '#880e4f', 'font': 'Georgia',      'header_bg': '#f8bbd0'},
    'signup':        {'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'session':       {'bg': '#fffde7', 'text': '#f57f17', 'font': 'Courier New',  'header_bg': '#ffe082'},
    'edit_favorites':{'bg': '#ede7f6', 'text': '#4a148c', 'font': 'Tahoma',       'header_bg': '#d1c4e9'},
    'default':       {'bg': '#ffffff', 'text': '#000000', 'font': 'sans-serif',   'header_bg': '#f0f0f0'}
}

# 🧱 Header bar content per page
page_display_config = {
    'start':          {'header_text': '🎵 Welcome to the Music Recommender', 'header_align': 'center', 'header_size': '20px'},
    'signin':         {'header_text': '🔑 Sign In',                          'header_align': 'center', 'header_size': '24px'},
    'signup':         {'header_text': '🔐 Create Account',                   'header_align': 'center', 'header_size': '22px'},
    'session':        {'header_text': '🧠 Session Mode',                     'header_align': 'center', 'header_size': '22px'},
    'edit_favorites': {'header_text': '🎨 Edit Favorites',                   'header_align': 'left',   'header_size': '22px'},
    'default':        {'header_text': '🎶 Music App',                        'header_align': 'center', 'header_size': '22px'}
}

# 🧩 Optional inline box headers per page
page_box_header_config = {
    'signin':         {'show': False, 'text': '🔑 Sign In'},
    'signup':         {'show': False, 'text': '🔐 Create Account'},
    'edit_favorites': {'show': True,  'text': '🎨 Your Artists'},
    'session':        {'show': False, 'text': '🧠 Dashboard'},
    'default':        {'show': False, 'text': '🎶 Music UI'}
}

# 🧰 Widget appearance per page
page_widget_styles = {
    'default': {
        'button_color': '#333333',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '6px',
        'font_size': '14px',
    },
    'signin': {
        'button_color': '#880e4f',
        'button_text_color': '#ffffff',
        'input_width': '350px',
        'input_border_radius': '8px',
        'font_size': '16px',
    },
    'signup': {
        'button_color': '#1b5e20',
        'button_text_color': '#ffffff',
        'input_width': '400px',
        'input_border_radius': '10px',
        'font_size': '15px',
    }
}

# 📦 Page box size + layout styling
page_box_shapes = {
    'default': {
        'max_width': '800px',
        'border_radius': '10px',
        'padding': '20px',
        'box_shadow': '0 2px 6px rgba(0,0,0,0.1)',
        'margin': 'auto'
    },
    'signin': {
        'max_width': '800px',
        'border_radius': '12px',
        'padding': '25px',
        'box_shadow': '0 0 10px rgba(136,14,79,0.25)',
        'margin': 'auto'
    },
    'signup': {
        'max_width': '2000px',
        'border_radius': '16px',
        'padding': '30px',
        'box_shadow': '0 0 12px rgba(27,94,32,0.25)',
        'margin': 'auto'
    }
}

# 🦶 Global footer message
footer_message = "Made with 💙 in Colab · Music for your mood"

# 🚀 Top-wide header bar with box-aligned layout
def app_header(text_color='white', font='Verdana', bg_color='#000000',
               text='🎵 My Music Recommender', align='center', size='24px', page='default'):
    shape = page_box_shapes.get(page, page_box_shapes['default'])

    return widgets.HTML(
        value=f"""
        <div style="display: flex; justify-content: center; margin-bottom: 10px;">
            <div style="
                background-color: {bg_color};
                color: {text_color};
                font-family: {font};
                font-size: {size};
                text-align: {align};
                padding: 12px;
                border-radius: {shape['border_radius']};
                max-width: {shape['max_width']};
                box-shadow: {shape['box_shadow']};
                margin: {shape['margin']};
                width: 100%;">
                {text}
            </div>
        </div>
        """
    )

# 🧩 Optional inline header inside the content box
def get_box_header_widget(page='default'):
    config = page_box_header_config.get(page, {})
    if not config.get('show'):
        return None
    style = page_styles.get(page, page_styles['default'])
    return widgets.HTML(
        value=f"<h3 style='color:{style['text']}; font-family:{style['font']}; margin-top:0;'>{config['text']}</h3>"
    )

# 🧱 Full page wrapper
def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    header_cfg = page_display_config.get(page, page_display_config['default'])
    shape = page_box_shapes.get(page, page_box_shapes['default'])

    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: {shape['padding']};
            border-radius: {shape['border_radius']};
            margin: {shape['margin']};
            max-width: {shape['max_width']};
            box-shadow: {shape['box_shadow']};
        }}
        button {{
            font-family: {style['font']};
        }}
        input, select, textarea {{
            font-family: {style['font']};
            font-size: {page_widget_styles.get(page, page_widget_styles['default']).get('font_size', '14px')};
        }}
    </style>
    """

    header = app_header(
        text_color=style['text'],
        font=style['font'],
        bg_color=style.get('header_bg', '#f0f0f0'),
        text=header_cfg['header_text'],
        align=header_cfg['header_align'],
        size=header_cfg['header_size'],
        page=page
    )

    return widgets.VBox([
        widgets.HTML(value=style_html),
        header,
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])

# 🎯 Apply per-page styles to individual widgets
def style_widgets(widgets_list, page='default'):
    config = page_widget_styles.get(page, page_widget_styles['default'])

    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = config.get('button_color', '#333')
            widget.style.font_weight = 'bold'
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px 12px',
                border_radius=config.get('input_border_radius', '6px')
            )
        elif isinstance(widget, (widgets.Text, widgets.Password, widgets.Combobox, widgets.IntText, widgets.Dropdown)):
            widget.layout = widgets.Layout(
                width=config.get('input_width', '300px'),
                padding='6px',
                border_radius=config.get('input_border_radius', '6px'),
                font_size=config.get('font_size', '14px')
            )


In [368]:
import ipywidgets as widgets
from IPython.display import display, HTML

# 🎨 Per-page background/text/font styles
page_styles = {
    'start':         {'bg': '#e3f2fd', 'text': '#000000', 'font': 'Verdana',      'header_bg': '#bbdefb'},
    'signin':        {'bg': '#50c5d3', 'text': '#ffffff', 'font': 'Georgia',      'header_bg': '#9f50d3'},
    'signup':        {'bg': '#e8f5e9', 'text': '#1b5e20', 'font': 'Verdana',      'header_bg': '#c8e6c9'},
    'session':       {'bg': '#fffde7', 'text': '#f57f17', 'font': 'Courier New',  'header_bg': '#ffe082'},
    'edit_favorites':{'bg': '#ede7f6', 'text': '#4a148c', 'font': 'Tahoma',       'header_bg': '#d1c4e9'},
    'default':       {'bg': '#ffffff', 'text': '#000000', 'font': 'sans-serif',   'header_bg': '#f0f0f0'}
}

# 🧱 Per-page header content
page_display_config = {
    'start':          {'header_text': '🎵 Welcome to the Music Recommender', 'header_align': 'center', 'header_size': '22px'},
    'signin':         {'header_text': '🔑 Sign In',                          'header_align': 'center', 'header_size': '22px'},
    'signup':         {'header_text': '🔐 Create Account',                   'header_align': 'center', 'header_size': '22px'},
    'session':        {'header_text': '🧠 Session Mode',                     'header_align': 'center', 'header_size': '22px'},
    'edit_favorites': {'header_text': '🎨 Edit Favorites',                   'header_align': 'left',   'header_size': '22px'},
    'default':        {'header_text': '🎶 Music App',                        'header_align': 'center', 'header_size': '22px'}
}

# 🧩 Per-page widget appearance
page_widget_styles = {
    'default': {
        'button_color': '#000000',
        'button_text_color': '#ffffff',
        'input_width': '300px',
        'input_border_radius': '6px',
        'font_size': '14px',
    },
    'signin': {
        'button_color': '#000000',
        'button_text_color': '#ffffff',
        'input_width': '350px',
        'input_border_radius': '8px',
        'font_size': '16px',
    },
    'signup': {
        'button_color': '#000000',
        'button_text_color': '#ffffff',
        'input_width': '400px',
        'input_border_radius': '10px',
        'font_size': '15px',
    }
}

# 📦 Per-page box shapes & layout
page_box_shapes = {
    'default': {
        'max_width': '800px',
        'border_radius': '0px',
        'padding': '20px',
        'box_shadow': '0 2px 6px rgba(0,0,0,0.1)',
        'margin': 'auto'
    },
    'signin': {
        'max_width': '1000px',
        'border_radius': '12px',
        'padding': '25px',
        'box_shadow': '0 0 10px rgba(136,14,79,0.25)',
        'margin': 'auto'
    },
    'signup': {
        'max_width': '600px',
        'border_radius': '16px',
        'padding': '5px',
        'box_shadow': '0 0 12px rgba(27,94,32,0.25)',
        'margin': 'auto'
    }
}

# 🦶 Footer message (shown on all pages)
footer_message = "Made with 💙 in Colab · Music for your mood"

# 🚀 Header builder
def app_header(text_color='white', font='Verdana', bg_color='#000000', text='🎵 My Music Recommender', align='center', size='24px'):
    return widgets.HTML(
        value=f"""
        <div style="
            background-color: {bg_color};
            color: {text_color};
            font-family: {font};
            font-size: {size};
            text-align: {align};
            padding: 5px;
            border-radius: 0px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.1);">
            {text}
        </div>
        """
    )

# 🦶 Footer builder
def app_footer():
    return widgets.HTML(
        f"<hr><p style='color:gray;font-size:12px;text-align:center;'>{footer_message}</p>"
    )

# 🧱 Full page wrapper
def wrap_page(content_widgets, page='default'):
    style = page_styles.get(page, page_styles['default'])
    header_cfg = page_display_config.get(page, page_display_config['default'])
    shape = page_box_shapes.get(page, page_box_shapes['default'])

    style_html = f"""
    <style>
        .custom-page {{
            background-color: {style['bg']};
            color: {style['text']};
            font-family: {style['font']};
            padding: {shape['padding']};
            border-radius: {shape['border_radius']};
            margin: {shape['margin']};
            max-width: {shape['max_width']};
            box-shadow: {shape['box_shadow']};
        }}
        button {{
            font-family: {style['font']};
        }}
        input, select, textarea {{
            font-family: {style['font']};
            font-size: {page_widget_styles.get(page, page_widget_styles['default']).get('font_size', '14px')};
        }}
    </style>
    """

    header = app_header(
        text_color=style['text'],
        font=style['font'],
        bg_color=style.get('header_bg', '#f0f0f0'),
        text=header_cfg['header_text'],
        align=header_cfg['header_align'],
        size=header_cfg['header_size']
    )

    return widgets.VBox([
        widgets.HTML(value=style_html),
        header,
        widgets.VBox(content_widgets, layout=widgets.Layout(padding='10px'), _dom_classes=['custom-page']),
        app_footer()
    ])

# 🎯 Apply per-page styles to individual widgets
def style_widgets(widgets_list, page='default'):
    config = page_widget_styles.get(page, page_widget_styles['default'])

    for widget in widgets_list:
        if isinstance(widget, widgets.Button):
            widget.style.button_color = config.get('button_color', '#333')
            widget.style.font_weight = 'bold'
            widget.layout = widgets.Layout(
                width='auto',
                padding='6px 12px',
                border_radius=config.get('input_border_radius', '6px')
            )
        elif isinstance(widget, (widgets.Text, widgets.Password, widgets.Combobox, widgets.IntText, widgets.Dropdown)):
            widget.layout = widgets.Layout(
                width=config.get('input_width', '300px'),
                padding='6px',
                border_radius=config.get('input_border_radius', '6px'),
                font_size=config.get('font_size', '14px')
            )

In [369]:
start_music_recommender()

✅ Multi-page interface with updated favorites-recommendation link loaded.


# aa

In [ ]:
DEFAULT_STYLE = {
    "background_color": "#00bcd4",
    "text_color": "#000000",
    "font_family": "monospace",
    "header_color": "#a6ff00",
    "max_width": 400,
    "border_radius": 10,
    "padding": 40,
    "box_shadow": "2px 2px 6px rgba(0,0,0,0.2)",
    "footer_message": "Made with ❤️ in Colab",
    "button_font_size": "13px",
    "button_text_color": "#000000",
    "button_hover_color": "#8aff00"
}
def apply_default_styles(style_dict):
    for key, value in DEFAULT_STYLE.items():
        if key not in style_dict:
            style_dict[key] = value
    return style_dict

def style_widget(widget, class_name=None, layout_dict=None):
    if layout_dict:
        for key, value in layout_dict.items():
            setattr(widget.layout, key, value)
    if class_name:
        widget.add_class(class_name)

def apply_styles_to_widget_list(widgets_list, class_name="styled-button", layout_overrides=None):
    layout_overrides = layout_overrides or {}
    for widget in widgets_list:
        layout = layout_overrides.get(widget.description, {})
        style_widget(widget, class_name=class_name, layout_dict=layout)
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_styled_page(title, icon, content_widgets, style_dict=None, footer_text=None):
    style_dict = apply_default_styles(style_dict or {})

    style_html = widgets.HTML(f"""
    <style>
    .container {{
      background-color: {style_dict['background_color']};
      padding: {style_dict['padding']}px;
      border-radius: {style_dict['border_radius']}px;
      box-shadow: {style_dict['box_shadow']};
      text-align: center;
      max-width: {style_dict['max_width']}px;
      margin: auto;
    }}
    .header-box {{
      background-color: {style_dict['header_color']};
      color: {style_dict['text_color']};
      padding: 5px;
      font-family: {style_dict['font_family']};
      font-size: 15px;
    }}
    .footer-box {{
      color: {style_dict['text_color']};
      text-align: center;
      padding: 10px;
      opacity: 0.7;
      margin-top: 20px;
    }}
    .styled-button {{
      background-color: {style_dict['header_color']};
      color: {style_dict['button_text_color']};
      font-family: {style_dict['font_family']};
      font-size: {style_dict['button_font_size']};
      border-radius: 5px;
      padding: 6px 12px;
    }}
    .styled-button:hover {{
      background-color: {style_dict['button_hover_color']};
    }}
    </style>
    """)

    header = widgets.HTML(f"{icon} {title}")
    header_box = widgets.VBox([header])
    header_box.add_class("header-box")

    content_box = widgets.VBox(content_widgets)
    content_box.layout.justify_content = 'center'

    footer = widgets.HTML(footer_text or style_dict['footer_message'])
    footer_box = widgets.VBox([footer])
    footer_box.add_class("footer-box")

    container = widgets.VBox([header_box, content_box])
    container.add_class("container")

    return widgets.VBox([style_html, container, footer_box])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################

In [ ]:
user_manager = UserManager(users_csv)
emotion_service = EmotionService()
recommender = RecommendationService(songs_csv, history_csv, users_csv)
logger = LoggingService(history_csv)

artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

session = {
    'user_id': None,
    'username': '',
    'mode': 'none',
    'current_page': 'start'
}

music_recommender_output = widgets.Output()

In [ ]:
def show_status(message, status='info'):
    colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
    with music_recommender_output:
        clear_output()
        if message:
            display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

def switch_to_page(page):
    session['current_page'] = page
    show_status("")
    if page == 'start':
        show_start_page()
    elif page == 'signin':
        show_signin_page()
    elif page == 'signup':
        show_signup_page()
    elif page == 'session':
        show_session_page()
    elif page == 'edit_favorites':
        show_edit_favorites_page()
    else:
        with music_recommender_output:
            clear_output()
            print(f"❌ Unknown page: {page}")